In [1]:
%pip install --quiet --upgrade pip
%pip install --quiet -U -r requirements.txt

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%load_ext autoreload
%autoreload 2

--- 

# Prepare Collection

In [18]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from movies import MovieCollection
from features import FeatureExtractor
from helpers import display_vocabulary, display_md
from tqdm.notebook import tqdm

# Create widgets for form inputs
stemming_checkbox = widgets.Checkbox(
    value=True,
    description='Use Stemming',
)

stopwords_checkbox = widgets.Checkbox(
    value=True, # chatgpt change to True 
    description='Remove Stopwords',
)

number_of_records = widgets.IntSlider(
    value=3000,
    min=1000,
    max=50000,
    step=1000,
    description='Number of Records:'
)

def create_collection():
    global collection, pipeline, features_set, vocabulary_df, vocabulary_idf, features_bag, features_tfidf
    
    clear_output(wait=True)
    display_md("### Creating Movies Collection")
    collection = MovieCollection(number_of_records.value)
    pipeline = FeatureExtractor(stemming=stemming_checkbox.value, stopwords=stopwords_checkbox.value)
    display_md("- extracting 'bag of words' features")
    features_bag = {movie.imdb_id: pipeline.bag_of_words(movie.to_text()) for movie in tqdm(collection.movies)}
    display_md("- extracting 'set of words' features")
    features_set = {id: set(bag_of_words.keys()) for (id, bag_of_words) in features_bag.items()}
    display_md("- building the vocabulary")
    vocabulary_df = pipeline.df(features_set.values())
    vocabulary_idf = pipeline.idf(features_set.values())
    display_md("- extracting 'tfidf' features")
    features_tfidf = {id: pipeline.tfidf(bag_of_words, vocabulary_idf) for (id, bag_of_words) in features_bag.items()}
    display_vocabulary(vocabulary_df, vocabulary_idf, n_samples=20)

index_button = widgets.Button(description="Create Collection", button_style='primary')
index_button.on_click(lambda _: create_collection())

# Create form container
form = widgets.VBox([widgets.VBox([number_of_records, stemming_checkbox, stopwords_checkbox]), index_button])

# Display the form
display(form)

### Creating Movies Collection

- reading dataset from local cache (./data/movie_dataset.jsonl)

- extracting 'bag of words' features

  0%|          | 0/3000 [00:00<?, ?it/s]

- extracting 'set of words' features

- building the vocabulary

- extracting 'tfidf' features

|token|df|idf|
|-|-|-|
|drama|1606|0.6242432019127342|
|fox|28|4.639071737663773|
|malkovich|15|5.233778845410465|
|mous|10|5.608472294851876|
|mischiev|6|6.060457418594933|
|nostalg|4|6.396929655216146|
|marwood|3|6.620073206530356|
|underdog|3|6.620073206530356|
|notr|2|6.907755278982137|
|jeann|2|6.907755278982137|
|wedman|1|7.313220387090301|
|dyke|1|7.313220387090301|
|teammat|1|7.313220387090301|
|alquist|1|7.313220387090301|
|momentum|1|7.313220387090301|
|araki|1|7.313220387090301|
|mcclaren|1|7.313220387090301|
|soulless|1|7.313220387090301|
|bermuda|1|7.313220387090301|
|gorder|1|7.313220387090301|
|abolitionist|1|7.313220387090301|


---

# Building the index

In [8]:
from boolean import BooleanRetriever

index = BooleanRetriever(features_set)
index.n_docs

2999

**Test Query**

In [24]:
# query = "star wars"
# query = "sergio leone"
# query = "What are Western Movies from or like Sergio Leone?"
# query = "sergio leone western"
query = "Movies like 'The good the bad and the ugly'"

# use the following for more precision
# result = index.query_and(pipeline.set_of_words(query))
# chatgpt suggestion for query02
result = index.query_or(pipeline.set_of_words(query))

collection.display_result(result)

|id|title|overview|cast
|-|-|-|-|
|tt0114885|**Waiting to Exhale** (1995)|**Friends are the people who let you be yourself... and never let you forget it.**<br/>[Comedy, Drama, Romance]<br/><br/>Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusive "good man" to break a string of less-than-stellar lovers. Friends and confidants Vannah, Bernie, Glo and Robin talk it all out, determined to find a better way to breathe.|Angela Bassett as Bernadine 'Bernie' Harris, Mykelti Williamson as Troy, Wesley Snipes as James Wheeler
|tt0113041|**Father of the Bride Part II** (1995)|**Just When His World Is Back To Normal... He's In For The Surprise Of His Life!**<br/>[Comedy]<br/><br/>Just when George Banks has recovered from his daughter's wedding, he receives the news that she's pregnant ... and that George's wife, Nina, is expecting too. He was planning on selling their home, but that's a plan that -- like George -- will have to change with the arrival of both a grandchild and a kid of his own.|Steve Martin as George Banks, Diane Keaton as Nina Banks, Eugene Levy as Mr. Habib
|tt0114319|**Sabrina** (1995)|**You are cordially invited to the most surprising merger of the year.**<br/>[Comedy, Romance]<br/><br/>An ugly duckling having undergone a remarkable change, still harbors feelings for her crush: a carefree playboy, but not before his business-focused brother has something to say about it.|Harrison Ford as Linus Larrabee, John Wood as Fairchild, Dana Ivey as Mack, Paul Giamatti as Scott, Margo Martindale as Nurse
|tt0112302|**Tom and Huck** (1995)|**The Original Bad Boys.**<br/>[Action, Adventure, Drama, Family]<br/><br/>A mischievous young boy, Tom Sawyer, witnesses a murder by the deadly Injun Joe. Tom becomes friends with Huckleberry Finn, a boy with no future and no family. Tom has to choose between honoring a friendship or honoring an oath because the town alcoholic is accused of the murder. Tom and Huck go through several adventures trying to retrieve evidence.|
|tt0113161|**Get Shorty** (1995)|**The mob is tough, but it’s nothing like show business.**<br/>[Comedy, Thriller, Crime]<br/><br/>Chili Palmer is a Miami mobster who gets sent by his boss, the psychopathic "Bones" Barboni, to collect a bad debt from Harry Zimm, a Hollywood producer who specializes in cheesy horror films. When Chili meets Harry's leading lady, the romantic sparks fly. After pitching his own life story as a movie idea, Chili learns that being a mobster and being a Hollywood producer really aren't all that different.|John Travolta as Chili Palmer, Gene Hackman as Harry Zimm, Rene Russo as Karen Flores, Danny DeVito as Martin Weir, Dennis Farina as Ray 'Bones' Barboni, Delroy Lindo as Bo Catlett, James Gandolfini as Bear, David Paymer as Leo Devoe
|tt0112431|**Babe** (1995)|**A little pig goes a long way.**<br/>[Fantasy, Drama, Comedy, Family]<br/><br/>Babe is a little pig who doesn't quite know his place in the world. With a bunch of odd friends, like Ferdinand the duck who thinks he is a rooster and Fly the dog he calls mom, Babe realizes that he has the makings to become the greatest sheep pig of all time, and Farmer Hogget knows it. With the help of the sheep dogs Babe learns that a pig can be anything that he wants to be.|Miriam Margolyes as Fly the Female Sheepdog (voice), James Cromwell as Farmer Arthur Hoggett, Roscoe Lee Browne as Narrator (voice)
|tt0113855|**Mortal Kombat** (1995)|**Nothing In This World Has Prepared You For This.**<br/>[Action, Fantasy]<br/><br/>For nine generations an evil sorcerer has been victorious in hand-to-hand battle against his mortal enemies. If he wins a tenth Mortal Kombat tournament, desolation and evil will reign over the multiverse forever. To save Earth, three warriors must overcome seemingly insurmountable odds, their own inner demons, and superhuman foes in this action/adventure movie based on one of the most popular video games of all time.|Bridgette Wilson as Sonya Blade
|tt0116126|**Don't Be a Menace to South Central While Drinking Your Juice in the Hood** (1996)|**Finally, the movie that proves that Justice isn't always Poetic, Jungle Fever isn't always pretty, and Higher Learning can be a waste of time.**<br/>[Comedy]<br/><br/>When Ashtray (Shawn Wayans) moves to South Central L.A. to live with his father (who appears to be the same age he is) and grandmother (who likes to talk tough and smoke reefer), he falls in with his gang-banging cousin Loc Dog (Marlon Wayans), who along with the requisite pistols and Uzi carries a thermo-nuclear warhead for self-defense. Will Ashtray be able to keep living the straight life?|Vivica A. Fox as Ashtray's Mother, Bernie Mac as Officer Self Hatred
|tt0114753|**Two Bits** (1995)|**You're never too old to believe in a dream. Or too young to make one come true!**<br/>[Drama, Family]<br/><br/>It's a hot summer day in 1933 in South Philly, where 12-year old Gennaro lives with his widowed mom and his ailing grandpa, who sits outside holding tight to his last quarter, which he's promised to Gennaro and which Gennaro would like to have to buy a ticket to the plush new movie theater. But grandpa's not ready to pass on the quarter or pass on to his final reward: he has some unfinished business with a woman from his past, and he enlists Gennaro to act as his emissary.|Al Pacino as Gitano Sabatoni, Joe Grifasi as Uncle Joe
|tt0113537|**Kicking and Screaming** (1995)|**Anxiety loves company.**<br/>[Comedy, Drama, Romance]<br/><br/>After college graduation, Grover's girlfriend Jane tells him she's moving to Prague to study writing. Grover declines to accompany her, deciding instead to move in with several friends, all of whom can't quite work up the inertia to escape their university's pull. Nobody wants to make any big decisions that would radically alter his life, yet none of them wants to end up like Chet, the professional student who tends bar and is in his tenth year of university studies.|Eric Stoltz as Chet, Parker Posey as Miami, Elliott Gould as Grover's Dad
|tt0116731|**The Juror** (1996)|**There is no defense.**<br/>[Drama, Thriller]<br/><br/>With his gangster boss on trial for murder, a mob thug known as "the Teacher" tells Annie Laird she must talk her fellow jurors into a not-guilty verdict, implying that he'll kill her son Oliver if she fails. She manages to do this, but, when it becomes clear that the mobsters might want to silence her for good, she sends Oliver abroad and tries to gather evidence of the plot against her, setting up a final showdown.|Demi Moore as Annie Laird, Alec Baldwin as Teacher, Anne Heche as Juliet, James Gandolfini as Eddie
|tt0114039|**Once Upon a Time... When We Were Colored** (1996)|[Romance, Drama]<br/><br/>This film relates the story of a tightly connected Afro-American community informally called Colored Town where the inhabitants live and depend on each other in a world where racist oppression is everywhere, as told by a boy called Cliff who spent his childhood there. Despite this, we see the life of the community in all its joys and sorrows, of those that live there while others decide to leave for a better life north. For those remaining, things come to a serious situation when one prominent businessman is being muscled out by a white competitor using racist intimidation. In response, the community must make the decision of whether to submit meekly like they always have, or finally fight for their rights.|
|tt0113490|**The Journey of August King** (1995)|[Drama]<br/><br/>The Journey of August King is a multi-dimensional drama about a North Carolina farmer in 1815. August King, a widower, is on his way home as he does every year after selling his produce and purchasing the stock and goods he will need to survive the winter. On his journey, he comes upon a run-away slave, a young woman about 19 and August King must decide to violate the law and help this slave to freedom or else leave her to be hunted down and, ultimately, returned to her slave owner.|
|tt0115639|**Beautiful Girls** (1996)|**Good times never seemed so good,**<br/>[Comedy, Drama, Romance]<br/><br/>During a snowy winter in the small fictional town of Knight"s Ridge, Massachusetts, a group of lifelong buddies hang out, drink and struggle to connect with the women who affect their decisions, dreams and desires.|Matt Dillon as Tommy 'Birdman' Rowland, Uma Thurman as Andera, Mira Sorvino as Sharon Cassidy, Michael Rapaport as Paul Kirkwood, Rosie O'Donnell as Gina Barrisano, Max Perlich as Kev, Martha Plimpton as Jan, David Arquette as Bobby Conway, John Carroll Lynch as Frank Womack
|tt0113283|**Heidi Fleiss: Hollywood Madam** (1995)|**   **<br/>[Documentary]<br/><br/>A documentary crew from the BBC arrives in L.A. intent on interviewing Heidi Fleiss, a year after her arrest for running a brothel but before her trial. Several months elapse before the interview, so the crew searches for anyone who'll talk about the young woman. Two people have a lot to say to the camera: a retired madam named Alex for whom Fleiss once worked and Fleiss's one-time boyfriend, Ivan Nagy, who introduced her to Alex. Alex and Nagy don't like each other, so the crew shuttles between them with "she said" and "he said." When they finally interview Fleiss, they spend their time reciting what Alex and Nagy have had to say and asking her reaction.|
|tt0112556|**Happiness Is in the Field** (1995)|[Comedy]<br/><br/>Francis, the boss of a small plumbing supply company, is harassed by tax collectors, striking employees, and an impossible wife and daughter. His only joy is sharing lunch with his friend Gerard. Then a TV show called "where are you?" shows a woman from Gers who is searching for her husband who disappeared 28 years ago. The lost husband looks like an identical twin of Francis...|
|tt0112373|**Anne Frank Remembered** (1995)|**She is perhaps Hitler's best known victim, but what was Anne Frank really like?**<br/>[Documentary, Drama, Foreign]<br/><br/>Using previously unreleased archival material in addition to contemporary interviews, this academy award-winning documentary tells the story of the Frank family and presents the first fully-rounded portrait of their brash and free-spirited daughter Anne, perhaps the world's most famous victim of the Holocaust. Written by Dawn M. Barclift|Kenneth Branagh as Narration (voice), Glenn Close as Diary Readings (voice)
|tt0112442|**Bad Boys** (1995)|**Whatcha gonna do?**<br/>[Action, Comedy, Crime, Thriller]<br/><br/>Marcus Burnett is a hen-pecked family man. Mike Lowry is a foot-loose and fancy free ladies' man. Both are Miami policemen, and both have 72 hours to reclaim a consignment of drugs stolen from under their station's nose. To complicate matters, in order to get the assistance of the sole witness to a murder, they have to pretend to be each other.|Tchéky Karyo as Fouchet, Joe Pantoliano as Captain C. Howard, Michael Imperioli as Jojo, Anna Levine as Francine, Kevin Corrigan as Elliot
|tt0112541|**Blue in the Face** (1995)|**Welcome to the planet Brooklyn.**<br/>[Comedy]<br/><br/>Auggie runs a small tobacco shop in Brooklyn, New York. The whole neighborhood comes to visit him to buy cigarettes and have some small talk. During the movie Lou Reed tries to explain why he has to have a cut on his health insurance bill if he keeps smoking and Madonna acts as a Singing Telegram.|Harvey Keitel as Auggie Wren, Michael J. Fox as Pete Maloney, Lily Tomlin as Waffle eater, Giancarlo Esposito as Tommy Finelli, Mira Sorvino as Young lady, Keith David as Jackie Robinson, Victor Argo as Vinnie, Jared Harris as Jimmy Rose
|tt0109508|**Crumb** (1994)|**Weird sex · Obsession · Comic books**<br/>[Documentary]<br/><br/>This movie chronicles the life and times of R. Crumb. Robert Crumb is the cartoonist/artist who drew Keep On Truckin', Fritz the Cat, and played a major pioneering role in the genesis of underground comix. Through interviews with his mother, two brothers, wife, and ex-girlfriends, as well as selections from his vast quantity of graphic art, we are treated to a darkly comic ride through one man's subconscious mind.|
|tt0113677|**Living in Oblivion** (1995)|**Nick is about to discover the first rule of filmmaking: if at first you don't succeed... PANIC!**<br/>[Drama, Comedy]<br/><br/>Film about filmmaking. It takes place during one day on set of non-budget movie. Ultimate tribute to all independent filmmakers.|Steve Buscemi as Nick Reve, Dermot Mulroney as Wolf, James Le Gros as Chad Palomino, Kevin Corrigan as Assistant Camera
|tt0113820|**Mighty Morphin Power Rangers: The Movie** (1995)|**The Power Is On!**<br/>[Action, Adventure, Science Fiction, Family, Fantasy]<br/><br/>Power up with six incredible teens who out-maneuver and defeat evil everywhere as the Mighty Morphin Power Ranger, But this time the Power Rangers may have met their match, when they face off with the most sinister monster the galaxy has ever seen.|
|tt0110620|**Nadja** (1994)|**Unseen. Unforgiving. Undead.**<br/>[Horror, Thriller]<br/><br/>This ultra-hip, post-modern vampire tale is set in contemporary New York City. Members of a dysfunctional family of vampires are trying to come to terms with each other, in the wake of their father's death. Meanwhile, they are being hunted by Dr. Van Helsing and his hapless nephew. As in all good vampire movies, forces of love are pitted against forces of destruction.|Jared Harris as Edgar, Martin Donovan as Jim
|tt0114095|**Party Girl** (1995)|**Sassy, savvy and definitely clued-in!**<br/>[Comedy]<br/><br/>Although Mary has little income, she still finds ways to spend her nights at clubs. After being arrested for throwing an illegal rave, she asks her aunt Judy for bail money. Judy then finds Mary a job at her library so that Mary can repay her. Initially, Mary finds the job as a clerk boring and stifling, and prefers to get to know a street food vendor whom she likes. However, Mary must refocus her life once she loses her job and apartment.|Parker Posey as Mary, Liev Schreiber as Nigel
|tt0114496|**Something to Talk About** (1995)|**A story about husbands, wives, parents, children and other natural disasters.**<br/>[Comedy]<br/><br/>In one of her best-ever roles, Julia Roberts is Grace, whose reaction to the infidelities of Eddie (Dennis Quaid) turns the lives and loves of the people around her into something like falling dominoes. Robert Duvall, Gena Rowlands, Kyra Sedgwick and others in "the year's best ensemble of characters" (Jack Matthews, 'Newsday') co-star in this juicy, truthful story written by Callie Khouri|Julia Roberts as Grace King Bichon, Dennis Quaid as Eddie Bichon, Robert Duvall as Wyly King, Gena Rowlands as Georgia King
|tt0114558|**Strange Days** (1995)|**You know you want it.**<br/>[Crime, Drama, Science Fiction, Thriller]<br/><br/>Set in the year 1999 during the last days of the old millennium, the movie tells the story of Lenny Nero, an ex-cop who now deals with data-discs containing recorded memories and emotions. One day he receives a disc which contains the memories of a murderer killing a prostitute. Lenny investigates and is pulled deeper and deeper in a whirl of blackmail, murder and rape. Will he survive and solve the case?|Angela Bassett as Lornette 'Mace' Mason, Tom Sizemore as Max Peltier, Juliette Lewis as Faith Justin, Vincent D'Onofrio as Burton Steckler, Michael Wincott as Philo Gant, Richard Edson as Tick
|tt0112679|**Circle of Friends** (1995)|[Drama, Family, Romance]<br/><br/>'Circle Of Friends' is set in 1950's Ireland. The movie focuses on Benny Hogan and her best friend, Eve Malone. The story centers around Benny and Eve as they enter student life at University College, Dublin. Here Benny and Eve reunite with their childhood friend, the ice-cool Nan Mahon, the 'college belle'. They also encounter the handsome and charming Jack Foley, whom Benny quickly falls for.|Chris O'Donnell as Jack Foley, Minnie Driver as Bernadette 'Benny' Hogan, Alan Cumming as Sean Walsh
|tt0109445|**Clerks** (1994)|**Just because they serve you doesn't mean they like you.**<br/>[Comedy]<br/><br/>Convenience and video store clerks Dante and Randal are sharp-witted, potty-mouthed and bored out of their minds. So in between needling customers, the counter jockeys play hockey on the roof, visit a funeral home and deal with their love lives.|
|tt0109642|**Dolores Claiborne** (1995)|**Sometimes, an accident can be an unhappy woman's best friend.**<br/>[Crime, Drama, Mystery]<br/><br/>Dolores Claiborne was accused of killing her abusive husband twenty years ago, but the court's findings were inconclusive and she was allowed to walk free. Now she has been accused of killing her employer, Vera Donovan, and this time there is a witness who can place her at the scene of the crime. Things look bad for Dolores when her daughter Selena, a successful Manhattan magazine writer, returns to cover the story.|Kathy Bates as Dolores Claiborne, Jennifer Jason Leigh as Selena St. George, Christopher Plummer as John Mackey, David Strathairn as Joe St. George, John C. Reilly as Const. Frank Stamshaw, Bob Gunton as Mr. Pease
|tt0109707|**Ed Wood** (1994)|**When it came to making bad movies, Ed Wood was the best.**<br/>[Comedy, Drama, History]<br/><br/>The mostly true story of the legendary "worst director of all time", who, with the help of his strange friends, filmed countless B-movies without ever becoming famous or successful.|Johnny Depp as Ed Wood, Martin Landau as Bela Lugosi, Sarah Jessica Parker as Dolores Fuller, Patricia Arquette as Kathy O'Hara, Jeffrey Jones as Criswell, Vincent D'Onofrio as Orson Welles, Bill Murray as Bunny Breckinridge, Mike Starr as Georgie Weiss, Rance Howard as Old Man McCoy
|tt0113198|**A Goofy Movie** (1995)|**It's the story of a father who couldn't be closer...to driving his son crazy.**<br/>[Romance, Animation, Family, Comedy, Adventure]<br/><br/>Though Goofy always means well, his amiable cluelessness and klutzy pratfalls regularly embarrass his awkward adolescent son, Max. When Max's lighthearted prank on his high-school principal finally gets his longtime crush, Roxanne, to notice him, he asks her on a date. Max's trouble at school convinces Goofy that he and the boy need to bond over a cross-country fishing trip like the one he took with his dad when he was Max's age, which throws a kink in his son's plans to impress Roxanne.|Jim Cummings as Peter Pete (voice), Wallace Shawn as Principal Arthur Mazur (voice), Frank Welker as Bigfoot (voice)
|tt0113089|**Fluke** (1995)|**Unleash your imagination... and come along on a magical journey. Follow him home.**<br/>[Drama, Family, Fantasy]<br/><br/>Workaholic Thomas Johnson dies in an auto accident and comes back to life as a dog. Remembering some of who he was, he returns to his wife and son to protect them from the man who caused his accident. But, as time goes by, he remembers more of his life, and realizes he wasn't such a good husband and father.|Samuel L. Jackson as Rumbo (voice), Matthew Modine as Thomas P. Johnson / Voice of Fluke, Eric Stoltz as Jeff Newman, Ron Perlman as Sylvester, Jon Polito as Boss, Bill Cobbs as Bert
|tt0113234|**Gumby: The Movie** (1995)|**The original green hero!**<br/>[Fantasy, Animation, Science Fiction, Family]<br/><br/>The band is back together! Gumby reunites with The Clayboys to perform at a concert benefiting local farmers. But things take an unexpected turn when Gumby s dog, Lowbelly, reacts to the music by crying tears of real pearl! Fortune turns into disaster as Gumby s archenemies, the Blockheads, devise an elaborate scheme to dognap Lowbelly and harvest her pearls for themselves. When the Blockheads initial plan fails, they kidnap The Clayboys as well...and replace them with clones! The battle between Clayboys and clones is filled with trains and planes, knights and fights, thrills and spills. True to classic Gumby adventures, Gumby: The Movie takes viewers in and out of books, to Toyland, Camelot, outer space and beyond!|
|tt0110116|**Immortal Beloved** (1994)|[Drama, Music, Romance]<br/><br/>The life and death of the legendary Ludwig van Beethoven. Beside all the work he is known for, the composer once wrote a famous love letter to a nameless beloved and the movie tries to find out who this beloved was. Not easy as Beethoven has had many women in his life.|Gary Oldman as Ludwig van Beethoven, Miriam Margolyes as Nanette Streicherová
|tt0110006|**Heavyweights** (1995)|**They don't run the fastest. They don't jump the highest. But they sure are getting the last laugh. Heavyweights. They never met a hot dog they didn't like ... until now.**<br/>[Comedy, Family, Drama]<br/><br/>Camp Hope is a summer retreat for overweight boys run by a kindly couple who make the campers feel comfortable with their extra pounds. But when tyrannical fitness guru Tony buys the camp, he puts the kids on a cruel regimen that goes too far. Sick of the endless weeks of "all work and no play," the kids stage a coup and reclaim their summer of fun.|Ben Stiller as Tony Perkis, Jeffrey Tambor as Maury Garner
|tt0103994|**Like Water for Chocolate** (1992)|**In Tita's kitchen, ordinary spices become a recipe for passion.**<br/>[Drama, Romance]<br/><br/>Tita is passionately in love with Pedro, but her controlling mother forbids her from marrying him. When Pedro marries her sister, Tita throws herself into her cooking and discovers she can transfer her emotions through the food she prepares, infecting all who eat it with her intense heartbreak.|
|tt0110443|**Major Payne** (1995)|**He's looking for a few good men... or a few guys old enough to shave.**<br/>[Adventure, Comedy, Family]<br/><br/>Major Benson Winifred Payne is being discharged from the Marines. Payne is a killin' machine, but the wars of the world are no longer fought on the battlefield. A career Marine, he has no idea what to do as a civilian, so his commander finds him a job - commanding officer of a local school's JROTC program, a bunch of ragtag losers with no hope.|Michael Ironside as Lt. Col. Stone
|tt0113691|**Losing Isaiah** (1995)|**Who decides what makes a mother?**<br/>[Drama]<br/><br/>Khaila Richards, a crack-addicted single mother, accidentally leaves her baby in a dumpster while high and returns the next day in a panic to find he is missing. In reality, the baby has been adopted by a warm-hearted social worker, Margaret Lewin, and her husband, Charles. Years later, Khaila has gone through rehab and holds a steady job. After learning that her child is still alive, she challenges Margaret for the custody.|Jessica Lange as Margaret Lewin, Halle Berry as Khaila Richards, David Strathairn as Charles Lewin, Samuel L. Jackson as Kadar Lewis, Cuba Gooding Jr. as Eddie Hughes
|tt0113755|**Man of the House** (1995)|**Jack wants to marry Ben's mother. But there are strings attached.**<br/>[Action, Comedy, Family]<br/><br/>Ben Archer is not happy. His mother, Sandy, has just met a man, and it looks like things are pretty serious. Driven by a fear of abandonment, Ben tries anything and everything to ruin the "love bubble" which surrounds his mom. However, after Ben and Jack's experiences in the Indian Guides, the two become much closer.|Chevy Chase as Jack Sturgess (Squatting Dog), George Wendt as Chet Bronski (Chief Running Horse)
|tt0110516|**Milk Money** (1994)|**You can't get enough of a good thing... But first you have to find it.**<br/>[Comedy, Romance]<br/><br/>Three young boys pool their money and pay V, a kindhearted prostitute, to strip for them. Afterward, she drives them home to the suburbs -- but then her car breaks down. It's just as well, though, because a mobster named Waltzer is after her, and V realizes the suburbs are the perfect place to hide. But things get a lot more complicated when V falls in love with Tom, a single father who is unaware of her real profession.|Melanie Griffith as V, Ed Harris as Tom Wheeler, Malcolm McDowell as Waltzer, Anne Heche as Betty, Philip Bosco as Jerry the Pope
|tt0113808|**Miami Rhapsody** (1995)|**Love is great. Marriage is a completely different affair.**<br/>[Comedy, Romance]<br/><br/>Gwyn Marcus has always wanted a marriage like her parents. She has just accepted the proposal of her boyfriend Matt, but she has misgivings about their future together. Her fear of commitment grows as she learns of the various affairs that her family is having. With her sister getting married and her brother already married, her mother is growing concerned about Gwyn's being the last single person in the family. But the more she thinks about marriage, the more she must search for the balance between career, marriage and family.|Sarah Jessica Parker as Gwyn Marcus, Antonio Banderas as Antonio, Mia Farrow as Nina Marcus, Kevin Pollak as Jordan Marcus
|tt0110684|**Nobody's Fool** (1994)|**In a town where nothing ever happens... everything is about to happen to Sully.**<br/>[Drama, Comedy]<br/><br/>Sully is a rascally ne'er-do-well approaching retirement age. While he is pressing a worker's compensation suit for a bad knee, he secretly works for his nemesis, Carl, and flirts with Carl's young wife Toby. Sully's long- forgotten son and family have moved back to town, so Sully faces unfamiliar family responsibilities. Meanwhile, Sully's landlady's banker son plots to push through a new development and evict Sully from his mother's life.|Paul Newman as Sully Sullivan, Bruce Willis as Carl Roebuck, Melanie Griffith as Toby Roebuck, Philip Seymour Hoffman as Officer Raymer, Philip Bosco as Judge Flatt, Alice Drummond as Hattie, Margo Martindale as Birdy
|tt0114151|**Poison Ivy II: Lily** (1996)|**Passion. Seduction. Betrayal. Lily wanted to be just like Ivy.**<br/>[Drama, Romance, Thriller]<br/><br/>A young and naive college art student becomes obsessed with assuming the identity and personality of a departed coed who used to live in her room, and in so doing causes complications that result in two men, a student and her art professor, lusting after her.|Xander Berkeley as Donald Falk
|tt0114069|**Outbreak** (1995)|**This animal carries a deadly virus... and the greatest medical crisis in the world is about to happen.**<br/>[Action, Drama, Science Fiction, Thriller]<br/><br/>A deadly airborne virus finds its way into the USA and starts killing off people at an epidemic rate. Col Sam Daniels' job is to stop the virus spreading from a small town, which must be quarantined, and to prevent an over reaction by the White House.|Dustin Hoffman as Sam Daniels, Rene Russo as Robby Keough, Morgan Freeman as Billy Ford, Kevin Spacey as Casey Schuler, Cuba Gooding Jr. as Salt, Donald Sutherland as Donald McClintock
|tt0114084|**Panther** (1995)|**The People Called Them Heroes. The F.B.I. Called Them Public Enemy Number One.**<br/>[Drama]<br/><br/>Panther is a semi-historic film about the origins of The Black Panther Party for Self-Defense. The movie spans about 3 years (1966-68) of the Black Panther's history in Oakland. Panther also uses historical footage (B/W) to emphasize some points.|Bokeem Woodbine as Tyrone, Angela Bassett as Dr. Betty Shabazz, M. Emmet Walsh as Dorsett
|tt0114268|**Rent-a-Kid** (1995)|[Comedy, Family, TV Movie]<br/><br/>Harry Habert, owner of a rent office has an original idea: To rent babies from a nearby orphanage to the local families. He rents the Ward brothers to his first customers...|Leslie Nielsen as Harry Haber, Christopher Lloyd as Lawrence 'Larry' Kayvey
|tt0114594|**Swimming with Sharks** (1994)|**Life is not a movie**<br/>[Crime, Comedy]<br/><br/>A young, naive Hollywood studio assistant finally turns the tables on his incredibly abusive producer boss.|Kevin Spacey as Buddy Ackerman, Benicio del Toro as Rex
|tt0114788|**The Underneath** (1995)|**For passion, betrayal and murder... there's still no place like home.**<br/>[Thriller, Crime]<br/><br/>Michael Chambers has come home to Austin, Texas. To the mother who's starting a new life and the brother driven by old jealousies. To the places he remembers and the memories he can't forget. And to Rachel, the woman he married and then betrayed with his passion for gambling. Now she's together with Tommy Dundee, a man no one trifles with. He takes care of her in a way Michael never could, but there's a price for his attention and Rachel knows this. And when Michael devises a plan to get Rachel out from under Tommy's control, they become entangled in a web of intrigue, danger and desire from which no one escapes unscathed.|Peter Gallagher as Michael Chambers, Joe Don Baker as Clay Hinkle, Paul Dooley as Ed Dutton, Shelley Duvall as Nurse, Elisabeth Shue as Susan Crenshaw
|tt0112435|**The Baby-Sitters Club** (1995)|**Friends Forever**<br/>[Comedy, Drama, Family]<br/><br/>It's the story about seven very different best friends, and one summer that will bring them together like never before.|Bruce Davison as Watson Brewer, Ellen Burstyn as Emily Haberman, Colleen Camp as Maureen McGill
|tt0104779|**Bitter Moon** (1992)|**A kinky voyage with a full head of steam.**<br/>[Thriller, Drama, Romance]<br/><br/>An embittered husband, paralyzed and in a wheelchair, buttonholes a complete stranger and begins to tell him the story of his marriage. The stranger would like to escape, but cannot. For one thing he grows fascinated by the story. For another he is mesmerized by the man's wife, who has perfected that trick of looking a man boldly in the eye until, by looking away, he concedes sexual supremacy.|Hugh Grant as Nigel, Kristin Scott Thomas as Fiona, Peter Coyote as Oscar
|tt0109348|**Bullets Over Broadway** (1994)|[Action, Comedy, Thriller]<br/><br/>Set in 1920's New York City, this movie tells the story of idealistic young playwright David Shayne. Producer Julian Marx finally finds funding for the project from gangster Nick Valenti. The catch is that Nick's girl friend Olive Neal gets the part of a psychiatrist, and Olive is a bimbo who could never pass for a psychiatrist as well as being a dreadful actress. Agreeing to this first compromise is the first step to Broadway's complete seduction of David, who neglects longtime girl friend Ellen. Meanwhile David puts up with Warner Purcell, the leading man who is a compulsive eater, Helen Sinclair, the grand dame who wants her part jazzed up, and Cheech, Olive's interfering hitman / bodyguard. Eventually, the playwright must decide whether art or life is more important.|John Cusack as David Shayne, Dianne Wiest as Helen Sinclair, Jennifer Tilly as Olive Neal, Chazz Palminteri as Cheech, Mary-Louise Parker as Ellen, Jack Warden as Julian Marx, Rob Reiner as Sheldon Flender, Joe Viterelli as Nick Valenti, Jim Broadbent as Warner Purcell
|tt0109813|**The Flintstones** (1994)|**Yabba-Dabba-Doo!**<br/>[Fantasy, Comedy, Family]<br/><br/>Modern Stone Age family the Flintstones hit the big screen in this live-action version of the classic cartoon. Fred helps Barney adopt a child. Barney sees an opportunity to repay him when Slate Mining tests its employees to find a new executive. But no good deed goes unpunished.|John Goodman as Fred Flintstone, Rick Moranis as Barney Rubble, Rosie O'Donnell as Betty Rubble, Halle Berry as Rosetta Stone
|tt0109831|**Four Weddings and a Funeral** (1994)|**Five good reasons to stay single.**<br/>[Comedy, Drama, Romance]<br/><br/>Four Weddings And A Funeral is a British comedy about a British Man named Charles and an American Woman named Carrie who go through numerous weddings before they determine if they are right for one another.|Hugh Grant as Charles, Andie MacDowell as Carrie, Simon Callow as Gareth, Kristin Scott Thomas as Fiona
|tt0110091|**I Like It Like That** (1994)|**For Lisette, everyday has become independence day.**<br/>[Comedy, Drama, Romance]<br/><br/>Lisette and husband Chino face marital difficulties. She is fed up with the kids, while he has job troubles. His mother Rosaria hates Lisette and the neighborhood tramp has designs on Chino. Things get even worse when Chino goes to jail and Lisette gets a good job uptown. Can this marriage be saved?|
|tt0107472|**The Wonderful, Horrible Life of Leni Riefenstahl** (1993)|[Documentary, Foreign]<br/><br/>This documentary recounts the life and work of one of most famous, and yet reviled, German film directors in history, Leni Riefenstahl. The film recounts the rise of her career from a dancer, to a movie actor to the most important film director in Nazi Germany who directed such famous propaganda films as Triumph of the Will and Olympiad. The film also explores her later activities after Nazi Germany's defeat in 1945 and her disgrace for being so associated with it which includes her amazingly active life over the age of 90.|
|tt0107426|**Little Buddha** (1993)|**A magical journey to a place where the past and present meet**<br/>[Drama, History]<br/><br/>Little Buddha is a movie about the life of Siddhartha starring Keanu Reeves and Bridget Fonda and directed by Bernardo Bertolucci.|Keanu Reeves as Siddharta, Bridget Fonda as Lisa
|tt0110588|**Mrs. Parker and the Vicious Circle** (1994)|**New York in the 1920's. The only place to be was the Algonquin, and the only person to know was Dorothy Parker.**<br/>[Drama]<br/><br/>Dorothy Parker remembers the heyday of the Algonquin Round Table, a circle of friends whose barbed wit, like hers, was fueled by alcohol and flirted with despair.|Jennifer Jason Leigh as Dorothy Parker, Matthew Broderick as Charles MacArthur, Gwyneth Paltrow as Paula Hunt, Peter Gallagher as Alan Campbell, Andrew McCarthy as Eddie Parker, Wallace Shawn as Horatio Byrd, Martha Plimpton as Jane Grant, Lili Taylor as Edna Ferber, James Le Gros as Deems Taylor, Heather Graham as Mary Kennedy Taylor, Stephen Baldwin as Roger Spalding, Matt Malloy as Marc Connelly, Stanley Tucci as Fred Hunter
|tt0111054|**Safe Passage** (1994)|**Life's best journeys take you home**<br/>[Drama]<br/><br/>A mother with seven sons feels like she's losing control of her life and her family. But personal...|Susan Sarandon as Margaret "Mag" Singer
|tt0111257|**Speed** (1994)|**Get ready for rush hour**<br/>[Action, Adventure, Crime]<br/><br/>Los Angeles SWAT cop Jack Traven is up against bomb expert Howard Payne, who's after major ransom money. First it's a rigged elevator in a very tall building. Then it's a rigged bus--if it slows, it will blow, bad enough any day, but a nightmare in LA traffic. And that's still not the end.|Keanu Reeves as Jack Traven, Dennis Hopper as Howard Payne, Sandra Bullock as Annie Porter, Joe Morton as Capt. McMahon, Jeff Daniels as Harold 'Harry' Temple, Beth Grant as Helen, Richard Schiff as Train Driver
|tt0111693|**When a Man Loves a Woman** (1994)|**Through the good times. Through the bad times. When a Man Loves a Woman it's for all times.**<br/>[Drama, Romance]<br/><br/>An airline pilot and his wife are forced to face the consequences of her alcoholism when her addictions threaten her life and their daughter's safety. While the woman enters detox, her husband must face the truth of his enabling behavior.|Andy García as Michael Green, Meg Ryan as Alice Green, Ellen Burstyn as Emily, Philip Seymour Hoffman as Gary
|tt0112443|**Bad Company** (1995)|**Bribery. Blackmail. Murder. Specialities of the house.**<br/>[Action, Crime, Romance, Thriller]<br/><br/>Nelson Crowe is a CIA operative under the thumb of the Company for a disputed delivery of $50,000 in gold. They blackmail him into working for the Grimes Organization, which is set up as a private company for hire, to blackmail prominent individuals. Crowe, working with Margaret Wells (another former Covert Operations operative), blackmails and bribes a State Supreme Court judge, but the deal sours. One of Crowe's co-workers, Tod Stapp, discovers Crowe's current CIA involvement in a plot to overthrow Grimes, and blackmails him to be cut in on the deal. More blackmail occurs as Wells manipulates Crowe to kill Grimes, then the CIA uses that discovery to blackmail Wells into killing Crowe. Who can you trust???|Laurence Fishburne as Nelson Crowe, Ellen Barkin as Margaret Wells, David Ogden Stiers as Judge Justin Beach
|tt0112849|**Desert Winds** (1997)|**It began with a whisper**<br/>[]<br/><br/>Jackie and Eugene are joined by a mystical wind tunnel which enables them to speak across a 500-mile desert. Believed by the Indians to be an omen of good luck, the wind inspires both characters to face their fears and follow their hearts.|Heather Graham as Jackie
|tt0113104|**Frank and Ollie** (1995)|[Documentary]<br/><br/>Before computer graphics, special effects wizardry, and out-of-this world technology, the magic of animation flowed from the pencils of two of the greatest animators The Walt Disney Company ever produced -- Frank Thomas and Ollie Johnston. Frank and Ollie, the talent behind BAMBI, PINOCCHIO, LADY AND THE TRAMP, THE JUNGLE BOOK, and others, set the standard for such modern-day hits as THE LION KING. It was their creative genius that helped make Disney synonymous with brilliant animation, magnificent music, and emotional storytelling. Take a journey with these extraordinary artists as they share secrets, insights, and the inspiration behind some of the greatest animated movies the world has ever known!|
|tt0112889|**Dos Crímenes** (1995)|[]<br/><br/>Greed and playing into the hand of providence provides the focus of this Mexican comedy adapted from a novel by Jorge Ibarguengoitia. Marcos, an architect, has just returned to the home of his wealthy uncle Ramon after squandering his money in Mexico City and subsequently finding himself falsely accused of a crime. Although he is flat-broke, he conceals this from Ramon, telling him that he has returned home to buy a local gold mine. Marcos finds the lies come easily as begins trying to induce his uncle to fund his endeavor. Irascible Ramon, who likes Marcos for his similar love of drinking and smoking is duped, but Ramon's sons are not fooled by Marcos. To them he is a threat, and they fear he will be placed in the will. Soon all of them are trying to out-manipulate each other. Even Ramon, who is not as innocent as he appears is involved in the mayhem.|
|tt0113409|**In the Mouth of Madness** (1994)|**Lived Any Good Books Lately?**<br/>[Horror, Mystery, Thriller]<br/><br/>With the disappearance of hack horror writer Sutter Cane, all Hell is breaking loose...literally! Author Cane, it seems, has a knack for description that really brings his evil creepy-crawlies to life. Insurance investigator John Trent is sent to investigate Cane's mysterious vanishing act and ends up in the sleepy little East Coast town of Hobb's End.|Sam Neill as John Trent, Jürgen Prochnow as Sutter Cane, David Warner as Dr. Wrenn, Charlton Heston as Jackson Harglow, John Glover as Saperstein
|tt0106220|**Addams Family Values** (1993)|**The Family Just Got A Little Stranger**<br/>[Comedy, Family, Fantasy]<br/><br/>Siblings Wednesday and Pugsley Addams will stop at nothing to get rid of Pubert, the new baby boy adored by parents Gomez and Morticia. Things go from bad to worse when the new "black widow" nanny, Debbie Jellinsky, launches her plan to add Fester to her collection of dead husbands.|Anjelica Huston as Morticia Addams, Christopher Lloyd as Uncle Fester Addams, Christina Ricci as Wednesday Addams, Joan Cusack as Debbie Jellinsky, Carol Kane as Granny, Christine Baranski as Becky Martin-Granger, Tony Shalhoub as Jorge, Ian Abercrombie as Driver
|tt0109067|**The Air Up There** (1994)|[Comedy, Family]<br/><br/>Jimmy Dolan is a college basketball coach who wants a big promotion. To get it, he needs to make a dramatic find. He ends up deep in Africa, hoping to recruit Saleh, a huge basketball prodigy Jimmy glimpsed in a home movie. But Saleh is the chief's son and has responsibilities at home, since the tribe's land is threatened by a mining company with its own hotshot basketball team.|Kevin Bacon as Jimmy Dolan
|tt0109198|**Bad Girls** (1994)|**It was a dangerous time to be a woman. And a good time to have friends.**<br/>[Action, Adventure, Comedy, Drama, Western]<br/><br/>Four former harlots try to leave the wild west (Colorado, to be exact) and head north to make a better life for themselves. Unfortunately someone from Cody's past won't let it happen that easily.|Madeleine Stowe as Cody Zamora, Mary Stuart Masterson as Anita Crown, Andie MacDowell as Eileen Spenser, Drew Barrymore as Lily Laronette, James Russo as Kid Jarrett, James Le Gros as William Tucker, Robert Loggia as Frank Jarrett, Dermot Mulroney as Josh McCoy
|tt0109361|**Cabin Boy** (1994)|**He's Setting Sail On The High Seas... Without A Rudder, A Compass, Or A Clue!**<br/>[Comedy]<br/><br/>A foul-mouthed finishing school graduate mistakenly winds up on an ill-fated fishing boat, and faces the wrath of a crew that considers him bad luck.|Brian Doyle-Murray as Skunk, Brion James as Big Teddy, Ann Magnuson as Calli, Mike Starr as Mulligan, Alfred Molina as School Professor
|tt0106505|**Calendar Girl** (1993)|**Three boy's journey to discovery in the Hollywood of the early sixties.**<br/>[Drama, Comedy]<br/><br/>Like many adolescent boys, Roy Darpinian had the hots for movie divas, and one in particular was his wet dream as half of America's in the 1950s: Marilyn Monroe. The difference is, one summer holiday he actually decided to enlist his spineless buddies, Scott Foreman and Ned Bleuer, to actually drive all the way to Hollywood and make as many desperate attempts as it takes to meet her or get arrested trying, and no setback or embarrassment (even publicly bare-ass) can stop or distract him. Against all odds, he finally even got a chance to help her...|Joe Pantoliano as Harvey Darpinian, Stephen Tobolowsky as Antonio
|tt0106519|**Carlito's Way** (1993)|**He's got a good future if he can live past next week.**<br/>[Action, Crime, Drama, Thriller]<br/><br/>A Puerto-Rican ex-con, just released from prison, pledges to stay away from drugs and violence despite the pressure around him and lead on to a better life outside of NYC.|Al Pacino as Carlito Brigante, Sean Penn as David Kleinfeld, John Leguizamo as Benny Blanco, Luis Guzmán as Pachanga, Viggo Mortensen as Lalin, James Rebhorn as Norwalk
|tt0106660|**Dangerous Game** (1993)|**TURBULENCE REDEMPTION HOLLYWOOD**<br/>[Drama]<br/><br/>A New York film director, working on his latest movie in Los Angeles, begins to reflect the actions in his movie and real life, especially when he begins an affair with the lead actress.|Harvey Keitel as Eddie Israel, James Russo as Francis Burns, Victor Argo as Director of Photography
|tt0106673|**Dave** (1993)|**In a country where anybody can become President, anybody just did.**<br/>[Comedy]<br/><br/>Bill Mitchell is the philandering and distant President of the United States. Dave Kovic is a sweet-natured and caring Temp Agency operator, who by a staggering coincidence looks exactly like the President. As such, when Mitchell wants to escape an official luncheon, the Secret Service hires Dave to stand in for him. Unfortunately, Mitchell suffers a severe stroke whilst having sex with one of his aides, and Dave finds himself stuck in the role indefinitely. The corrupt and manipulative Chief of Staff, Bob Alexander, plans to use Dave to elevate himself to the White House - but unfortunately, he doesn't count on Dave enjoying himself in office, using his luck to make the country a better place, and falling in love with the beautiful First Lady...|Kevin Kline as Dave Kovic / Bill Mitchell, Sigourney Weaver as Ellen Mitchell, Kevin Dunn as Alan Reed, Ving Rhames as Duane Stevenson, Ben Kingsley as Vice-President Nance, Bonnie Hunt as White House Tour Guide, Arnold Schwarzenegger as Himself, George Martin as President's Physician
|tt0106677|**Dazed and Confused** (1993)|**See it with a bud.**<br/>[Comedy, Drama]<br/><br/>The adventures of a group of Texas teens on their last day of school in 1976, centering on student Randall Floyd, who moves easily among stoners, jocks and geeks. Floyd is a star athlete, but he also likes smoking weed, which presents a conundrum when his football coach demands he sign a "no drugs" pledge.|Matthew McConaughey as David Wooderson, Ben Affleck as Fred O'Bannion, Joey Lauren Adams as Simone Kerr, Parker Posey as Darla Marks, Nicky Katt as Clint, Renée Zellweger as Girl in blue pickup truck (uncredited)
|tt0106880|**Fear of a Black Hat** (1994)|**The First Drive-By Comedy**<br/>[Comedy]<br/><br/>Chronicling the controversial career of bad boys N.W.H. (Niggaz With Hats), this uproarious 'mockumentary' lampoons all of hardcore rap's hot-button issues. This underground laugh riot recounts the rise, fall and resurrection of a clueless bunch of would-be rappers, Ice Cold, Tone-Def &amp; Tasty Taste performing as N.W.H.|
|tt0109891|**Getting Even with Dad** (1994)|**..He's not getting mad, he's Getting Even With Dad.**<br/>[Family, Comedy, Crime]<br/><br/>Ray, an ex-con and widower, is planning a coin heist with two accomplices to help him to buy his own bakery. However, he doesn't expect his son Timmy, who was living with Ray's sister, to show up at the house right in the middle of planning. Timmy is ignored and Ray and his buddies pull off the heist. Timmy gets his father's attention by stealing the coins and hiding them. To get them back, his father must take him to a number of different places and treat him like he enjoys his presence. They grow fond of each other but Timmy won't stay with his dad unless he gives up the coins.|Glenne Headly as Detective Theresa Walsh
|tt0109913|**Go Fish** (1994)|**The girl is out there**<br/>[Drama, Romance]<br/><br/>Max is a trendy, pretty, young lesbian, who is having trouble finding love. A friend sets her up with Ely, whom Max likes, but Ely is frumpy, homely, and older. Nor do they have much in common. Can Max learn to look past the packaging?|
|tt0109920|**A Good Man in Africa** (1994)|[Comedy, Drama]<br/><br/>Morgan Leafy is a secretary to the British High Commissioner to an Africa nation. Leafy is a man that makes himself useful to his boss, the snobbish Arthur Fanshawe, who has no clue about what's going on around him, but who wants to use his secretary to carry on his dirty work, which involves getting one of the most powerful men in the country to do business with his country.The young secretary has an eye for beautiful women around him, especially Hazel, a native beauty, with whom he is having an affair. Things get complicated because Sam Adekunle, a man running for president of the country, wants a favor from Leafy in return after he has accepted the invitation to visit London. The proposition involves swaying a prominent doctor's opposition to a plan that will make Adenkule filthy rich.|Sean Connery as Dr. Alex Murray, John Lithgow as Arthur Fanshawe, British High Commissioner, Louis Gossett, Jr. as 
|tt0107096|**Heaven & Earth** (1993)|**An amazing true story of survival.**<br/>[History, War, Action, Drama]<br/><br/>Le Ly lives in a small Vietnamese village whose serenity is shattered when war breaks out. Caught between the Viet Cong and the South Vietnamese army, the village is all but destroyed. After being both brutalized and raped, Le Ly resolves to flee. She leaves for the city, surviving desperate situations, but surviving nonetheless. Eventually she meets a U.S. Marine named Steve Butler who treats her kindly and tells her he would like to be married -- maybe to her.|Tommy Lee Jones as Steve Butler, Robert John Burke as Paul, Timothy Carhart as Big Mike
|tt0107286|**Judgment Night** (1993)|**Don't move. Don't whisper. Don't even breathe.**<br/>[Action, Thriller, Crime]<br/><br/>While racing to a boxing match, Frank, Mike, John and Rey get more than they bargained for. A wrong turn lands them directly in the path of Fallon, a vicious, wise-cracking drug lord. After accidentally witnessing Fallon murder a disloyal henchman, the four become his unwilling prey in a savage game of cat &amp; mouse as they are mercilessly stalked through the urban jungle in this taut suspense drama|Emilio Estevez as Frank Wyatt, Cuba Gooding Jr. as Mike Peterson, Denis Leary as Fallon, Jeremy Piven as Ray Cochran
|tt0107362|**Last Action Hero** (1993)|**This isn't the movies anymore.**<br/>[Adventure, Fantasy, Action, Comedy, Family]<br/><br/>Danny is obsessed with a fictional movie character action hero Jack Slater. When a magical ticket transports him into Jack's latest adventure, Danny finds himself in a world where movie magic and reality collide. Now it's up to Danny to save the life of his hero and new friend.|Arnold Schwarzenegger as Jack Slater, F. Murray Abraham as John Practice, Robert Prosky as Nick, Ian McKellen as Death, Joan Plowright as Teacher, Colleen Camp as Ratcliff, Danny DeVito as Whiskers (Voice), Jean-Claude Van Damme as Jean-Claude Van Damme, Bridgette Wilson as Whitney/Meredith, Chevy Chase as Chevy Chase, Jim Belushi as James Belushi
|tt0107413|**Life With Mikey** (1993)|**He's a talent agent. She's a thief. Looks like they've already got something in common.**<br/>[Comedy, Family]<br/><br/>Michael Chapman was once a child TV star. But when he grew up, he couldn't get work. So he and his brother, Ed start their own talent agency that specializes in child acts. They can't seem to find the next big thing and they have to deal with another agency who's not above bribery to get the kids to sign with them. One day Michael meets a girl named Angie and she's a real spitfire. Michael thinks she could be what they are looking for. Problem is that she has a big chip on her shoulder.|Michael J. Fox as Michael Chapman
|tt0110353|**Lightning Jack** (1994)|[Action, Comedy, Western]<br/><br/>Lightning Jack Kane is an Australian outlaw in the wild west. During a bungled bank robbery he picks up mute Ben Doyle as a hostage. The two become good friends, with Jack teaching Ben how to rob banks, while they plan Jack's last heist.|Beverly D'Angelo as Lana Castel, Cuba Gooding Jr. as Ben Doyle, Pat Hingle as U.S. Marshal Dan Kurtz, Richard Riehle as Marcus
|tt0107478|**Made in America** (1993)|**At the sperm-bank she asked for a tall, intelligent, black man. One out of three ain't bad.**<br/>[Comedy]<br/><br/>A young black woman discovers that her father was a sperm donor, and if that weren't bad enough, he's white.|Whoopi Goldberg as Sarah Mathews, Jennifer Tilly as Stacy
|tt0107497|**Malice** (1993)|**Her doctor wasn't playing God. He thought he was God.**<br/>[Drama, Crime]<br/><br/>A tale about a happily married couple who would like to have children. Tracy teaches infants, Andy's a college professor. Things are never the same after she is taken to hospital and operated upon by Jed, a "know all" doctor.|Alec Baldwin as Dr. Jed Hill, Nicole Kidman as Tracy Kennsinger, Bill Pullman as Andy Sarafian, Bebe Neuwirth as Det. Dana Harris, Anne Bancroft as Mrs. Kennsinger, Peter Gallagher as Atty. Dennis Riley, Gwyneth Paltrow as Paula Bell
|tt0107653|**Naked** (1993)|[Comedy, Drama]<br/><br/>Johnny flees Manchester for London, to avoid a beating from the family of a girl he has raped. There he finds an old girlfriend, and spends some time homeless, spending much of his time ranting at strangers, and meeting characters in plights very much like his own.|David Thewlis as Johnny
|tt0110657|**The Next Karate Kid** (1994)|**Who says the good guy has to be a guy?**<br/>[Adventure]<br/><br/>During a commemoration for Japanese soldiers fighting in the US Army during World War II, Mr. Miyagi meets the widow of his commanding officer. He gets to know her granddaughter Julie, an angry teenager who is still feeling the pain of losing both her parents in an accident and is having problems with her grandmother and her fellow pupils. Mr. Miyagi decides to teach her karate to get her through her pain and issues and back on the right path.|Michael Ironside as Colonel Dugan
|tt0107983|**Romeo Is Bleeding** (1993)|**The story of a cop who wanted it bad and got it worse.**<br/>[Action, Crime, Drama, Thriller]<br/><br/>A corrupt cop gets in over his head when he tries to assassinate a beautiful Russian hit-woman.|Gary Oldman as Jack Grimaldi, Annabella Sciorra as Natalie Grimaldi, Juliette Lewis as Sheri, Will Patton as Martie, James Cromwell as Cage, Michael Wincott as Sal, Dennis Farina as Nick Gazzara, Ron Perlman as Jack's Lawyer
|tt0108065|**Searching for Bobby Fischer** (1993)|**Every journey begins with a single move.**<br/>[Drama]<br/><br/>A prepubescent chess prodigy refuses to harden himself in order to become a champion like the famous but unlikable Bobby Fischer.|Joe Mantegna as Fred Waitzkin, Ben Kingsley as Bruce Pandolfini, Laurence Fishburne as Vinnie, David Paymer as Kalev, William H. Macy as Tunafish Father, Dan Hedaya as Tournament Director
|tt0108162|**Sliver** (1993)|**You like to watch... don't you?**<br/>[Drama, Thriller]<br/><br/>A women moves into an apartment in Manhattan and learns that the previous tenant's life ended mysteriously after the tenant fell from the balcony.|Sharon Stone as Carly Norris, Colleen Camp as Judy Marks, Martin Landau as Alex Parsons
|tt0108174|**So I Married an Axe Murderer** (1993)|**The honeymoon was killer.**<br/>[Comedy, Crime, Romance]<br/><br/>Just after a bad breakup, Charlie MacKenzie falls for lovely butcher Harriet Michaels and introduces her to his parents. But, as voracious consumers of sensational tabloids, his parents soon come to suspect that Harriet is actually a notorious serial killer -- "Mrs. X" -- wanted in connection with a string of bizarre honeymoon killings. Thinking his parents foolish, Charlie proposes to Harriet. But while on his honeymoon with her, he begins to fear they were right.|Amanda Plummer as Rose Michaels, Phil Hartman as Ranger John 'Vicky' Johnson - Alcatraz Guide, Debi Mazar as Susan, Tony's Girlfriend, Alan Arkin as Tony's Boss / Precinct Capt. (uncredited)
|tt0110763|**The Pagemaster** (1994)|**All The Adventure Your Imagination Can Hold.**<br/>[Fantasy, Science Fiction, Family, Animation]<br/><br/>Tyler knows a lot about accidents. So much so, he is scared to do anything that might endanger him, like riding his bike, or climbing into his treehouse. While in an old library, he is mystically transported into the unknown world of books, and he has to try and get home again. Along the way he meets some interesting characters, like Fantasy, Adventure, and Horror... Written by Colin Tinto|Christopher Lloyd as Mr. Dewey, Patrick Stewart as Adventure (voice), Whoopi Goldberg as Fantasy (voice), Ed Begley Jr. as Alan Tyler, Frank Welker as Horror (voice), Phil Hartman as Tom Morgan (voice), Jim Cummings as Long John Silver (voice)
|tt0111709|**Wide Eyed and Legless** (1993)|[TV Movie, Drama, Romance, Comedy]<br/><br/>Diana and Deric have an ideal marriage: they thrive in each other's company, they're funny, and they enjoy their two grown children and Deric's dotty mother; the trouble is, Diana can no longer walk and her malady defies medical diagnosis. To care for Diana, Deric is letting his business slide, but at a civic luncheon, he is seated next to Aileen Armitage, a novelist who is blind. They have a nice time, and on the sly, Diana contacts Aileen to made an odd request. Diana's declining health and her resolve bring this triangle of unlikely friends to a surprising place.|Jim Broadbent as Deric Longden
|tt0112572|**The Brady Bunch Movie** (1995)|[Comedy]<br/><br/>The original 70's TV family is now placed in the 1990's, where they're even more square and out of place than ever.|David Graf as Sam Franklin, Michael McKean as Mr. Larry Dittmeyer
|tt0029583|**Snow White and the Seven Dwarfs** (1937)|**The Happiest, Dopiest, Grumpiest, Sneeziest movie of the year.**<br/>[Fantasy, Animation, Family]<br/><br/>A beautiful girl, Snow White, takes refuge in the forest in the house of seven dwarfs to hide from her stepmother, the wicked Queen. The Queen is jealous because she wants to be known as "the fairest in the land," and Snow White's beauty surpasses her own.|
|tt0065214|**The Wild Bunch** (1969)|**If you only want to spend two hours in a movie theatre and go home and forget it, stay away from THE WILD BUNCH.**<br/>[Adventure, Western]<br/><br/>Aging outlaw Pike Bishop (William Holden) prepares to retire after one final robbery. Joined by his gang, which includes Dutch Engstrom (Ernest Borgnine) and brothers Lyle (Warren Oates) and Tector Gorch (Ben Johnson), Bishop discovers the heist is a setup orchestrated in part by his old partner, Deke Thornton (Robert Ryan). As the remaining gang takes refuge in Mexican territory, Thornton trails them, resulting in fierce gunfights with plenty of casualties|Ernest Borgnine as Dutch Engstrom, Dub Taylor as Reverend Wainscoat
|tt0117247|**One Fine Day** (1996)|**She was having a perfectly bad day... Then he came along and spoiled it.**<br/>[Comedy, Romance]<br/><br/>Melanie Parker, an architect and mother of Sammy, and Jack Taylor, a newspaper columnist and father of Maggie, are both divorced. They meet one morning when overwhelmed Jack is left unexpectedly with Maggie and forgets that Melanie was to take her to school. As a result, both children miss their school field trip and are stuck with the parents. The two adults project their negative stereotypes of ex-spouses on each other, but end up needing to rely on each other to watch the children as each must save his job. Humor is added by Sammy's propensity for lodging objects in his nose and Maggie's tendency to wander.|Michelle Pfeiffer as Melanie Parker, George Clooney as Jack Taylor, Charles Durning as Lew, Joe Grifasi as Manny Feldstein, George Martin as Mr. Smith Leland, Holland Taylor as Rita, Michael Badalucco as Lt. Bonomo
|tt0110374|**Loaded** (1996)|[Drama, Mystery, Thriller]<br/><br/>A group of young friends convene in the countryside to shoot a horror movie. But an experiment with LSD sees normal boundaries between them collapsing, and tragedy subsequently striking.|
|tt0114671|**Land and Freedom** (1995)|**The fight against fascism begins with the fight against Stalinism.**<br/>[Drama, History, War, Romance]<br/><br/>The movie narrates the story of David Carr, an unemployed worker and member of the Communist Party of Great Britain. In 1936 he decides to fight for the Republican side in the Spanish Civil War, an anti-fascist coalition of liberals, communists and anarchists. Similar to George Orwell's experiences documented in Homage to Catalonia, he joins the POUM worker militia and witnesses first hand the betrayal of the Spanish revolution by the Stalinists, loyal only to the dictat of Moscow.|Ian Hart as David Carr
|tt0112509|**Billy's Holiday** (1995)|**As of today, he's not the man he used to be!**<br/>[]<br/><br/>This movie is all about a normal guy called Billy Apples. He is a social drinker and sometimes lead singer of a band. While he is doing all of this, he still manages to bring up his teenage daughter. One night while sitting at home Billy finds that he has a talent! He finds that he is able to sing like his idol, 50's singer Billie Holiday. From there he is going places, from huge live performances to even cutting a hit record, but in the end he must make a choice between the life he has, and the life he once knew. Written by Graham Wilson Jr|
|tt0115591|**August** (1996)|**It's really how life can appear with desillusions, good and funny moments, peace, love and hate.**<br/>[Romance, Comedy, Drama]<br/><br/>Set in 1890s North Wales over a long, hot August weekend, the Victorian calm of a household is suddenly upset with the arrival of a London couple who impose their city ways and thoughts on the more rurally based family. An adaptation of Anton Chekhov's play, "Uncle Vanya."|Anthony Hopkins as Ieuan Davies
|tt0114787|**Underground** (1995)|[War, Drama, Comedy]<br/><br/>Black marketeers Marko (Miki Manojlovic) and Blacky (Lazar Ristovski) manufacture and sell weapons to the Communist resistance in WWII Belgrade, living the good life along the way. Marko's surreal duplicity propels him up the ranks of the Communist Party, and he eventually abandons Blacky and steals his girlfriend. After a lengthy stay in a below-ground shelter, the couple reemerges during the Yugoslavian Civil War of the 1990s as Marko realizes that the situation is ripe for exploitation.|
|tt0113720|**All Things Fair** (1995)|**He was a student. She was his teacher. Their love was forbidden.**<br/>[War, Drama, Romance]<br/><br/>This film is set in 1943 when the whole of Europe was embroiled in WWII. It deals with attraction of a 15 year old boy Stig to his teacher Viola. The whole movie revolves around the sexual encounters between Stig and Viola and how he eventually grows out of it.|
|tt0048473|**Pather Panchali** (1955)|[Drama]<br/><br/>The film is a coming-of-age story of a young boy named Apu, and life in his small Indian village in the early years of the 20th century. Apu is born to a poor Brahmin family in Bengal. His father Harihar is a priest who dreams of becoming a successful poet and a playwright; he does not earn enough, but the mother Sarbajaya keeps the family going. Because of their limited resources, Sarbajaya resents having to share her home with Harihar's elderly cousin: the old and helpless cripple Indir. Apu's sister Durga is always getting into trouble for stealing guavas from the neighbour's orchards for Indir. She cares for Apu like a good older sister but loves to affectionately tease him. Together, they make do with what they have and enjoy the simple joys of life.|
|tt0117128|**Mystery Science Theater 3000: The Movie** (1996)|**Every year Hollywood makes hundreds of movies. This is one of them.**<br/>[Comedy, Science Fiction]<br/><br/>The mad and evil scientist, Dr. Clayton Forrester, has created an evil little scheme that is bound to give him world global domination but first things first. He plans to torment Mike Nelson and the robots by sending them a real stinker of a film to watch called, "This Island Earth." He is convinced that this movie will drive them insane. Will this be the ultimate cheese that breaks the boys' spirits?|
|tt0082206|**Coup de Torchon** (1981)|[Drama]<br/><br/>In 1938 in the french West Africa, Lucien Cordier is a weak and corrupted policeman despised by the local bad boys. His wife is openly cheating on him and he hasn't got any self-respect anymore. But when comes the occasion, his revenge will be terrible.|
|tt0111430|**Tigrero: A Film That Was Never Made** (1994)|[Documentary, Foreign]<br/><br/>In 1993, Sam Fuller takes Jim Jarmusch on a trip into Brazil's Mato Grosso, up the River Araguaia to the village of Santa Isabel Do Morro, where 40 years before, Zanuck had sent Fuller to scout a location and write a script for a movie based on a tigrero, a jaguar hunter. Sam hopes to find people who remember him, and he takes film he shot in 1954. He's Rip Van Winkle, and, indeed, a great deal changed in the village. There are televisions, watches, and brick houses. But, the same Karajá culture awaits as well. He gathers the villagers to show his old film footage, and people recognize friends and relatives, thanking Fuller for momentarily bringing them back to life.|
|tt0107727|**The Eye of Vichy** (1993)|[]<br/><br/>Directed by French filmmaker Claude Chabrol, this documentary examines Nazi and Vichy newsreels and propaganda films from World War II meant to turn the French against the Jews and the Allied Forces and into Nazi sympathizers.  This movie is only made of archive pictures: the official newsreels that were broadcasted on French movie screens during 1940 and 1944 (the Occupation).|
|tt0114736|**True Crime** (1996)|**Trust no one.**<br/>[Crime, Mystery, Thriller]<br/><br/>Mary Giordano is a bright, intelligent student who goes to a catholic school. She also has a addiction to mystery novels and detective magazines (hence the title of the movie), which inspire her to do her own detective work. When she starts snooping around on the case of a murderer of teenage girls, it gets her in hot water with her mentor Detective Jerry Gunn. But it also starts a team up with police cadet Tony Campbell. The two work together to find the murderer. But the closer Mary gets to solving the murder, the more danger she puts herself in of being the next victim.|Bill Nunn as Detective Jerry Guinn
|tt0112604|**Butterfly Kiss** (1995)|[Crime, Drama, Romance, Thriller]<br/><br/>Eunice is walking along the highways of northern England from one filling station to another. She is searching for Judith, the woman, she says to be in love with. It's bad luck for the women at the cash desk not to be Judith, because Eunice is eccentric, angry and extreme dangerous. One day she meets Miriam, hard of hearing and a little ingenuous, who feels sympathy for Eunice and takes her home. Miriam is very impressed by Eunice's fierceness and willfulness and follows her on the search for Judith. Shocked by Eunice's cruelty she tries to make her a better person, but she looses ground herself.|Amanda Plummer as Eunice
|tt0095776|**Oliver & Company** (1988)|**The first Disney movie with attitude.**<br/>[Animation, Comedy, Family]<br/><br/>This animated take on Oliver Twist re-imagines Oliver as an adorable orphaned kitten who struggles to survive in New York City and falls in with a band of canine criminals led by an evil human. First, Oliver meets Dodger, a carefree mutt with street savoir faire. But when Oliver meets wealthy Jenny on one of the gang's thieving missions, his life changes forever.|Cheech Marin as Tito (voice), Roscoe Lee Browne as Francis (voice), Dom DeLuise as Fagin (voice), Robert Loggia as Sykes (voice)
|tt0073778|**Switchblade Sisters** (1975)|**So Easy to Kill, So Hard to Love**<br/>[Action, Crime, Drama]<br/><br/>A tough gang of teenage girls are looking for love and fighting for turf on the mean streets of the city! Bad girls to the core, these impossibly outrageous high school hoodlums go where they want ... and create mayhem wherever they go!|
|tt0108500|**The Visitors** (1993)|**They Weren't Born Yesterday!**<br/>[Fantasy, Comedy, Science Fiction]<br/><br/>This outrageous time-travel comedy follows the misadventures of a wacky medieval knight (Jean Reno) and his faithful servant when they are accidentally transported to contemporary times by a senile sorcererMayhem rules as these 12th-century visitors try adapting to the wildly confusing modern world. To avoid being stuck here for good, however, they soon begin an all-out cosmic assault on their former castle -- now a luxury hotel -- in their quest to return to the past|Jean Reno as Godefroy de Papincourt
|tt0113270|**The Haunted World of Edward D. Wood, Jr.** (1996)|**Can your MIND stand the SHOCKING TRUTH?**<br/>[Documentary]<br/><br/>The strange life and the wonderfully awful films of 1950's Hollywood Z movie director Ed Wood are profiled in this documentary that was conceived of and researched several years before commercial-filmmaker Tim Burton made his feature film tribute. Actually, Wood does not appear much in this film. Rather, it centers on the lives and thoughts of his entourage and those who knew him. Among those interviewed are Wood's former lover and star of his earliest films, Dolores Fuller, whom he abruptly replaced in the middle of Bride of the Monster with actress Loretta King who is also interviewed.|Bela Lugosi as Himself
|tt0333373|**Two Friends** (2002)|[Drama, Foreign]<br/><br/>Two Sicilian friends, Nunzio and Pino, share the same apartment in Turin. Nunzio works in a factory but is laid off because of his illness. Pino, on the other hand, is a mysterious man and he is always traveling because of his work. Nunzio would very much like to know what his friend is doing for a living but Pino will not tell him. Nunzio spends his free time the best he can in his friend's absences. He ends up falling in love with Maria, a commercial employee, whereas his health condition deteriorates...|
|tt0117260|**Original Gangstas** (1996)|**The original bad boys are stepping up in '96!**<br/>[Action, Drama, Crime, Thriller]<br/><br/>A violent street gang, the Rebels, rule the streets of Gary, Indiana. The Rebels shoot Marvin Bookman, a store-keeper, for giving the police information about a drive-by shooting they committed. Marvin's son, former NFL star John who created the Rebels, returns to Gary to be with his father and, with a little help from his friends, to destroy the Rebels his way.|Charles Napier as Mayor
|tt0116405|**Getting Away with Murder** (1996)|**One bad apple can spoil the whole line-up.**<br/>[Comedy]<br/><br/>When the very moralistic college ethics instructor Jack Lambert finds himself living next door to an accused German death camp commander, he takes it upon himself to rid the world of this man.|Jack Lemmon as , Dan Aykroyd as , Bonnie Hunt as Gail Holland, Lily Tomlin as Inga Mueller
|tt0112586|**Brothers in Trouble** (1995)|[Comedy, Drama]<br/><br/>Amir is an illegal Pakistani immigrant smuggled into England in the 1960's to work, to send money to his family and perhaps even bring them over with him. A skilled laborer, he is forced to do unskilled work like shoveling sheep dung and processing wool. He lives in a boarding house with nearly a dozen other men, under the supervision of Hussein Shah. He befriends a young student, Sakib, who dreams of being a writer. Their existence is punctuated by secret movies, a visiting prostitute, fear of detection and deportation, and the gangster-like smuggler who comes by for his take every week. The household is shaken up by the arrival of a white girl, Shah's girlfriend, and the sense of femininity and family she brings.|
|tt0057012|**Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb** (1964)|**The hot-line suspense comedy**<br/>[Drama, Comedy, War]<br/><br/>Insane General Jack D. Ripper initiates a nuclear strike on the Soviet Union. As soon as the actions of General "Buck" Turgidson are discovered, a war room full of politicians, generals and a Russian diplomat all frantically try to stop the nuclear strike. Near the end is a scene that is probably the most uniquely unforgettable performance of Slim Pickens in his movie career. Peter Sellers plays multiple roles in this film.|Keenan Wynn as Colonel Bat Guano, James Earl Jones as Lt. Lothar Zogg
|tt0116669|**Jack** (1996)|**Jack Powell is about to tackle his biggest adventure ever... the fifth grade!**<br/>[Comedy, Drama, Science Fiction]<br/><br/>Jack Powell suffers from an affliction that makes him grow four times faster than normal, so the 10-year-old looks like a 40-year-old man. After years of being tutored at home, Jack convinces his overprotective parents to send him to public school. The children don't know what to make of Jack, but with the help of his fifth-grade teacher, he makes an effort to win them over.|Robin Williams as Jack Charles Powell, Jennifer Lopez as Miss Marquez
|tt0063715|**Spirits of the Dead** (1968)|**The ultimate orgy of evil**<br/>[Drama, Mystery, Horror]<br/><br/>Three directors each adapt a Poe short story to the screen: "Toby Dammit" features a disheveled drugged and drunk English movie star who nods acceptance in the Italian press and his producers fawn over him. "Metzengerstein" features a Mediveal countess who has a love-hate relationship with a black stallion - who it turns out is really her dead lover. "William Wilson" tells the story of a sadistic Austrian student with an exact double whom he later kills.|Terence Stamp as Toby Dammit (segment "Toby Dammit"), Vincent Price as Narrator (English version) (uncredited)
|tt0116629|**Independence Day** (1996)|**Earth. Take a good look. It might be your last.**<br/>[Action, Adventure, Science Fiction]<br/><br/>On July 2, a giant alien mothership enters orbit around Earth and deploys several dozen saucer-shaped 'destroyer' spacecraft that quickly lay waste to major cities around the planet. On July 3, the United States conducts a coordinated counterattack that fails. On July 4, a plan is devised to gain access to the interior of the alien mothership in space, in order to plant a nuclear missile.|Bill Pullman as President Thomas J. Whitmore, Jeff Goldblum as David Levinson, Mary McDonnell as First Lady Marilyn Whitmore, Robert Loggia as General William Grey, Randy Quaid as Russell Casse, Vivica A. Fox as Jasmine Dubrow, James Rebhorn as Albert Nimzicki, Brent Spiner as Dr. Brackish Okun
|tt0118113|**Walking and Talking** (1996)|**A movie for everyone who wants to get married and stay single at the same time.**<br/>[Comedy, Drama, Romance]<br/><br/>Things have been tough lately for Amelia. Her best friend moved out of the apartment, her cat got cancer, and now her best friend, Laura, is getting married. She copes with things, from the help of Andrew, Frank, Laura, and a brief romance with Bill "The Ugly Guy".|Anne Heche as Laura, Liev Schreiber as Andrew, Alice Drummond as Betsy, Kevin Corrigan as Bill
|tt0116531|**High School High** (1996)|**A Dangerous Mind is a terrible thing to waste**<br/>[Comedy]<br/><br/>Richard Clark has just left the well-known Wellington Academy to teach at Marion Barry High School. Now, he will try to inspire the D-average students into making good grades and try to woo a fellow teacher.|Jon Lovitz as Richard Clark, Louise Fletcher as Schuldirektorin Evelyn Doyle
|tt0117008|**Matilda** (1996)|**A little magic goes a long way.**<br/>[Comedy, Family, Fantasy]<br/><br/>A grouchy couple are parents to a very sweet girl, Matilda. Unlike her bratty brother and mean parents, Matilda becomes a very sweet and extremely intelligent 6 year old girl, who is very keen to go to school and read books. After a while, her parents send her to school with the worst principal in the world, a very sweet teacher, and good friends. While trying to put up with her parents' and principal's cruelty, she starts to unwittingly unleash telekinetic powers, destroying a television and making a newt fly onto the principal. With enough practice, Matilda starts to learn to control her telekinetic powers and soon using them on her principal so she can drive her away from the school.|Danny DeVito as Harry Wormwood, Paul Reubens as FBI Agent Bob, Tracey Walter as FBI Agent Bill
|tt0115836|**Carpool** (1996)|**A dad in a hurry. A man on the run. And five kids with an attitude.**<br/>[Comedy, Family]<br/><br/>A man with an important business meeting finds himself having to take care of the carpool for the neighborhood school children when his wife gets sick. Stopping to get donuts for the kids, things go even more awry when he finds himself a victim of a robbery. However, the situation only gets worse as a desperate man who had been contemplating a bank robbery robs the robbers and takes the man and the kids hostage in their van as his truck is blocked by an armored car. The thing then proceeds into a comedic chase movie. The father finds his kids don't really respect him and they react better to the robber. The end result is everyone gets a lifestyle change, including the original store owner.|Tom Arnold as Franklin Laszlo, David Paymer as Daniel Miller, Rod Steiger as Mr. Hammerman
|tt0075169|**Shadow of Angels** (1976)|[Drama]<br/><br/>Beautiful, detached, laconic, consumptive Lily Brest is a streetwalker with few clients. She loves her idle boyfriend Raoul who gambles away what little she earns. The town's power broker, called the rich Jew, discovers she is a good listener, so she's soon busy. Raoul imagines grotesque sex scenes between Lily and the Jew; he leaves her for a man. Her parents, a bitter Fascist who is a cabaret singer in drag and her wheelchair-bound mother, offer no refuge. Even though all have a philosophical bent, the other whores reject Lily because she tolerates everyone, including men. She tires of her lonely life and looks for a way out. Even that act serves the local corrupt powers.|
|tt0112607|**Bye-Bye** (1995)|[Drama]<br/><br/>In Paris, Ismaél, a young Tunisian, cares for two brothers, Nouredine, a cripple, and streetwise Mouloud, 14. In haste, Ismaél and Mouloud go to Marseilles where an uncle lives. Nouredine has died in a fire, and Ismaél feels guilt on top of grief. Ismaél becomes friends with Jacky, a white man whose father and brother hate immigrants. Mouloud hangs out with cousin Rhida who breaks Islamic rules and deals hash. Ismaél decides Mouloud must return to Tunisia, but the boy runs off, becoming an acolyte to Rhida's supplier. Ismaél and Jacky's Arab girlfriend start an affair, friends betray friends, and the racism gets ugly. Can Ismaél rescue himself and Mouloud or will life in France crush them?|
|tt0109001|**1-900** (1994)|[]<br/><br/>Sarah, 30, single, well educated, likes art, places a voice ad for phone sex, inviting replies from men under 35, sturdy and sensual. Wilbert, a chubby middle aged architect, leaves a message that he is Thomas, 32 and well formed. A week later, she phones. She likes his voice, he likes her laugh, so once a week she phones him (he's not to know her last name, where she lives, or her number). Portraits emerge of humor and sadness. He presses for details of her life and tells her about his work; she wants these Thursday calls to be fantasy and release, separate from the rest of who she is. The film dramatizes these tensions of lies and truth, connection and distance. Written by|
|tt0035896|**For Whom the Bell Tolls** (1943)|**Thunderous! Tender! Touching!**<br/>[Drama, History, Romance, Thriller]<br/><br/>Spain in the 1930s is the place to be for a man of action like Robert Jordan. There is a civil war going on and Jordan who has joined up on the side that appeals most to idealists of that era -- like Ernest Hemingway and his friends -- has been given a high-risk assignment up in the mountains. He awaits the right time to blow up a bridge in a cave.|
|tt0045152|**Singin' in the Rain** (1952)|**What a Glorious Feeling!**<br/>[Comedy, Music, Romance]<br/><br/>In 1927 Hollywood, Don Lockwood and Lina Lamont are a famous on-screen romantic pair in silent movies, but Lina mistakes the on-screen romance for real love. When their latest film is transformed into a musical, Don has the perfect voice for the songs, but strident voice faces the studio to dub her voice. Aspiring actress, Kathy Selden is brought in and, while she is working on the movie, Don falls in love with her.|Bess Flowers as Audience Member (uncredited), Kathleen Freeman as Phoebe Dinsmore (uncredited)
|tt0053604|**The Apartment** (1960)|**Movie-wise, there has never been anything like it - laugh-wise, love-wise, or otherwise-wise!**<br/>[Comedy, Drama, Romance]<br/><br/>Bud Baxter is a minor clerk in a huge New York insurance company, until he discovers a quick way to climb the corporate ladder. He lends out his apartment to the executives as a place to take their mistresses. Although he often has to deal with the aftermath of their visits, one night he's left with a major problem to solve.|Jack Lemmon as C.C. Baxter, Shirley MacLaine as Fran Kubelik
|tt0053291|**Some Like It Hot** (1959)|**The movie too HOT for words!**<br/>[Comedy, Romance]<br/><br/>Two musicians witness a mob hit and struggle to find a way out of the city before they are found by the gangsters. Their only opportunity is to join an all-girl band as they leave on a tour. To make their getaway they must first disguise themselves as women, then keep their identities secret and deal with the problems this brings - such as an attractive bandmate and a very determined suitor.|Jack Lemmon as Jerry (Daphne), Arthur Tovey as Speakeasy Patron (uncredited)
|tt0032484|**Foreign Correspondent** (1940)|**The thrill spectacle of the year!**<br/>[Mystery, Thriller, War]<br/><br/>The European war was only beginning to erupt across national borders. Its titular hero, Johnny Jones, is an American crime reporter dispatched by his New York publisher to put a fresh spin on the drowsy dispatches emanating from overseas, his nose for a good story promptly leading him to the crime of fascism and Nazi Germany's designs on European conquest In attempting to learn more about a seemingly noble peace effort, Jones who walks into the middle of an assassination, uncovers a spy ring, and, not entirely coincidentally, falls in love.|Gino Corrado as Luncheon Waiter, Sam Harris as Luncheon Guest, Alfred Hitchcock as Man with Newspaper on Street
|tt0048960|**Around the World in Eighty Days** (1956)|**It's a wonderful world, if you'll only take the time to go around it!**<br/>[Adventure, Comedy, Family, Fantasy, Romance]<br/><br/>Based on the famous book by Jules Verne the movie follows Phileas Fogg on his journey around the world. Which has to be completed within 80 days, a very short period for those days.|Peter Lorre as Steward, Shirley MacLaine as Princess Aouda, John Gielgud as Foster, Bess Flowers as Extra (uncredited), Sam Harris as Extra (uncredited), Arthur Tovey as Extra (uncredited)
|tt0038300|**Angel on My Shoulder** (1946)|**With an ANGEL on his shoulder... And the DEVIL in his heart!**<br/>[Comedy, Fantasy, Romance]<br/><br/>Gangster Eddie Kagel is killed by a trusted lieutenant and finds himself in Harry Redmond Jr's special effects Hell, where Nick/The Devil sees that he is an-exact double for a judge of whom Nick doesn't approve. Eddie is agreeable to having his soul transferred to the judge's body, as it will give him a chance to avenge himself on his killer. But every action taken by Eddie (as the judge) results in good rather than evil and, to Nick's dismay, the reputation and influence of the judge is enhanced, rather than impaired by Eddie. And Eddie also falls in love with the judge's fiancée, Barbara. Even Eddie's planned revenge fails and Nick is forced to concede defeat. He returns to Hell, taking Eddie with him, after Eddie has extracted his promise that Nick will not molest the judge or Barbara in the future.|
|tt0116442|**Grace of My Heart** (1996)|**For years her songs brought fame to other people. Then she found her own voice.**<br/>[Drama, Comedy, Music]<br/><br/>An aspiring singer, Denise Waverly/Edna Buxton, sacrifices her own singing career to write hit songs that launch the careers of other singers. The film follows her life from her first break, through the pain of rejection from the recording industry and a bad marriage, to her final triumph in realizing her dream to record her own hit album.|Illeana Douglas as Denise Waverly / Edna Buxton, John Turturro as Joel Millner, Eric Stoltz as Howard Cazsatt, Matt Dillon as Jay Phillips, Bridget Fonda as Kelly Porter, Richard Schiff as Audition Record Producer, Bruce Davison as John Murray, Lucinda Jenney as Marion
|tt0117473|**The Rich Man's Wife** (1996)|**Someone Is Playing A Very Deadly Game...**<br/>[Drama, Thriller]<br/><br/>A rich man's wife finds she has a bad prenuptial agreement with an even worse husband. Over drinks with a stranger, she fantasizes about doing her husband in to void the prenupt. The stranger decides to turn her imagination into reality much to the wife's surprise.|Halle Berry as Josie Potenza, Frankie Faison as Detective Ron Lewis, Christopher McDonald as Tony Potenza
|tt0116167|**Ed's Next Move** (1996)|**It's the hardest move a guy has to make.**<br/>[]<br/><br/>Ed lives in Wisconsin, but he's just been dumped by his girlfriend. So he moves to New York City for a big change...but will he ever find love again? He loves his new job studying the genetics of different strains of rice. He has a good partnership with his promiscuous roommate Ray. And he's made some great friends, characters at the local diner. But then there's Natalie (Lee for short). She's just what Ed thinks he's looking for, and she's available...or is she?|
|tt0116421|**The Glimmer Man** (1996)|**Two good cops. One bad situation.**<br/>[Action, Adventure, Crime, Drama, Thriller]<br/><br/>A former government operative renowned for his stealth, Jack Cole is now a Los Angeles police detective. When a series of horrible murders occurs in the metro area, Cole is assigned to the case, along with tough-talking fellow cop Jim Campbell. Although the two men clash, they gradually become effective partners as they uncover a conspiracy linked to the killings, which also involves terrorism and organized crime.|Bob Gunton as Frank Deverell, Brian Cox as Mr. Smith, Stephen Tobolowsky as Christopher Maynard
|tt0064603|**The Love Bug** (1968)|**Herbie will honk his way into your heart.**<br/>[Comedy, Family, Fantasy]<br/><br/>Herbie is a car - but no ordinary car. The story follows the Volkswagen Beetle with a mind of its own from the showroom to the race track, with various close escapes in between. Three further Herbie movies were to follow.|
|tt0059793|**That Darn Cat!** (1965)|**It takes a Siamese secret agent to unravel the PURR-fect crime!**<br/>[Drama, Family, Comedy]<br/><br/>A young woman suspects foul play when her cat comes home wearing a wrist watch. Convincing the FBI, though, and catching the bad guys is tougher than she imagined.|Roddy McDowall as Gregory Benson
|tt0038166|**The Three Caballeros** (1944)|[Animation, Family, Music]<br/><br/>For Donald's birthday he receives a box with three gifts inside. The gifts, a movie projector, a pop-up book, and a pinata, each take Donald on wild adventures through Mexico and South America.|
|tt0102798|**Robin Hood: Prince of Thieves** (1991)|**For the good of all men, and the love of one woman, he fought to uphold justice by breaking the law.**<br/>[Adventure]<br/><br/>When the dastardly Sheriff of Nottingham murders Robin's father, the legendary archer vows vengeance. To accomplish his mission, Robin joins forces with a band of exiled villagers (and comely Maid Marian), and together they battle to end the evil sheriff's reign of terror.|Kevin Costner as Robin Hood, Morgan Freeman as Azeem, Christian Slater as Will Scarlett, Michael Wincott as Guy of Gisborne, Sean Connery as King Richard (uncredited)
|tt0058331|**Mary Poppins** (1964)|**It's supercalifragilisticexpialidocious!**<br/>[Comedy, Family, Fantasy]<br/><br/>The movie combines a diverting story, songs, color and sequences of live action blended with the movements of animated figures. Mary Poppins is a kind of Super-nanny who flies in with her umbrella in response to the request of the Banks children and proceeds to put things right with the aid of her rather extraordinary magical powers before flying off again.|
|tt0095016|**Die Hard** (1988)|**40 Stories. Twelve Terrorists. One Cop.**<br/>[Action, Thriller]<br/><br/>NYPD cop, John McClane's plan to reconcile with his estranged wife is thrown for a serious loop when minutes after he arrives at her office, the entire building is overtaken by a group of terrorists. With little help from the LAPD, wisecracking McClane sets out to single-handedly rescue the hostages and bring the bad guys down.|Bruce Willis as John McClane, Bonnie Bedelia as Holly Gennero McClane, William Atherton as Richard Thornburg, Grand L. Bush as FBI Agent Little Johnson, Mary Ellen Trainor as Gail Wallens
|tt0114597|**Synthetic Pleasures** (1995)|**technology control passion**<br/>[Documentary]<br/><br/>Conceived as an electronic road movie, this documentary investigates cutting edge technologies and their influence on our culture as we approach the 21st century. It takes off from the idea that mankind's effort to tap the power of Nature has been so successful that a new world is suddenly emerging,an artificial reality. Virtual Reality, digital and biotechnology, plastic surgery and mood-altering drugs promise seemingly unlimited powers to our bodies, and our selves. This film presents the implications of having access to such power as we all scramble to inhabit our latest science fictions.|
|tt0120107|**Shadow Conspiracy** (1997)|[Action, Thriller]<br/><br/>Bobby Bishop (Sheen) is a special assistant to the President of the United States. Accidentally, he meets his friend professor Pochenko on the street. Pochenko has time to tell Bishop about some conspiracy in the White House but then immediately gets killed by an assassin. Now bad guys are after Bobby as the only man who knows about a plot. Bishop must now not only survive, but to stop the conspirators from achieving their goal. And he doesn't know whom to trust.|Charlie Sheen as Bobby Bishop, Donald Sutherland as Jacob Conrad
|tt0117802|**Swingers** (1996)|**Cocktails first. Questions later.**<br/>[Comedy, Romance]<br/><br/>This is a story about Mike, a guy who left his girl in New York when he came to LA to be a star. It's been six months since his girlfriend left him and he's not doing so good. So, his pal and some other friends try and get him back in the social scene and forget about his 6 year relationship.|Heather Graham as Lorraine
|tt0039286|**Crossfire** (1947)|**Hate is like a loaded gun!**<br/>[Crime, Drama, Mystery, Thriller]<br/><br/>A man is murdered, apparently by one of a group of soldiers just out of the army. But which one? And why?|
|tt0059243|**The Great Race** (1965)|**The movie with 20,000-mile or one-million-laughs guarantee!**<br/>[Adventure, Comedy]<br/><br/>Professional daredevil and white-suited hero, The Great Leslie, convinces turn-of-the-century auto makers that a race from New York to Paris (westward across America, the Bering Straight and Russia) will help to promote automobile sales. Leslie's arch-rival, the mustached and black-attired Professor Fate vows to beat Leslie to the finish line in a car of Fate's own invention.|Jack Lemmon as Prof. Fate, Keenan Wynn as Hezekiah Sturdy
|tt0048545|**Rebel Without a Cause** (1955)|**The bad boy from a good family.**<br/>[Drama]<br/><br/>After moving to a new town, troublemaking teen Jim Stark is supposed to have a clean slate, although being the new kid in town brings its own problems. While searching for some stability, Stark forms a bond with a disturbed classmate, Plato, and falls for local girl Judy. However, Judy is the girlfriend of neighborhood tough, Buzz. When Buzz violently confronts Jim and challenges him to a drag race, the new kid's real troubles begin.|Dennis Hopper as Goon
|tt0115580|**The Associate** (1996)|**Behind Every Great Man Is A Woman... Wishing He'd Get The Hell Out Of Her Way.**<br/>[Comedy]<br/><br/>Laurel Ayres is a businesswoman trying to make it but unfortunately she works at a investment firm where she does all the work but all the senior investors like Frank Peterson grab all the credit. She then leaves and starts her own firm. While trying to find clients Laurel pretends that she has a male partner named Robert Cutty. And when she starts to do well all of her clients wants to meet Cutty which is difficult since he doesn't exist.|Whoopi Goldberg as Laurel Ayres, Dianne Wiest as Sally Dugan, Bebe Neuwirth as Camille Scott, George Martin as Walter Manchester, Colleen Camp as Detective Jones, Allison Janney as Sandy
|tt0112907|**Drunks** (1997)|**All they want is another shot...**<br/>[Drama]<br/><br/>At the beginning of a nightly Alcoholics Anonymous meeting, Jim seems particularly troubled. His sponsor encourages him to talk that night, the first time in seven months, so he does - and leaves the meeting right after. As Jim wanders the night, searching for some solace in his old stomping grounds, bars and parks where he bought drugs, the meeting goes on, and we hear the stories of survivors and addicts - some, like Louis, who claim to have wandered in looking for choir practice, who don't call themselves alcoholic, and others, like Joseph, whose drinking almost caused the death of his child - as they talk about their lives at the meeting|Amanda Plummer as Shelley, Faye Dunaway as Becky, Parker Posey as Debbie, Dianne Wiest as Rachel, Kevin Corrigan as Cam, George Martin as Marty, Zach Grenier as Al, Anna Levine as Tanya (as Anna Thomson)
|tt0117318|**The People vs. Larry Flynt** (1996)|**You may not like what he does, but are you prepared to give up his right to do it?**<br/>[Drama]<br/><br/>Larry Flynt is the hedonistically obnoxious, but indomitable, publisher of Hustler magazine. The film recounts his struggle to make an honest living publishing his girlie magazine and how it changes into a battle to protect the freedom of speech for all people.|Woody Harrelson as Larry Flynt, James Cromwell as Charles Keating, Crispin Glover as Arlo, Vincent Schiavelli as Chester
|tt0116587|**Hustler White** (1996)|**See it with someone you've paid for.**<br/>[Romance, Drama]<br/><br/>Reminiscent of Sunset Boulevard, Hustler White transposes the action from the silver screen's old movie backlots to contemporary male prostitution and the porn industry. Said to be an homage to classic Hollywood cinema.|
|tt0116212|**Entertaining Angels - The Dorothy Day Story** (1996)|[Drama]<br/><br/>"Entertaining Angels" is an interesting title for this movie about the 1920's and 30's social activist, Dorothy Day, for it can be regarded that what this woman did, sometimes single-handedly, always controversially, in her fight against social injustice would, indeed, be wonderful entertainment for angels... or it could mean that her work was for the benefit of the 'angels' at the bottom of the social ladder for whom she fought daily against those who would hold them down... this included her work as a suffregette. However it is meant, this film captures much of the real-life drama that took place on big city streets, and of the very personal trials which eventually led her to convert to roman Catholicism, and a dedication to helping the poor.|Martin Sheen as Peter Maurin, Heather Graham as Maggie Bowen
|tt0117677|**Snowriders** (1996)|**Man Wasn't Meant To Fly... Until Now!**<br/>[Documentary]<br/><br/>Spectacular shots of snowcapped peaks and extreme skiers gracefully defying gravity, not to mention common sense, are the highlights of this entertaining video from the venerable guru of skiing movies, Warren Miller. Sparing no expense, Miller's film crews span the globe, shooting snowboarders careening down absurdly steep Alaskan peaks, ski guides helicoptering to remote slopes in British Columbia, and hardy (and uninhibited) Scotsmen who boldly catch some "big air" in kilts. This video is fraught with offbeat humor, including shots of a snowshoe race that could have been filmed by Mack Sennett, and footage of "face jumping," the oddball diversion of hurling oneself off Alpine glaciers and parachuting to earth. Miller's gentle narration, replete with trademark corny remarks reminiscent of old movie newsreels, mixes with a musical soundtrack of contemporary rock that is often uncannily synched with the astounding footage of extreme skiing.|
|tt0118147|**When We Were Kings** (1996)|**The untold story of the Rumble in the Jungle.**<br/>[Documentary]<br/><br/>It's 1974. Muhammad Ali is 32 and thought by many to be past his prime. George Foreman is ten years younger and the heavyweight champion of the world. Promoter Don King wants to make a name for himself and offers both fighters five million dollars apiece to fight one another, and when they accept, King has only to come up with the money. He finds a willing backer in Mobutu Sese Suko, the dictator of Zaire, and the "Rumble in the Jungle" is set, including a musical festival featuring some of America's top black performers, like James Brown and B.B. King.|
|tt0110173|**JLG/JLG: Self-Portrait in December** (1995)|[Documentary]<br/><br/>Director Jean-Luc Godard reflects in this movie about his place in film history, the interaction of film industry and film as art, as well as the act of creating art.|
|tt0040723|**Raw Deal** (1948)|**Bullets!  Women! -- Can't Hold a Man Like That!**<br/>[Crime]<br/><br/>In this film noir classic, a revenge-seeking gangster (Dennis O'Keefe) is sent to prison after being framed for a crime he didn't commit. After seducing a beautiful young woman, he uses her to help him carry out his plot for vengeance, leading him to the crazy pyromaniac (Raymond Burr) who set him up.|
|tt0115610|**Bad Moon** (1996)|**Half man.  Half wolf.  Total terror.**<br/>[Horror]<br/><br/>One man's struggle to contain the curse he hides within... and his last-ditch attempt to free himself with the love of family. But when it looks as if he is losing his battle, and endangering all he holds most dear, the family dog, Thor, is the last hope for his family's survival... and the end to his Werewolf curse.|
|tt0112483|**Best of the Best 3: No Turning Back** (1995)|[Action]<br/><br/>Tommy Lee (Phillip Rhee) comes to visit his sister and finds that the town is being overrun with members of the Arian brotherhood. These Arian brothers certainly weren't expecting a Taekwondo expert to show up. Tommy and his sister's husband played by Christopher Mcdonald team up to stop the racism. whats better than watching neo-nazis get the crap kicked out of them? when the person kicking the crap out of them is dressed like homie the killer clown, complete with big red shoes and a foam nose. and thats just ONE of the MANY many many many many many cinematic jewels contained within this action packed blessing to the silver screen.this is the best worst martial arts film since sonny chiba's street fighter! you don't believe me? see for yourself namby-pants and check out the best of the best 3. now!!!|Christopher McDonald as Sheriff Jack Banning, Gina Gershon as Margo Preston, Mark Rolston as Donnie Hansen
|tt0103850|**Bob Roberts** (1992)|**Vote first.  Ask questions later.**<br/>[Comedy, Drama]<br/><br/>Mock documentary about an upstart candidate for the U.S. Senate written and directed by actor Tim Robbins. Bob Roberts is a folksinger with a difference: He offers tunes that protest welfare chiselers, liberal whining, and the like. As the filmmakers follow his campaign, Robbins gives needle-sharp insight into the way candidates manipulate the media.|Tim Robbins as Bob Roberts, Giancarlo Esposito as Bugs Raplin, David Strathairn as Mack Laflin, James Spader as Chuck Marlin, Jack Black as Roger Davis, Susan Sarandon as Tawna Titan, John Cusack as Cutting Edge Host, Jeremy Piven as Candle Seller, Peter Gallagher as Dan Riley, Bob Balaban as Michael Janes
|tt0095765|**Cinema Paradiso** (1988)|**A celebration of youth, friendship, and the everlasting magic of the movies.**<br/>[Drama, Romance]<br/><br/>A filmmaker recalls his childhood, when he fell in love with the movies at his village's theater and formed a deep friendship with the theater's projectionist.|
|tt0109942|**Dead Tired** (1994)|[Comedy]<br/><br/>Stressed and overworked, famous French movie star Michel Blanc (Michel Blanc) is beginning to wear down, physically and mentally, from the pressure and demands of fame. Already in a fragile state of mind, strange events start to transpire all around him, and he gradually loses his grip. Taking the advice of a psychiatrist, Blanc retreats to the countryside with his friend, Carole Bouquet (Carole Bouquet), but Blanc still has not managed to escape all of his problems.|
|tt0102426|**Mediterraneo** (1991)|**On a magical Greek island a soldier is about to discover that it is better to make love instead of war**<br/>[Comedy, Romance, War]<br/><br/>Greek Sea, World War II. An Italian ship leaves a handful of soldiers in a little island; their mission is to spot enemy ships and to hold the island in case of attack. The village of the island seems abandoned and there isn't a single enemy in sight, so the soldiers begin to relax a little. Things change when their ship is hit and destroyed by the enemy, and the soldiers find themselves abandoned there. Actually, the island isn't deserted and when the Greeks understand that those Italians are harmless, they came out of their hiding places in the mountains and continue their peaceful lives. Soon the soldiers discover that being left behind in a God-forgotten Greek island isn't such a bad thing, after all...|
|tt0105107|**Passion Fish** (1992)|[Drama]<br/><br/>Soap actress May-Alice Culhane is paralyzed from the waist down in an accident. She returns to her old home in Louisiana, where she proceeds to drink quite steadily and to drive away five personal attendants in the first twenty minutes of the movie. Then she meets with Chantelle, whose stubbornness matches her own -- if only because Chantelle, herself a recovering cocaine addict, cannot afford to lose this job. Their mutual dislike gradually develops into an armed truce as the two women deal with their own problems and with each others'.|Mary McDonnell as May-Alice Culhane, Alfre Woodard as Chantelle, Angela Bassett as Rhonda / Dawn, David Strathairn as Rennie
|tt0096257|**The Thin Blue Line** (1988)|**A softcore movie, Dr. Death, a chocolate milkshake, a nosey blonde and "The Carol Burnett Show." Solving this mystery is going to be murder.**<br/>[Mystery, Documentary, Crime]<br/><br/>Errol Morris's unique documentary dramatically re-enacts the crime scene and investigation of a police officer's murder in Dallas.|
|tt0078446|**Up in Smoke** (1978)|**The movie that'll have you rolling in the aisles.**<br/>[Comedy]<br/><br/>An unemployed pot-smoking slacker and amateur drummer, Anthony Stoner ditches his strict parents and hits the road, eventually meeting kindred spirit Pedro de Pacas. While the drug-ingesting duo is soon arrested for possession of marijuana, Anthony and Pedro get released on a technicality, allowing them to continue their many misadventures and ultimately compete in a rock band contest, where they perform the raucous tune "Earache My Eye."|Cheech Marin as Pedro De Pacas, Tom Skerritt as Strawberry
|tt0060196|**The Good, the Bad and the Ugly** (1966)|**For three men the Civil War wasn't hell. It was practice.**<br/>[Western]<br/><br/>While the Civil War rages between the Union and the Confederacy, three men – a quiet loner, a ruthless hit man and a Mexican bandit – comb the American Southwest in search of a strongbox containing $200,000 in stolen gold.|Clint Eastwood as Blondie, Philip Ettington as Pedro (uncredited)
|tt0094336|**Withnail & I** (1987)|**If you don't remember the sixties, don't worry - neither did they.**<br/>[Comedy, Drama]<br/><br/>Two 'resting' actors living in a squalid Camden Flat - and living off a diet of booze and pills - take a trip to a country house (belonging to Withnail’s uncle) to 'rejuvenate'. Faced with bad weather, altercations with the locals, and the unexpected arrival (and advances) of Uncle Monty, the pairs wits and friendship are tested...  Set in 1969, the year in which the hippy dreams of so many young Englishmen went sour, 1986's Bruce Robinson's Withnail and I is an enduring British cult. Withnail is played by the emaciated but defiantly effete Richard E Grant, "I" (i.e., Marwood) by Paul McGann. Out-of-work actors living in desperate penury in a rancid London flat, their lives are a continual struggle to keep warm, alive and in Marwood's case sane, until the pubs open.  A sojourn in the country cottage of Withnail's Uncle Monty only redoubles their privations.|Richard E. Grant as Withnail
|tt0066921|**A Clockwork Orange** (1971)|**Being the adventures of a young man whose principal interests are rape, ultra-violence and Beethoven.**<br/>[Science Fiction, Drama]<br/><br/>Demonic gang-leader Alex goes on the spree of rape, mugging and murder with his pack of "droogs". But he's a boy who also likes Beethoven's Ninth and a bit of "the old in-out, in-out". He later finds himself at the mercy of the state and its brainwashing experiment designed to take violence off the streets.|Malcolm McDowell as Alexander  DeLarge
|tt0056592|**To Kill a Mockingbird** (1962)|[Crime, Drama]<br/><br/>In a small Alabama town in the 1930s, scrupulously honest and highly respected lawyer, Atticus Finch puts his career on the line when he agrees to represent Tom Robinson, a black man accused of rape. The trial and the events surrounding it are seen through the eyes of Finch's six-year-old daughter, Scout. While Robinson's trial gives the movie its momentum, there are plenty of anecdotal occurrences before and after the court date: Scout's ever-strengthening bond with older brother, Jem, her friendship with precocious young Dill Harris, her father's no-nonsense reactions to such life-and-death crises as a rampaging mad dog, and especially Scout's reactions to, and relationship with, Boo Radley, the reclusive 'village idiot' who turns out to be her salvation when she is attacked by a venomous bigot.|Robert Duvall as Arthur 'Boo' Radley, Arthur Tovey as Juror (uncredited)
|tt0095250|**The Big Blue** (1988)|**Danger...Like Passion...Runs Deep**<br/>[Adventure, Drama, Romance]<br/><br/>Two men answer the call of the ocean in this romantic fantasy-adventure.  Jacques (Jean-Marc Barr) and Enzo (Jean Reno) are a pair of friends who have been close since childhood, and who share a passion for the dangerous sport of free diving. Professional diver Jacques opted to follow in the footsteps of his father, who died at sea when Jacques was a boy; to the bewilderment of scientists, Jacques harbors a remarkable ability to adjust his heart rate and breathing pattern in the water, so that his vital signs more closely resemble that of dolphins than men (he even considers a school of dolphins as his extended family).  As Enzo persuades a reluctant Jacques to compete against him in a free diving contest -- determining who can dive deeper and longer without scuba gear -- Jacques meets Johana (Rosanna Arquette), a beautiful insurance investigator from America, and he finds that he must choose between his love for her and his love of the sea.|Jean Reno as Enzo Molinari
|tt0092610|**Bad Taste** (1987)|[Action, Comedy, Horror, Science Fiction]<br/><br/>A team from the intergalactic fast food chain Crumb's Crunchy Delights descends on Earth, planning to make human flesh the newest taste sensation. After they wipe out the New Zealand town Kaihoro, the country's Astro-Investigation and Defense Service is called in to deal with the problem. Things are complicated due to Giles, an aid worker who comes to Kaihoro the same day to collect change from the residents. He is captured by the aliens, and AIaDS stages a rescue mission that quickly becomes an all-out assault on the aliens' headquarters.|
|tt0056218|**The Manchurian Candidate** (1962)|**When you've seen it all, you'll swear there's never been anything like it!**<br/>[Mystery, Thriller]<br/><br/>The Manchurian Candidate is a political thriller from American director John Frankenheimer. An American soldier is brainwashed into being a killer for the communist Russians during the Korean War.|Bess Flowers as Woman in Dream Sequence
|tt0052618|**Ben-Hur** (1959)|**The entertainment experience of a lifetime.**<br/>[Action, Adventure, Drama, History]<br/><br/>Ben-Hur is a 1959 epic film directed by William Wyler, the third film version of Civil War vet Lew Wallace's 1880 novel Ben-Hur: A Tale of the Christ. It premiered at Loew's State Theatre in New York City on November 18, 1959. The movie's reputation as a classic is primarily based on two spectacular action sequences:  the great chariot race and a Roman naval battle, along with lavish production values and strong performances.  The plot of Ben Hur revolves around a Jewish prince who is betrayed and sent into slavery by a Roman friend and how he regains his freedom and comes back for revenge. However, instead he finds redemption in Christ, the theme is ultimately about being saved in the Christian sense.  The film went on to win a record of eleven Academy Awards, including Best Picture and Best Actor (Charlton Heston as Ben Hur). This record-setting Oscars sweep has since been equaled by Titanic in 1998 and The Lord of the Rings: The Return of the King in 2004, but never broken.|Charlton Heston as Judah Ben-Hur
|tt0084503|**Pink Floyd: The Wall** (1982)|**The Memories. The Madness. The Music... The Movie.**<br/>[Music]<br/><br/>A troubled rock star descends into madness in the midst of his physical and social isolation from everyone.|Bob Hoskins as Rock and Roll Manager
|tt0101458|**Until the End of the World** (1991)|**. . . the ultimate road movie**<br/>[Thriller, Drama, Science Fiction, Action]<br/><br/>Until The End of the World is an odyssey for the modern age. As with Homer's Odyssey, the purpose of the journey is to restore sight -- a spiritual reconciliation between an obsessed father and a deserted son. Dr. Farber, in trying to find a cure for his wife's blindness, has created a device that allows the user to send images directly to the brain, enabling the blind to see.|William Hurt as Sam Farber, alias Trevor McPhee, Sam Neill as Eugene Fitzpatrick, Max von Sydow as Henry Farber
|tt0116589|**Hype!** (1996)|[Documentary, Music]<br/><br/>This documentary examines the Seattle scene as it became the focus of a merging of punk rock, heavy metal, and innovation. Building from the grass roots, self-promoted and self-recorded until break-out success of bands like Nirvana brought the record industry to the Pacific Northwest, a phenomenon was born.|
|tt0085159|**Amityville 3-D** (1983)|**WARNING: in this movie you are the victim.**<br/>[Horror, Thriller]<br/><br/>To debunk the Amityville house's infamous reputation and take advantage of a rock-bottom asking price, skeptical journalist John Baxter buys the place and settles in to write his first novel. But as soon as the ink on the deed has dried, people who have come into contact with John and the house begin to meet with shocking fates. Is it all just coincidence, or is the house really the gateway to hell?|Meg Ryan as Lisa
|tt0087800|**A Nightmare on Elm Street** (1984)|**If Nancy Doesn't Wake Up Screaming, She Won't Wake Up At All!**<br/>[Horror]<br/><br/>Teenagers in a small town are dropping like flies, apparently in the grip of mass hysteria causing their suicides. A cop's daughter, Nancy Thompson (Heather Langenkamp) traces the cause to child molester Fred Krueger (Robert Englund), who was burned alive by angry parents many years before. Krueger has now come back in the dreams of his killers' children, claiming their lives as his revenge. Nancy and her boyfriend, Glen (Johnny Depp), must devise a plan to lure the monster out of the realm of nightmares and into the real world...|Johnny Depp as Glen Lantz, Robert Englund as Freddy Krueger, Charles Fleischer as Dr. King
|tt0115495|**Albino Alligator** (1996)|**They Planned The Perfect Crime... Until It All Went Perfectly Wrong!**<br/>[Crime, Drama, Thriller]<br/><br/>Three petty thieves who the police believe to be major criminals are chased into a basement bar where they take five hostages including all the bar employees. The rest of the movie deals with the cops lurking outside the bar while the trio try to get hold of the situation inside.|Matt Dillon as Dova, Faye Dunaway as Janet Boudreaux, Viggo Mortensen as Guy Foucard, Skeet Ulrich as Danny Boudreaux, Frankie Faison as Agent Marv Rose, M. Emmet Walsh as Dino, Joe Mantegna as A.T.F. Agent G.D. Browning
|tt0084116|**Identification of a Woman** (1982)|[Romance, Drama]<br/><br/>The movie director Niccolò has just been left by his wife. Subsequently he embarks on an obsessive relationship with a young woman who eventually leaves him and disappears (shades also of L'Avventura) while searching for her, he meets a variety of other willing girls. This gives him the idea of making a movie about women's relationships. He starts to search for a woman who can play the leading part in the movie.|
|tt0117372|**The Preacher's Wife** (1996)|**They needed help. What they got was a miracle.**<br/>[Fantasy, Drama, Comedy, Science Fiction, Romance, Family]<br/><br/>Good natured Reverend Henry Biggs finds that his marriage to choir mistress Julia is flagging, due to his constant absence caring for the deprived neighborhood they live in. On top of all this, his church is coming under threat from property developer Joe Hamilton. In desperation, Rev. Biggs prays to God for help - and help arrives in the form of an angel named Dudley.|Denzel Washington as Dudley
|tt0113557|**Zero Kelvin** (1995)|[Drama, Thriller]<br/><br/>Larsen, an aspiring poet in '20s Oslo, leaves his girlfriend to spend a year as a trapper in East Greenland. There he is teamed with a seemingly rough old sailor/trapper, Randbæk, and a scientist, Holm. Trapped in a tiny hut together as the Arctic winter sets in, a complex and intense love/hate relationship develops between Randbæk and Larsen, who are more similar than either would like to admit. A powerful psychological and physical drama set against stunningly bleak Arctic scenery.|Stellan Skarsgård as Randbæk
|tt0115988|**The Crucible** (1996)|**Arthur Miller's timeless tale of truth on trial.**<br/>[Drama, History]<br/><br/>The movie is centered around the Salem Massachusetts witch trials of 1692. The movie is based on the play "The Crucible" by Arthur Miller. He also wrote the screen play adaptation.|Daniel Day-Lewis as John Proctor, Winona Ryder as Abigail Williams, Bruce Davison as Reverend Parris, Jeffrey Jones as Thomas Putnam, Peter Vaughan as Giles Corey
|tt0042354|**The Forbidden Christ** (1951)|[Drama]<br/><br/>Strange Deception combines a standard revenge tale with a postwar reenactment of the first four books of the New Testament. Freshly released from a Russian POW camp, Italian soldier Raf Vallone tries to discover who betrayed his brother to the Nazis. Alain Cuny is an enigmatic carpenter who has confessed to causing the brother's death. Cuny is slain by Vallone, whereupon it is revealed that the carpenter sacrificed himself on behalf of the real culprit, Phillipe Lemaire. Vallone catches up with Lemaire, but is unable to kill him, thanks to the Christlike example of Cuny. Originally titled Il Cristo Proibito (The Forbidden Christ, just so we don't miss the point), this film represented the movie directorial debut of novelist Curzio Malaparte, who also wrote the musical score.|
|tt0077631|**Grease** (1978)|**Grease is the word**<br/>[Romance]<br/><br/>Australian good girl Sandy and greaser Danny fell in love over the summer. But when they unexpectedly discover they're now in the same high school, will they be able to rekindle their romance despite their eccentric friends?|John Travolta as Danny Zuko, Stockard Channing as Betty Rizzo
|tt0100114|**Marked for Death** (1990)|**He's a good cop. In a bad mood.**<br/>[Action, Thriller]<br/><br/>Just retired from the Drug Enforcement Agency, John Hatcher (Seagal) returns to his hometown and quickly discovers that drugs have infiltrated his old neighborhood. Determined to drive the dealers out, Hatcher crosses pathes with a ferocious Jamaican druglord who vowes that Hatcher and his family are now marked for death.|Keith David as Max, Kevin Dunn as Sal, Danny Trejo as Hector
|tt0116695|**Jerry Maguire** (1996)|**Everybody loved him... Everybody disappeared.**<br/>[Comedy, Drama, Romance]<br/><br/>Jerry Maguire used to be a typical sports agent: willing to do just about anything he could to get the biggest possible contracts for his clients, plus a nice commission for himself. Then, one day, he suddenly has second thoughts about what he's really doing. When he voices these doubts, he ends up losing his job and all of his clients, save Rod Tidwell, an egomaniacal football player.|Tom Cruise as Jerry Maguire, Cuba Gooding Jr. as Rod Tidwell, Renée Zellweger as Dorothy Boyd, Kelly Preston as Avery Bishop, Bonnie Hunt as Laurel Boyd, Donal Logue as Rick, Eric Stoltz as Ethan Valhere
|tt0115633|**Bastard Out of Carolina** (1996)|[Drama]<br/><br/>Difficult tale of poor, struggling South Carolinian mother &amp; daughter, who each face painful choices with their resolve and pride. Bone, the eldest daughter, and Anney her tired mother, grow both closer and farther apart: Anney sees Glen as her last chance.  The film won an Emmy Award for "Outstanding Casting for a Miniseries or a Special" and was nominated for "Outstanding Directing for a Miniseries or a Special", "Outstanding Supporting Actress in a Miniseries or a Special", and "Outstanding Made for Television Movie". It was screened in the Un Certain Regard section at the 1996 Cannes Film Festival.|Jennifer Jason Leigh as Anney Boatwright, Christina Ricci as Dee Dee
|tt0116410|**Ghosts of Mississippi** (1996)|**From the director of A FEW GOOD MEN comes a compelling TRUE STORY.**<br/>[History, Drama]<br/><br/>Ghosts of Mississippi is a drama covering the final trial of the assassin, Bryon De La Beckwith, of the 60s civil rights leader Medgar Evers. It begins with the murder and the events surrounding the two initial trials which both ended in a hung jury. The movie then covers District Attorney, Bobby DeLaughters transformation and alliance with Myrlie Evers, wife of Medgar Evers, of the, as he becomes more involved with bringing Beckwith to trial for the third time 30 years later. Some of the characters are played by the actual participants in this story.|Alec Baldwin as Bobby DeLaughter, James Woods as Byron De La Beckwith, Whoopi Goldberg as Myrlie Evers, William H. Macy as Charlie Crisco
|tt0117571|**Scream** (1996)|**Someone has taken their love of scary movies one step too far.**<br/>[Crime, Horror, Mystery]<br/><br/>A killer known as Ghostface begins killing off teenagers, and as the body count begins rising, one girl and her friends find themselves contemplating the 'rules' of horror films as they find themselves living in a real-life one.|David Arquette as Deputy Dwight 'Dewey' Riley, Matthew Lillard as Stu Macher, Skeet Ulrich as Billy Loomis, Drew Barrymore as Casey Becker, Liev Schreiber as Cotton Weary
|tt0116640|**Inside** (1996)|[Crime, Drama, TV Movie]<br/><br/>An emotional and thrilling tale of life and death behind the walls of a maximum security prison. From the acclaimed director of 'Bonnie and Clyde', and featuring a powerful international cast.|Nigel Hawthorne as Colonel, Eric Stoltz as Marty, Louis Gossett, Jr. as Questioner
|tt0080714|**Falling in Love Again** (1980)|[Drama, Romance]<br/><br/>Harry and Sue Lewis met in the 40s as teenagers living in the Bronx. He was an aspiring architect, she was the most beautiful girl in school, and both had a fondness for bran muffins. They fell in love, got married, moved to Los Angeles, and had two kids. While struggling with his midlife crisis, Harry receives an invitation for his high school's reunion back so he takes Sue and their teenage kids on a cross-country car trip back to the Big Apple. Will they see in the Bronx what they expected? Will the good memories from their past help rekindle their fading love? Is it too late to dream?|Elliott Gould as Harry Lewis (Present day), Michelle Pfeiffer as Sue Wellington (1940's)
|tt0106266|**Amos & Andrew** (1993)|**Misunderstood. Misplaced. Mismatched.**<br/>[Action, Comedy]<br/><br/>When Andrew Sterling (Samuel L. Jackson), a successful black urbanite writer, buys a vacation home on a resort in New England the police mistake him for a burglar. After surrounding his home with armed men, Chief Tolliver (Dabney Coleman) realizes his mistake and to avoid the bad publicity offers a thief in his jail, Amos Odell (Nicolas Cage) a deal.|Samuel L. Jackson as Andrew Sterling, Michael Lerner as Phil Gillman, Nicolas Cage as Amos Odell, Brad Dourif as Officer Donnie Donaldson
|tt0118111|**Waiting for Guffman** (1996)|**There's a good reason some talent remains undiscovered.**<br/>[Music, Comedy]<br/><br/>Corky St. Clair is a director, actor and dancer in Blaine, Missouri. When it comes time to celebrate Blaine's 150th anniversary, Corky resolves to bring down the house in Broadway style in this hilarious mockumentary from the people who brought you "This is Spinal Tap!"|Eugene Levy as Dr. Allan Pearl, Catherine O'Hara as Sheila Albertson, Larry Miller as Glenn Welsch, Parker Posey as Libby Mae Brown, Bob Balaban as Lloyd Miller, Paul Dooley as UFO Abductee, Brian Doyle-Murray as Red Savage
|tt0119951|**Private Parts** (1997)|**Never before has a man done so much with so little.**<br/>[Comedy, Drama]<br/><br/>The auto-biographical story of Howard Stern, the radio-rebel who is now also a TV-personality, an author and a movie star.|Paul Giamatti as Kenny 'Pig Vomit' Rushton, Allison Janney as Dee Dee, Mia Farrow as Herself
|tt0118064|**The Van** (1996)|**A comedy About Friends, Family and Fish.**<br/>[Comedy, Drama]<br/><br/>Set in "Barrytown", a fictitious working-class quarter of Dublin. "Bimbo" Reeves gets laid off from his job. With his redundancy, he buys a van and sells fish and chips with his buddy Larry. Due to Ireland's surprising success at the 1990 FIFA World Cup, their business starts off well, but the relationship between the two friends soon becomes strained as Bimbo behaves more like a typical boss.|Colm Meaney as Larry
|tt0118829|**Cats Don't Dance** (1997)|**It's a Jungle Out There!**<br/>[Animation, Comedy, Family, Music]<br/><br/>Danny, an ambitious singing/dancing cat, goes to Hollywood and overcomes several obstacles to fulfill his dream of becoming a movie star.|
|tt0120389|**Turbo: A Power Rangers Movie** (1997)|**The Power Is Back!**<br/>[Action, Adventure, Family, Fantasy, Science Fiction, Thriller]<br/><br/>The legendary Power Rangers must stop the evil space pirate Divatox from releasing the powerful Maligore from his volcanic imprisonment on the island of Muranthias, where only the kindly wizard Lerigot has the key to release him. The hope of victory lies in the Ranger's incredible new Turbo powers and powerful Turbo Zords.|
|tt0116762|**Keys to Tulsa** (1997)|**Two Men Taken By One Woman**<br/>[Crime, Drama, Thriller]<br/><br/>Richter Boudreau is on a bad streak: Languishing in the shadow of his celebrity mother, he loses his job as a film critic for the town paper, and now he's been approached with a dangerous proposition that ultimately leads to blackmail. Richter's friend Ronnie ropes him into a scheme to steal the inheritance of his wife, Vicky.|Eric Stoltz as Richter Boudreau, James Spader as Ronnie Stover, Michael Rooker as Keith Michaels, James Coburn as Harmon Shaw, Cameron Diaz as Trudy
|tt0120087|**The Second Jungle Book: Mowgli & Baloo** (1997)|[Adventure, Family]<br/><br/>In this exciting live-action adventure, young Mowgli, an orphan raised by wolves, is spotted by a scout for a giant circus. Accompanied by a cruel hunter and a snake charmer, the scout sets out to trap Mowgli. But with the help of Baloo the bear and Bagheera the panther, little Mowgli leads the adults into his biggest and wildest adventure yet! A fun-filled movie every member of the family will enjoy.|Roddy McDowall as King Murphy
|tt0118127|**Wedding Bell Blues** (1996)|[Comedy, Romance]<br/><br/>Micki, Tanya and Jasmine have 24 hours to get divorced. There's just one problem...They're still single.  When Jasmine, Tanya and Micki decide they'd rather be divorced than unmarried, they take off to Las Vegas in search of the perfect ex. In the tangled web that follows, Micki lassos a cowboy, Tanya acquires a rich businessman, and Jasmine finds true love in the arms of a regular guy. The result is an outrageous comedy of love, marriage and misadventure.  As intelligent as it is absurd, Wedding Bell Blues pokes great fun at the twists and turns of modern love, with outstanding performances from a talented young cast, including Illeana Douglas, Paulina Porizkova and Julie Warner, it is without a doubt one of the year's most hilarious movies.|Illeana Douglas as Jasmine, Richard Edson as Tom
|tt0120034|**Roseanna's Grave** (1997)|**A romantic comedy about the things we do for love.**<br/>[Comedy, Romance, Drama]<br/><br/>Roseanna is dying of a heart condition, and all she wants is to be buried next to her daughter, in a cemetery that is getting full fast. The cemetery can't expand because Capestro, the man who owns the land next to the cemetery, won't sell. While Marcello is doing good deeds to make sure no one dies, Roseanna thinks of Marcello's future.|Jean Reno as Marcello
|tt0105569|**Tetsuo II: Body Hammer** (1992)|[Science Fiction, Horror, Drama, Thriller]<br/><br/>Tetsuo II: Body Hammer is a 1992 Japanese science fiction/horror film directed by Shinya Tsukamoto. It is a bigger-budget reworking of the same director's 1989 movie Tetsuo: The Iron Man, utilizing similar themes and ideas to the earlier film (a Japanese salaryman, played by cult actor Tomorowo Taguchi, finds his body transforming into a weapon through sheer rage after his son is kidnapped by a gang of violent thugs.) It was not as well received as its predecessor but it did win the Critic's Award at the 3rd Yubari International Fantastic Film Festival in February 1992.|
|tt0119094|**Face/Off** (1997)|**In order to catch him, he must become him.**<br/>[Action, Crime, Science Fiction, Thriller]<br/><br/>An antiterrorism agent goes under the knife to acquire the likeness of a terrorist and gather details about a bombing plot. When the terrorist escapes custody, he undergoes surgery to look like the agent so he can get close to the agent's family.|John Travolta as Sean Archer, Nicolas Cage as Castor Troy, Gina Gershon as Sasha Hassler, John Carroll Lynch as Walt
|tt0116334|**For Ever Mozart** (1996)|[Drama, Comedy, War]<br/><br/>For Ever Mozart is an episodic film that follows a theater troupe from France attempting to put on a play in Sarajevo. Along their journey they are captured and held in a POW camp, and they call for help from their friends and relations in France. Director Jean-Luc Godard presents stories about this troop to ask how one can make art while slaughters like the one in Bosnia are taking place, and he throws in a strong critique of the European Union. For Ever Mozart is one of Godard's most disjointed and difficult films. Its stories sometimes seem to form a whole and at other times the links among them are unclear. One gets the impression that in each episode Godard attempts to start a film only to come to the conclusion that it is impossible to continue. It features some of the most beautiful shots of tanks in the cinema.|
|tt0118884|**Contact** (1997)|**If it's just us, it seems like an awful waste of space.**<br/>[Drama, Science Fiction, Mystery]<br/><br/>Contact is a science fiction film about an encounter with alien intelligence. Based on the novel by Carl Sagan the film starred Jodie Foster as the one chosen scientist who must make some difficult decisions between her beliefs, the truth, and reality.|Jodie Foster as Eleanor Arroway, Matthew McConaughey as Palmer Joss, James Woods as Michael Kitz, John Hurt as S. R. Hadden, Tom Skerritt as David Drumlin, David Morse as Ted Arroway, Angela Bassett as Rachel Constantine
|tt0118887|**Cop Land** (1997)|**No One Is Above The Law.**<br/>[Action, Crime, Drama]<br/><br/>Freddy Heflin is the sheriff of a place everyone calls “Cop Land” — a small and seemingly peaceful town populated by the big city police officers he’s long admired. Yet something ugly is taking place behind the town’s peaceful facade. And when Freddy uncovers a massive, deadly conspiracy among these local residents, he is forced to take action and make a dangerous choice between protecting his idols and upholding the law.|Sylvester Stallone as Sheriff Freddy Heflin, Harvey Keitel as Ray Donlan, Ray Liotta as Gary " Figgsy " Figgis, Robert De Niro as Lt. Moe Tilden, Michael Rapaport as Murray " Superboy " Babitch, Annabella Sciorra as Liz Randone, Janeane Garofalo as Deputy Cindy Betts, Cathy Moriarty as Rose Donlan, Paul Calderon as Hector - Medic, Philip Ettington as Detective Strickland (uncredited)
|tt0118570|**Air Bud** (1997)|**He Sits. He Stays. He Shoots. He Scores.**<br/>[Comedy, Family, Drama]<br/><br/>A young boy and a talented stray dog with an amazing basketball playing ability become instant friends. Rebounding from his father's accidental death, 12-year-old Josh Framm moves with his family to the small town of Fernfield, Washington. The new kid in town, Josh has no friends and is too shy to try out for the school basketball team. Instead he prefers to practice alone on an abandoned court, he befriends a runaway golden retriever named Buddy. Josh is amazed when he realizes that Buddy loves basketball...that is playing basketball...and he is GOOD! Josh eventually makes the school team and Buddy is named the Team Mascot. Josh and Buddy become the stars of halftime. Buddy's half-time talent draws media attention. Unfortunately, when Buddy's mean former owner, Norm Snively, comes along with a scheme to cash in on the pup's celebrity, it looks like they are going to be separated.|Michael Jeter as Norm Snively, Bill Cobbs as Arthur Chaney
|tt0119509|**Leave it to Beaver** (1997)|**The Beav is back.**<br/>[Adventure, Comedy, Drama, Family]<br/><br/>Cleavers are an all-American family living in Ohio - wise father Ward, loving mother June, teen-age son Wally and 8-year-old "Beaver" Theodore. Beaver hopes to get a bike as a gift from his father and to please him tries out for his school football team and he makes it, only to be embarrassed. The bike he gets is quickly stolen. Meanwhile Wally is trying to help his friend Eddie Haskell to get the heart of pretty classmate Karen, but Karen seems to like Wally more, and that leads to tensions between the friends.|Christopher McDonald as Ward Cleaver
|tt0119695|**Money Talks** (1997)|**This ain't no buddy movie.**<br/>[Action, Adventure, Comedy]<br/><br/>Money Talks is a 1997 American comedy film directed by Brett Ratner. Sought by police and criminals, a small-time huckster makes a deal with a TV newsman for protection.|Charlie Sheen as James Russell, Paul Sorvino as Guy Cipriani
|tt0119457|**Kicked in the Head** (1997)|**A satirical love story.**<br/>[Comedy, Crime, Drama, Romance]<br/><br/>Redmond is a young guy who can't find what to do with his life. When his uncle Sam gives him the bag to deliver to some uptown connection he fails to do so and it gets them in trouble with Jack, low-key criminal. After that tough guy Stretch wants Redmond to take part in his illegal beer business, but before Redmond gets involved, the business ends in a bad way. Redmond is also having affair with flight attendant Megan.|Kevin Corrigan as Redmond, Michael Rapaport as Stretch, Lili Taylor as Happy, James Woods as Uncle Sam, Burt Young as Jack
|tt0120151|**A Smile Like Yours** (1997)|**The shortest distance between two people is a certain smile.**<br/>[Comedy, Romance]<br/><br/>A comedy about a couple who cannot conceive a baby|Joan Cusack as Nancy Tellen, Christopher McDonald as Richard Halstrom, Donald Moffat as Dr. Felber, Shirley MacLaine as Martha (uncredited)
|tt0118842|**Chasing Amy** (1997)|**It's not who you love. It's how.**<br/>[Comedy, Drama, Romance]<br/><br/>Holden and Banky are comic book artists. Everything is going good for them until they meet Alyssa, also a comic book artist. Holden falls for her, but his hopes are crushed when he finds out she's a lesbian.|Ben Affleck as Holden McNeil, Joey Lauren Adams as Alyssa Jones, Matt Damon as Shawn Oran, Illeana Douglas as Alyssa's Roommate (uncredited)
|tt0119280|**Mrs Brown** (1997)|**Loyalty without question. Friendship without equal.**<br/>[Drama, History, Romance]<br/><br/>Queen Victoria is deeply depressed after the death of her husband, disappearing from public. Her servant Brown, who adores her, through caress and admiration brings her back to life, but that relationship creates scandalous situation and is likely to lead to monarchy crisis.|Judi Dench as Queen Victoria
|tt0119107|**Fast, Cheap & Out of Control** (1997)|[Documentary]<br/><br/>Hilarious, sad, absurd, eerie and beautiful, "FAST, CHEAP &amp; OUT OF CONTROL" is a film like no other. Starting as a darkly funny contemplation of the Sisyphus-like nature of human striving, it ultimately becomes a profoundly moving meditation on the very nature of existence.|
|tt0120539|**Year of the Horse** (1997)|[Documentary, Music]<br/><br/>Indie director Jim Jarmusch lenses a low-tech tribute to protean rocker Neil Young and his long-standing band, Crazy Horse. Stitched together from archival material shot in 1976 and 1986 along with candid scenes of Young and the band kicking back between shows, this rockumentary is as ragged as it is direct. Concert performances include renditions of hits such as "Sedan Delivery" and "Like a Hurricane."|
|tt0120257|**Swept from the Sea** (1998)|[Drama, Romance]<br/><br/>The film tells the story of Russian emigree and the only survivor from ship crash Yanko Goorall and servant Amy Foster in the end of 19th century. When Yanko enters a farm sick and hungry after the shipwreck, everyone is afraid of him, except for Amy, who is very kind and helps him. Soon he becomes like a son for Dr. James Kennedy and romance between Yanko and Amy follows.|Ian McKellen as Dr. James Kennedy, Kathy Bates as Miss Swaffer, Joss Ackland as Mr. Swaffer
|tt0119832|**One Night Stand** (1997)|**Sometimes an entire life can change in just one night.**<br/>[Drama]<br/><br/>In Los Angeles, Max Carlyle makes a good living directing commercials and has a happy home life with his wife, Mimi, and two children. When Carlyle travels to New York City to visit his friend Charlie, who has been diagnosed with AIDS, he has repeat run-ins with a beautiful woman, Karen, and eventually sleeps with her. Though he goes home the next day and doesn't return until a year later, Carlyle's infidelity still lingers.|Wesley Snipes as Max Carlyle, Nastassja Kinski as Karen, Robert Downey Jr. as Charlie
|tt0120201|**Starship Troopers** (1997)|**The only good bug is a dead bug.**<br/>[Adventure, Action, Thriller, Science Fiction]<br/><br/>Set in the future, the story follows a young soldier named Johnny Rico and his exploits in the Mobile Infantry. Rico's military career progresses from recruit to non-commissioned officer and finally to officer against the backdrop of an interstellar war between mankind and an arachnoid species known as "the Bugs".|Michael Ironside as Lieutenant Jean Rasczak, Dean Norris as Commanding Officer
|tt0123385|**Artemisia** (1997)|**Her passion for her art changed the face of history.**<br/>[Drama, History]<br/><br/>Artemisia Gentileschi (1593-1653) was one of the first well-known female painters. The movie tells the story of her youth, when she was guided and protected by her father, the painter Orazio Gentileschi. Her professional curiosity about the male anatomy, forbidden for her eyes, led her to the knowledge of sexual pleasure. But she was also well known because in 1612 she had to appear in a courtroom because her teacher, Agostino Tassi, was suspected of raping her. She tried to protect him, but was put in the thumb screws...|
|tt0109266|**Bang** (1995)|**Sometimes you've got to break the rules**<br/>[Drama]<br/><br/>A young woman in L.A. is having a bad day: she's evicted, an audition ends with a producer furious she won't trade sex for the part, and a policeman nabs her for something she didn't do, demanding fellatio to release her. She snaps, grabs his gun, takes his uniform, and leaves him cuffed to a tree where he's soon having a defenseless chat with a homeless man. She takes off on the cop's motorcycle and, for an afternoon, experiences a cop's life. She talks a young man out of suicide and then is plunged into violence after a friendly encounter with two "vatos." She is torn between self-protection and others' expectations. Is there any resolution for her torrent of feelings?|
|tt0119217|**Good Will Hunting** (1997)|**Some people can never believe in themselves, until someone believes in them.**<br/>[Drama]<br/><br/>Will Hunting has a genius-level IQ but chooses to work as a janitor at MIT. When he solves a difficult graduate-level math problem, his talents are discovered by Professor Gerald Lambeau, who decides to help the misguided youth reach his potential. When Will is arrested for attacking a police officer, Professor Lambeau makes a deal to get leniency for him if he will get treatment from therapist Sean Maguire.|Robin Williams as Sean Maguire, Matt Damon as Will Hunting, Ben Affleck as Chuckie Sullivan, Stellan Skarsgård as Prof. Gerald Lambeau, Minnie Driver as Skylar
|tt0119668|**Midnight in the Garden of Good and Evil** (1997)|[Crime, Drama, Mystery, Thriller]<br/><br/>A visiting city reporter's assignment suddenly revolves around the murder trial of a local millionaire, whom he befriends.|John Cusack as John Kelso, Kevin Spacey as James Williams, Bob Gunton as Finley Largent
|tt0118230|**Full Speed** (1996)|[Romance, Drama]<br/><br/>A brief extract of four kids' lives somewhere in France. Quentin, who won a writers contest and now pays more attention to his career as an author than to his friends, beautiful Julie, his girl-friend, much more mature than she looks, falling in love with Quentin's very best friend Jimmy, who is kind of stuck in his unability of self-expression and grown up under bad social circumstances. And there is the shy boy Samir, exiled from Algeria, who lost his "brother" and only friend some time ago. Samir heavily falls in love with Quentin, but he can't handle it...|
|tt0118715|**The Big Lebowski** (1998)|**Times like these call for a Big Lebowski.**<br/>[Comedy, Crime]<br/><br/>Jeffrey "The Dude" Lebowski, a Los Angeles slacker who only wants to bowl and drink white Russians, is mistaken for another Jeffrey Lebowski, a wheelchair-bound millionaire, and finds himself dragged into a strange series of events involving nihilists, adult film producers, ferrets, errant toes, and large sums of money.|Jeff Bridges as The Dude, John Goodman as Walter Sobchak, Julianne Moore as Maude Lebowski, Steve Buscemi as Theodore Donald "Donny" Kerabatsos, Philip Seymour Hoffman as Brandt, John Turturro as Jesus Quintana, David Thewlis as Knox Harrington, Jon Polito as Da Fino
|tt0119590|**My Life in Pink** (1997)|**Sometimes you just have to be yourself.**<br/>[Drama, Comedy]<br/><br/>Ludovic is a small boy who cross-dresses and generally acts like a girl, talks of marrying his neighbor's son and can not understand why everyone is so surprised about it. His actions lead to problems for him and his family.|
|tt0120594|**B. Monkey** (1998)|**Trouble never looked so good.**<br/>[Romance, Crime, Drama]<br/><br/>When wistful introvert Alan Furnace meets quick-witted bombshell Beatrice, he has no idea of her secret life as "B. Monkey" -- the top thief-for-hire in London's criminal underworld. Charmed by Furnace's innocent and chivalrous ways, Beatrice resolves to reform. But to cash in on her first chance at real love, she must escape her former partner in crime, the ruthless Paul Neville -- and a dark past that seems to haunt her every step.|Jared Harris as Alan Furnace, Rupert Everett as Paul Neville, Ian Hart as Steve Davis
|tt0120888|**The Wedding Singer** (1998)|**He's gonna party like it's 1985.**<br/>[Romance, Comedy]<br/><br/>Robbie, a local rock star turned wedding singer, is dumped on the day of his wedding. Meanwhile, waitress Julia finally sets a wedding date with her fiancée Glenn. When Julia and Robbie meet and hit it off, they find that things are more complicated than anybody thought.|Adam Sandler as Robbie Hart, Drew Barrymore as Julia Sullivan, Jon Lovitz as Jimmie Moore, Steve Buscemi as David 'Dave' Veltri
|tt0120782|**Palmetto** (1998)|**In a town this bad, it's hard to be good...**<br/>[Mystery, Thriller, Drama, Romance, Crime]<br/><br/>A recently released ex-con gets involved in a fake kidnapping scheme that turns very real.|Woody Harrelson as Harry Barber, Gina Gershon as Nina, Elisabeth Shue as Mrs. Donnelly / Rhea Malroux, Michael Rapaport as Donnely
|tt0119822|**As Good as It Gets** (1997)|**A comedy from the heart that goes for the throat.**<br/>[Comedy, Romance]<br/><br/>New York City. Melvin Udall, a cranky, bigoted, obsessive-compulsive writer, finds his life turned upside down when neighboring gay artist Simon is hospitalized and his dog is entrusted to Melvin. In addition, Carol, the only waitress who will tolerate him, must leave work to care for her sick son, making it impossible for Melvin to eat breakfast.|Jack Nicholson as Melvin Udall, Cuba Gooding Jr. as Frank Sachs, Skeet Ulrich as Vincent Lopiano, Harold Ramis as Dr. Bettes, Brian Doyle-Murray as Handyman, Matt Malloy as Men's Store Salesman
|tt0119574|**Love and Death on Long Island** (1997)|[Drama, Romance, Foreign]<br/><br/>Giles De'Ath is a widower who doesn't like anything modern. He goes to movies and falls in love with film star, Ronnie Bostock. He then investigates everything about the movie and Ronnie. After that he travels to Long Island city where Ronnie lives and meets him, pretending that Ronnie is a great actor and that's why Giles admires him.|John Hurt as Giles De'Ath, Maury Chaykin as Irving 'Irv' Buckmuller
|tt0120890|**Wild Things** (1998)|**They're dying to play with you.**<br/>[Thriller]<br/><br/>When teen-socialite Kelly Van Ryan (Richards) and troubled bad girl Suzie Toller (Campbell) accuse guidance counselor Sam Lombardo (Dillon) of rape, he's suspended by the school, rejected by the town, and fighting to get his life back. One cop (Bacon) suspects conspiracy, but nothing is what it seems...|Matt Dillon as Sam Lombardo, Kevin Bacon as Sergeant Ray Duquette, Bill Murray as Kenneth Bowden
|tt0119780|**Niagara, Niagara** (1997)|**Love can't be controlled.**<br/>[Drama, Romance]<br/><br/>"Niagra, Niagra" begins quietly in a drugstore in Poughkeepsie, where Marcie, the film's disarming heroine, likes to shoplift. She literally crashes into Seth, a quiet outsider, also on a shoplifting spree. Marcie invites Seth to accompany her to Canada to find a black hairstyling head. They set off in Seth's beat-up station wagon, destined for a toy store in Toronto. While on the road, Marcie confides to Seth that she has Tourette's syndrome, necessitating a series of detours to liquor stories and pharmacies along the roads of upstate New York.|
|tt0120510|**Wide Awake** (1998)|**Meeting your best friend. Finding your favorite teacher. Having your first crush. Remember what it felt like to be...**<br/>[Comedy, Drama, Family]<br/><br/>A feel-good comedy about the laughter, excitement and fun that come with being a kid! Joshua is a thoughtful 10-year-old looking for some simple answers to life's eternal questions. The problem is that no one - including his concerned parents and his 5th grade teacher - seems to be making any sense!|Denis Leary as Mr. Beal, Rosie O'Donnell as Sister Terry
|tt0120765|**My Giant** (1998)|**A comedy of incredible proportions.**<br/>[Comedy, Drama, Family]<br/><br/>Billy Crystal plays a Hollywood agent who stumbles upon Max, a giant living in Romania, and tries to get him into the movies.|Billy Crystal as Sam 'Sammy' Kamin, Kathleen Quinlan as Serena Kamin, Joss Ackland as Monsignor Popesou
|tt0120213|**Slappy and the Stinkers** (1998)|**Five Kids. One Sea-Lion. No Rules.**<br/>[Adventure, Family]<br/><br/>Five 2nd-grade kids who don't follow strict rules by their school principal Brinway are dubbed "Stinkers" by him. On the class visit to an aquarium the Stinkers decide that a sea lion called Slappy doesn't feel too good there, "free" him, and plant him into Brinway's hot-tub.|
|tt0120906|**Zero Effect** (1998)|**The world's most private detective.**<br/>[Comedy, Crime, Mystery, Thriller]<br/><br/>Daryl Zero is a private investigator. Along with his assistant, Steve Arlo he solves impossible crimes and puzzles. Though a master investigator, when he is not working, Zero doesn't know what to do with himself. He has no social skills, writes bad music, and drives Arlo crazy. In his latest case, Zero must find out who is blackmailing a rich executive, and when his client won't tell him, why.|Bill Pullman as Daryl Zero, Ben Stiller as Steve Arlo
|tt0118577|**An Alan Smithee Film: Burn, Hollywood, Burn** (1998)|[Comedy]<br/><br/>Filmmaker Alan Smithee finds himself the unwilling puppet of a potentially bad, big budget action film which he proceeds to steal the reels and leave the cast and crew in a frenzy.|Jackie Chan as Jackie Chan, Sylvester Stallone as Sylvester Stallone, Whoopi Goldberg as Whoopi Goldberg, Eric Idle as Alan Smithee, Billy Bob Thornton as Billy Bob Thornton
|tt0113184|**Go Now** (1995)|[Romance, Drama, TV Movie]<br/><br/>Nick, is a young Scottish soccer player living in the big city. He meets Karen, and the two fall in love and move in together. Soon after, Nick exhibits signs of serious illness. As his body slowly succumbs to multiple sclerosis, he experiences a wide sweep of jagged emotions, and in the process gives himself and those who love him the strength to carry on.|
|tt0120800|**Quest for Camelot** (1998)|**An evil knight gives nobility a bad name.**<br/>[Fantasy, Animation, Drama, Romance, Family]<br/><br/>During the times of King Arthur, Kayley is a brave girl who dreams of following her late father as a Knight of the Round Table. The evil Ruber wants to invade Camelot and take the throne of King Arthur, and Kayley has to stop him.|Cary Elwes as Garrett (voice), Gary Oldman as Ruber (voice), Pierce Brosnan as King Arthur (voice), Gabriel Byrne as Lionel (voice), John Gielgud as Merlin (voice), Frank Welker as Ayden (voice), Eric Idle as Devon (voice)
|tt0119547|**Little Boy Blue** (1997)|[Drama, Thriller]<br/><br/>Living in rural Texas is a dysfunctional family: an abusive dad, a Vietnam vet with a war wound that's left him impotent; a compliant wife and a son of about 20, two small sons who look a lot like their brother. The dad harbors a secret, and he goes to murderous lengths to keep it hidden. The young man, Jimmy, who has suspicions, but little comes out until a Yankee woman comes to town.|Ryan Phillippe as Jimmy West, Nastassja Kinski as Kate West, John Savage as Ray West
|tt0120401|**The Ugly** (1997)|**Murder is in the eye of the beholder.**<br/>[Horror, Thriller]<br/><br/>A female psychologist wants to understand the minds of a confessed serial killer who spent the last five years in a mental hospital because of his state.|
|tt0120828|**Six Days Seven Nights** (1998)|**After this week in paradise, they’re going to need a vacation.**<br/>[Action, Adventure, Comedy, Romance]<br/><br/>When Quinn, a grouchy pilot living the good life in the South Pacific, agrees to transfer a savvy fashion editor, Robin, to Tahiti, he ends up stranded on a deserted island with her after their plane crashes. The pair avoid each other at first, until they're forced to team up to escape from the island -- and some pirates who want their heads.|Harrison Ford as Quinn Harris, Anne Heche as Robin Monroe, Allison Janney as Marjorie, Danny Trejo as Pierce
|tt0118998|**Doctor Dolittle** (1998)|**He doesn't just talk to the animals!**<br/>[Comedy, Family, Fantasy]<br/><br/>A successful physician and devoted family man, John Dolittle (Eddie Murphy) seems to have the world by the tail, until a long suppressed talent he possessed as a child, the ability to communicate with animals is suddenly reawakened with a vengeance! Now every creature within squawking distance wants the good doctor's advice, unleashing an outrageous chain of events that turns his world upside down!|Eddie Murphy as Dr. John Dolittle, Oliver Platt as Dr. Mark Weller, Peter Boyle as Calloway, Richard Schiff as Dr. Gene Reiss, Jeffrey Tambor as Dr. Fish, Beth Grant as Woman, Albert Brooks as Jacob "Jake" the Tiger, Paul Giamatti as Blaine Hammersmith
|tt0120591|**Armageddon** (1998)|**The Earth's Darkest Day Will Be Man's Finest Hour**<br/>[Action, Thriller, Science Fiction, Adventure]<br/><br/>When an asteroid threatens to collide with Earth, NASA honcho Dan Truman determines the only way to stop it is to drill into its surface and detonate a nuclear bomb. This leads him to renowned driller Harry Stamper, who agrees to helm the dangerous space mission provided he can bring along his own hotshot crew. Among them is the cocksure A.J. who Harry thinks isn't good enough for his daughter, until the mission proves otherwise.|Bruce Willis as Harry S. Stamper, Billy Bob Thornton as Dan Truman, NASA Administrator, Ben Affleck as A.J. Frost, Will Patton as Charles 'Chick' Chapple, Steve Buscemi as Rockhound, Chris Ellis as Flight Director Clark, Keith David as General Kimsey, Charlton Heston as Narrator, Udo Kier as Psychologist, Matt Malloy as NASA Techs
|tt0036872|**Going My Way** (1944)|**When the St. Louis Browns lost Bing, the Cardinal got a good singer!**<br/>[Drama, Comedy, Music]<br/><br/>Youthful Father Chuck O'Malley (Bing Crosby) led a colourful life of sports, song, and romance before joining the Roman Catholic clergy. After being appointed to a run-down New York parish , O'Malley's worldly knowledge helps him connect with a gang of boys looking for direction, eventually winning over the aging, conventional Parish priest (Barry Fitzgerald).|
|tt0041113|**All the King's Men** (1949)|**He Might Have Been A Pretty Good Guy … If Too Much Power … And Women … Hadn’t Gone To his Head!**<br/>[Drama]<br/><br/>All The King's Men is the story of the rise of politician Willie Stark from a rural county seat to the governor's mansion.|Sam Harris as Politician
|tt0086425|**Terms of Endearment** (1983)|[Drama, Comedy]<br/><br/>Aurora and Emma are mother and daughter who march to different drummers. Beginning with Emma's marriage, Aurora shows how difficult and loving she can be. The movie covers several years of their lives as each finds different reasons to go on living and find joy. Aurora's interludes with Garrett Breedlove, retired astronaut and next door neighbor are quite striking.|Shirley MacLaine as Aurora Greenway, Jack Nicholson as Garrett Breedlove, Danny DeVito as Vernon Dahlart, Jeff Daniels as Flap Horton, John Lithgow as Sam Burns
|tt0093629|**A Nightmare on Elm Street 3: Dream Warriors** (1987)|**If you think you're ready for Freddy, think again!**<br/>[Horror, Thriller]<br/><br/>It's been many years since Freddy Krueger's first victim, Nancy, came face-to-face with Freddy and his sadistic, evil ways. Now, Nancy's all grown up; she's put her frightening nightmares behind her and is helping teens cope with their dreams. Too bad Freddy's decided to herald his return by invading the kids' dreams and scaring them into committing suicide.|Robert Englund as Freddy Krueger, Patricia Arquette as Kristen Parker, Laurence Fishburne as Max
|tt0089173|**Friday the 13th: A New Beginning** (1985)|**A New Beginning to the first step in terror.**<br/>[Horror, Thriller]<br/><br/>Homicidal maniac Jason returns from the grave to cause more bloody mayhem. Young Tommy may have escaped from Crystal Lake, but he’s still haunted by the gruesome events that happened there. When gory murders start happening at the secluded halfway house for troubled teens where he now lives, it seems like his nightmarish nemesis, Jason, is back for more sadistic slaughters. But as things spiral out of control and the body count rises, Tommy begins to wonder if he’s become the killer he fears most.|Corey Feldman as Tommy Jarvis Age 12
|tt0095271|**Halloween 4: The Return of Michael Myers** (1988)|**Horror has returned to Haddonfield.**<br/>[Horror, Thriller]<br/><br/>The legend of that creepy masked-man, Michael Myers, comes to life once again in this fourth installment of the successful horror franchise. This time, it's Michael's niece, Jamie, who can't seem to escape her crazy uncle. With Michael on the loose, Jamie enlists the help of good old Dr. Loomis to stop the murderer. This time, though, there seems to be no end to Michael's madness.|Donald Pleasence as Dr. Sam Loomis
|tt0081383|**Prom Night** (1980)|**If you're not back by midnight... you won't be coming home!**<br/>[Horror, Mystery, Thriller]<br/><br/>This slasher movie follows a relentless killer who is out to avenge the death of a young girl who died after being bullied and teased by four of her classmates. Now high-school students, the guilt-ridden kids have kept their involvement a secret, but when they start being murdered, one by one, it's clear that someone knows the truth. Also coping with the past are members of the dead girl's family, most notably her prom-queen sister, Kim Hammond (Jamie Lee Curtis).|Leslie Nielsen as Mr. Hammond, Jamie Lee Curtis as Kim
|tt0093176|**Hello Mary Lou: Prom Night II** (1987)|**You Can't Keep a Bad Girl Down**<br/>[Horror]<br/><br/>When Hamilton High's Prom Queen of 1957, Mary Lou Maloney is killed by her jilted boyfriend, she comes back for revenge 30 years later. Bill Nordham is now the principle of Hamilton High and his son is about to attend the prom with Vicki Carpenter. However, she is possessed by Mary Lou Maloney after opening a trunk in the school's basement. Now Bill must face the horror he left behind in 1957.|Michael Ironside as Bill Nordham
|tt0103956|**Child's Play 3** (1991)|**Chucky has a new playmate.**<br/>[Thriller, Horror, Action]<br/><br/>It's been eight years since the events in the second film, we now see that Andy is a teenager who has been enrolled in a military school. Play Pals Toy Company decides to re-release its Good Guys line, feeling that after all this time, the bad publicity has died down. As they re-used old materials, the spirit of Charles Lee Ray once again comes to life.|Brad Dourif as Chucky
|tt0093409|**Lethal Weapon** (1987)|**If these two can learn to stand each other... the bad guys don't stand a chance.**<br/>[Adventure, Action, Comedy, Thriller, Crime]<br/><br/>Veteran buttoned-down LAPD detective Roger Murtaugh is partnered with unhinged cop Martin Riggs, who -- distraught after his wife's death -- has a death wish and takes unnecessary risks with criminals at every turn. The odd couple embark on their first homicide investigation as partners, involving a young woman known to Murtaugh with ties to a drug and prostitution ring.|Mel Gibson as Sergeant Martin Riggs, Danny Glover as Sergeant Roger Murtaugh, Gary Busey as Mr. Joshua, Mitchell Ryan as General Peter McAllister, Mary Ellen Trainor as Psychologist, Steve Kahan as Captain Ed Murphy
|tt0117898|**This World, Then the Fireworks** (1997)|**Meet Marty and Carol. Two people who are very good at being bad.**<br/>[Crime, Drama, Thriller]<br/><br/>Marty Lakewood is a reporter forced to leave Chicago and his family because he had uncovered too much police corruption. He returns to his small home town on the California coast to his ailing mother and prostitute sister, with whom he had an incestuous affair. Being short of money, he seduces a woman cop in order to sell her house.|Billy Zane as Marty Lakewood, Richard Edson as Joe, Gina Gershon as Carol Lakewood Morton, Seymour Cassel as Detective Harris, Will Patton as Lt. Morgan
|tt0102757|**The Rapture** (1991)|**"It goes places movies seldom dare to go."**<br/>[Mystery, Drama]<br/><br/>A lonely telephone operator leading an empty, amoral life finds God — only to have her faith continually tested in ways beyond what she could ever have imagined.|Mimi Rogers as Sharon, Will Patton as Deputy Foster, James Le Gros as Tommy
|tt0134619|**Disturbing Behavior** (1998)|**In Cradle Bay it doesn’t matter if you’re not perfect.  You will be.**<br/>[Mystery, Horror, Science Fiction]<br/><br/>Steve Clark (James Marsden) is a newcomer in the town of Cradle Bay, and he quickly realizes that there's something odd about his high school classmates. The clique known as the "Blue Ribbons" are the eerie embodiment of academic excellence and clean living. But, like the rest of the town, they're a little too perfect. When Steve's rebellious friend Gavin (Nick Stahl) mysteriously joins their ranks, Steve searches for the truth with fellow misfit Rachel (Katie Holmes).|
|tt0137386|**Billy's Hollywood Screen Kiss** (1998)|**Lights!... Camera!... Attraction!**<br/>[Comedy, Romance]<br/><br/>Billy, a struggling young gay photographer (who likes Polaroids), tired of being the "other man", falls in love with Gabriel, a waiter and aspiring musician who is probably straight but possibly gay or at least curious. Billy tries to get Gabriel to model for his latest project, a series of remakes of famous Hollywood screen kisses, featuring male couples, while also trying to win his affections.|
|tt0062737|**Blackbeard's Ghost** (1968)|**He's out'a sight...literally!**<br/>[Fantasy, Comedy, Family]<br/><br/>Peter Ustinov stars as the eponymous wraith, who returns to Earth to aid his descendant, elderly Elsa Lanchester (Stowecroft). The villains want to kick Lanchester and her friends out of their group home so that they can build a crooked casino. Good guy Steve Walker (Jones) gets caught in the middle of the squabble after evoking Blackbeard's ghost.|
|tt0075807|**Candleshoe** (1977)|[Family]<br/><br/>Candleshoe is a 1977 Walt Disney Productions live action movie based on the Michael Innes novel Christmas at Candleshoe and starring Jodie Foster, Helen Hayes in her last screen appearance, David Niven and Leo McKern.|Jodie Foster as Casey
|tt0065566|**The Computer Wore Tennis Shoes** (1969)|[Comedy, Family]<br/><br/>Some college students manage to persuade the town's big businessman, A. J. Arno, to donate a computer to their college. When the problem- student, Dexter Riley, tries to fix the computer, he gets an electric shock and his brain turns to a computer; now he remembers everything he reads. Unfortunately, he also remembers information which was in the computer's memory, like Arno's illegal businesses..|Kurt Russell as Dexter Riley, Frank Welker as Henry
|tt0089853|**The Purple Rose of Cairo** (1985)|[Fantasy, Comedy, Romance]<br/><br/>Cecilia is a waitress in New Jersey, living a dreary life during the Great Depression. Her only escape from her mudane reality is the movie theatre. After losing her job, Cecilia goes to see 'The Purple Rose of Cairo' in hopes of raising her spirits, where she watches dashing archaeologist Tom Baxter time and again.|Mia Farrow as Cecilia, Jeff Daniels as Tom Baxter / Gil Shepherd, Danny Aiello as Monk, John Wood as Jason, Dianne Wiest as Emma, Glenne Headly as Hooker
|tt0046183|**Peter Pan** (1953)|**It will live in your heart forever!**<br/>[Animation, Music, Family, Adventure, Fantasy]<br/><br/>Leaving the safety of their nursery behind, Wendy, Michael and John follow Peter Pan to a magical world where childhood lasts forever. But while in Neverland, the kids must face Captain Hook and foil his attempts to get rid of Peter for good.|
|tt0120694|**Halloween: H20** (1998)|**Blood is thicker than Water.**<br/>[Horror, Thriller]<br/><br/>Two decades after surviving a massacre on October 31, 1978, former baby sitter Laurie Strode finds herself hunted by persistent knife-wielder Michael Myers. Laurie now lives in Northern California under an assumed name, where she works as the headmistress of a private school. But it's not far enough to escape Myers, who soon discovers her whereabouts. As Halloween descends upon Laurie's peaceful community, a feeling of dread weighs upon her -- with good reason.|Jamie Lee Curtis as Laurie Strode / Keri Tate
|tt0079367|**The Jerk** (1979)|**A rags to riches to rags story.**<br/>[Comedy]<br/><br/>After discovering he's not really black like the rest of his family, likable dimwit Navin Johnson runs off on a hilarious misadventure in this comedy classic that takes him from rags to riches and back to rags again. The slaphappy jerk strikes it rich, but life in the fast lane isn't all it's cracked up to be and, in the end, all that really matters to Johnson is his true love.|Steve Martin as Navin R. Johnson, M. Emmet Walsh as Madman
|tt0090305|**Weird Science** (1985)|**If you can't get a date, make one!**<br/>[Comedy, Romance, Fantasy]<br/><br/>Two unpopular teenagers, Gary and Wyatt, fail at all attempts to be accepted by their peers. Their desperation to be liked leads them to "create" a woman via their computer. Their living and breathing creation is a gorgeous woman, Lisa, whose purpose is to boost their confidence level by putting them into situations which require Gary and Wyatt to act like men.|Bill Paxton as Chet Donnelly, Robert Downey Jr. as Ian
|tt0061584|**Doctor Dolittle** (1967)|**Ride across the sea inside the GIANT PINK SEA SNAIL!**<br/>[Adventure, Fantasy, Comedy, Family, Music, Romance]<br/><br/>Get ready for the wildest adventure of a lifetime in the most ambitious production ever brought to film. Earning a 1967 Academy Award nomination for Best Picture, this dazzling fantasy turns both ordinary and exotic animals into talking, dancing and singing sensations! Rex Harrison is unforgettable in this inspiring adaptation of Hugh Lofting's classic stories.Step into the English country home of the good doctor as he performs remarkable treatments on the wildest variety of patients you could imagine. Discover his secret cures and watch with wide-eyed excitement as he and his four-legged, fine-feathered friends charm their way into your heart!|
|tt0057372|**The Nutty Professor** (1963)|**What does he become? What kind of monster?**<br/>[Comedy, Family, Romance, Science Fiction]<br/><br/>Jerry Lewis directed, co-wrote and starred in this riotously funny movie that set a new standard for screen comedy and inspired the hit remake. Lewis plays a timid, nearsighted chemistry teacher who discovers a magical potion that can transform him into a suave and handsome Romeo. The Jekyll and Hyde game works well enough until the concoction starts to wear off at the most embarrassing times.|Kathleen Freeman as Millie Lemmon
|tt0070016|**Charlotte's Web** (1973)|**That humble radiant terrific book is now a humble radiant terrific movie.**<br/>[Animation, Drama, Family]<br/><br/>Wilbur the pig is scared of the end of the season, because he knows that come that time, he will end up on the dinner table. He hatches a plan with Charlotte, a spider that lives in his pen, to ensure that this will never happen.|
|tt0091790|**Pretty in Pink** (1986)|**Blane's a pretty cool guy. Andie's pretty in pink. And Ducky's pretty crazy.**<br/>[Comedy]<br/><br/>Andie is an outcast, hanging out either with her older boss, who owns the record store where she works, or her quirky high school classmate Duckie, who has a crush on her. When one of the rich and popular kids at school, Blane, asks Andie out, it seems too good to be true. As Andie starts falling for Blane, she begins to realize that dating someone from a different social sphere is not easy.|Andrew McCarthy as Blane McDonough, Harry Dean Stanton as Jack Walsh, James Spader as Steff, Gina Gershon as Trombley, Girl Friend/Gym Class
|tt0120703|**How Stella Got Her Groove Back** (1998)|**Sometimes you have to break the rules to free your heart.**<br/>[Comedy, Drama, Romance]<br/><br/>Through good times and bad, Stella and Delilah have always had each other. Now, Stella's so busy building a life that she's forgotten how to really live. But Delilah is about to change all that. What starts as a quick trip to Jamaica, end as an exhilarating voyage of self discovery as Stella learns to open her heart and find love - even if it's with a man 20 years her junior.|Angela Bassett as Stella Payne, Whoopi Goldberg as Delilah Abraham, Danny Glover as (scenes deleted)
|tt0099763|**Henry: Portrait of a Serial Killer** (1986)|**The shocking true story of Henry Lee Lucas.**<br/>[Crime, Drama, Horror, Thriller]<br/><br/>Henry likes to kill people, in different ways each time. Henry shares an apartment with Otis. When Otis' sister comes to stay, we see both sides of Henry: "the guy next door" and the serial killer.|Michael Rooker as Henry
|tt0120611|**Blade** (1998)|**Part Man. Part Vampire. All Hero.**<br/>[Horror, Action]<br/><br/>When Blade's mother was bitten by a vampire during pregnancy, she did not know that she gave her son a special gift while dying: All the good vampire attributes in combination with the best human skills. Blade and his mentor Whistler battle an evil vampire rebel (Deacon Frost) who plans to take over the outdated vampire council, capture Blade and resurrect voracious blood god La Magra.|Wesley Snipes as Blade, Kris Kristofferson as Whistler, Donal Logue as Quinn, Udo Kier as Dragonetti
|tt0120576|**Dance with Me** (1998)|**In the dance of life, love can happen in a heartbeat.**<br/>[Drama, Music, Romance]<br/><br/>Young Cuban Rafael just buried his mother, and comes to Houston to meet his father John for the first time. The difficult part is that John doesn't know he is Rafael's father. John runs a dance studio, and everyone prepares for the World Open Dance championship in Las Vegas. It soon becomes clear Rafael is a very good dancer, and Ruby is the biggest hope for the studio at the championship.|Kris Kristofferson as John Burnett, Joan Plowright as Bea Johnson, Beth Grant as Lovejoy
|tt0120901|**Wrongfully Accused** (1998)|**It's not just a movie. It's every movie.**<br/>[Action, Adventure, Comedy]<br/><br/>Ryan Harrison, a violin god, superstar and sex symbol does not want to cheat on sexy Lauren Goodhue's husband with her. Mr. Goodhue is found murdered and Ryan suddenly finds himself being the main suspect. After being sentenced to death he manages to flee while being transferred to his execution site. Now, all the world is after him as he stumbles from one unfortunate incident to the next in order to prove himself innocent - by finding a mysterious one-eyed, one-armed, one-legged man...|Leslie Nielsen as Ryan Harrison
|tt0119346|**I Married a Strange Person!** (1998)|[Drama, Comedy, Animation]<br/><br/>A newlywed develops a strange lump on his neck that gives him the ability to transform people or objects at will. His wife is very upset. Meanwhile, the CEO of Smilecorp learns of this man and his ability and sees a way to achieve world domination if only the man can be taken alive. Animated movie by Bill Plympton.|
|tt0120654|**Dirty Work** (1998)|**Revenge is a dish best served DIRTY.**<br/>[Comedy]<br/><br/>Unemployed and recently dumped, Mitch and his buddy Sam start a revenge-for-hire business to raise the $50,000 that Sam's father needs to get a heart transplant. The dirty duo brings down a movie theater manager and hires hookers to pose as dead bodies during a live TV ad. When a wealthy developer hires the guys to trash a building (so that he can have it condemned), problems arise and a feud ensues.|Jack Warden as 'Pops' McKenna, Christopher McDonald as Travis Cole, Chevy Chase as Dr. Farthing, John Goodman as Mayor Adrian Riggins, Adam Sandler as Satan
|tt0035279|**Saboteur** (1942)|**You'd like to say - IT CAN'T HAPPEN HERE!... but every jolting scene is TRUE!!**<br/>[Thriller, War]<br/><br/>Aircraft factory worker Barry Kane goes on the run across the United States when he is wrongly accused of starting a fire that killed his best friend.|
|tt0021015|**Juno and the Paycock** (1930)|[Drama]<br/><br/>During the Irish revolution, a family earns a big inheritance. They start leading a rich life forgetting what the most important values of are. At the end, they discover they will not receive that inheritance; the family is destroyed and penniless. They must sell their home and start living like vagabonds.|
|tt0085346|**Class** (1983)|**The good news is, Jonathan's having his first affair. The bad news is, she's his roommate's mother!**<br/>[Comedy, Drama]<br/><br/>Naive Midwestern prep student Jonathan bonds with his more worldly roommate, Skip, who takes the small-town boy under his wing. At Skip's urging, the inexperienced Jonathan is emboldened to seek out older women in the cocktail lounges of nearby Chicago, where he meets and beds the alluring Ellen, who unfortunately turns out to be Skip's mother. The division between the friends is further deepened when a cheating scandal engulfs the school.|Andrew McCarthy as Jonathan Ogner, John Cusack as Roscoe Maibaum
|tt0087359|**Grandview, U.S.A.** (1984)|**Where dreams have a funny way of coming true.**<br/>[Drama]<br/><br/>Life in the small town of Grandview, Illinois is one that is just like any other city or town. Tim Pearson, soon to be graduating high school wants to go to Florida to study oceanography. He meets Michelle "Mike" Cody and is attracted to her. She runs the local Demolition Derby place. Ernie "Slam" Webster is on of the drivers in the derby who's wife is cheating on him and wants to later on be with Mike. Tim falls for Mike and a big love triangle is about to happen.|Jamie Lee Curtis as Michelle 'Mike' Cody, Patrick Swayze as Ernie 'Slam' Webster, Jennifer Jason Leigh as Candy Webster, M. Emmet Walsh as Mr. Clark, John Cusack as Johnny Maine, Joan Cusack as Mary Maine
|tt0096463|**Working Girl** (1988)|**For anyone who's ever won. For anyone who's ever lost. And for everyone who's still in there trying.**<br/>[Comedy, Drama, Romance]<br/><br/>Witty, romantic look at life in the corporate jungle. Tess McGill is an ambitious secretary with a unique approach for climbing the ladder to success. When her classy, but villainous boss breaks a leg skiing, Tess takes over her office, her apartment and even her wardrobe. She creates a deal with a handsome investment banker that will either take her to the top, or finish her off for good.|Harrison Ford as Jack Trainer, Melanie Griffith as Tess McGill, Sigourney Weaver as Katharine Parker, Alec Baldwin as Mick Dugan, Joan Cusack as Cyn, Philip Bosco as Oren Trask, Oliver Platt as Lutz, Kevin Spacey as Bob Speck, Timothy Carhart as Tim Draper, Zach Grenier as Jim, Caroline Aaron as Petty Marsh Secretary
|tt0095593|**Married to the Mob** (1988)|**They're her family... Whether she likes it or not.**<br/>[Comedy, Crime, Romance]<br/><br/>Angela de Marco is fed up with her gangster husband's line of work and wants no part of the crime world. When her husband is killed for having an affair with the mistress of mob boss Tony "The Tiger" Russo, Angela and her son depart for New York City to make a fresh start. Unfortunately, Tony has set his sights upon Angela -- and so has an undercover FBI agent looking to use her to bust Tony.|Michelle Pfeiffer as Angela de Marco, Matthew Modine as Mike Downey, Oliver Platt as Ed Benitez, Alec Baldwin as Frank de Marco, Dean Stockwell as Tony Russo, Joan Cusack as Rose, Joe Spinell as Leonard 'Tiptoes' Mazzilli, Charles Napier as Angela's Hairdresser, Tracey Walter as Mr. Chicken Lickin'
|tt0084938|**Young Doctors in Love** (1982)|**Warning: This Movie May Be Dangerous To Your Health. You May Never Stop Laughing.**<br/>[Comedy]<br/><br/>An "Airplane!"-style spoof of hospital soap operas: a brilliant young trainee can't stand the sight of blood; a doctor romances the head nurse in order to get the key to the drugs cabinet; there's a mafioso on the loose disguised as a woman - in other words all the usual ingredients present and correct, though in this case the laughs are intentional.|Michael McKean as Dr. Simon August, Sean Young as Dr. Stephanie Brody, Héctor Elizondo as Angelo / Angela Bonafetti, Harry Dean Stanton as Dr. Oliver Ludwig, Taylor Negron as Dr. Phil Burns, Demi Moore as New intern, Ed Begley Jr. as The Flashback - Young Simon's Father
|tt0084472|**Parasite** (1982)|**You will not feel the terror until you experience the movie!**<br/>[Horror, Science Fiction]<br/><br/>Paul Dean has created a deadly parasite that is now attached to his stomach. He and his female companion, Patricia Welles, must find a way to destroy it while also trying to avoid Ricus, his rednecks, and an evil government agent named Merchant.|Demi Moore as Patricia Welles
|tt0104257|**A Few Good Men** (1992)|[Drama]<br/><br/>When cocky military lawyer Lt. Daniel Kaffee and his co-counsel, Lt. Cmdr. JoAnne Galloway, are assigned to a murder case, they uncover a hazing ritual that could implicate high-ranking officials such as shady Col. Nathan Jessep.|Tom Cruise as Lt. Daniel Kaffee, Jack Nicholson as Col. Nathan R. Jessep, Demi Moore as Lt. Cdr. JoAnne Galloway, Kevin Bacon as Capt. Jack Ross, Kevin Pollak as Lt. Sam Weinberg, J.T. Walsh as Lt. Col. Matthew A. Markinson, Kiefer Sutherland as Lt. Jonathan Kendrick, Xander Berkeley as Capt. Whitaker, Cuba Gooding Jr. as Cpl. Carl Hammaker
|tt0120162|**Somewhere in the City** (1998)|[Comedy, Drama]<br/><br/>The film portrays life of New York City's Lower East Side inhabitants. Introvert therapist Betty is befriended with Chinese immigrant Lu Lu, leftist radical Che is also Lu Lu's friend. Italian immigrant and wife of supervisor of building where they all live, Marta, is having an affair with her neighbour Frankie who likes scams. One of his scams goes terribly wrong.|Robert John Burke as Frankie
|tt0146336|**Urban Legend** (1998)|**It Happened To Someone Who Knows Someone You Know... You're Next.**<br/>[Horror, Thriller]<br/><br/>There's a campus killer on the loose who's making urban legends, like the one about eating pop rocks and soda at the same time will make your stomach explode and the one about a psycho with an axe stepping into the backseat of your car at the gas station when not looking, into reality.|Robert Englund as Professor William Wexler
|tt0126604|**Pecker** (1998)|**He never realized how far 35 millimeters would take him.**<br/>[Comedy]<br/><br/>A Baltimore sandwich shop employee becomes an overnight sensation when photographs he's taken of his weird family become the latest rage in the art world. The young man is called "Pecker" because he pecks at his food like a bird.|Christina Ricci as Shelley, Martha Plimpton as Tina, Lili Taylor as Rorey Wheeler
|tt0080731|**The Fiendish Plot of Dr. Fu Manchu** (1980)|**BEWARE! See this movie - An hour later you'll want to see it again!**<br/>[Comedy, Horror]<br/><br/>Fu Manchu's 168th birthday celebration is dampened when a hapless flunky spills Fu's age-regressing elixir vitae. Fu sends his lackeys to round up ingredients for a new batch of elixir, starting with the Star of Leningrad diamond, nabbed from a Soviet exhibition in Washington. The FBI sends agents Capone and Williams to England to confer with Nayland Smith, an expert on Fu.|Helen Mirren as Alice Rage
|tt0120889|**What Dreams May Come** (1998)|**After life there is more. The end is just the beginning.**<br/>[Drama, Fantasy, Romance]<br/><br/>Chris Neilson dies to find himself in a heaven more amazing than he could have ever dreamed of. There is one thing missing: his wife. After he dies, his wife, Annie killed herself and went to hell. Chris decides to risk eternity in hades for the small chance that he will be able to bring her back to heaven.|Robin Williams as Chris Nielsen, Cuba Gooding Jr. as Albert Lewis, Annabella Sciorra as Annie Collins-Nielsen, Max von Sydow as The Tracker, Lucinda Jenney as Mrs. Jacobs
|tt0073440|**Nashville** (1975)|**Wild. Wonderful. Sinful. Laughing. Explosive.**<br/>[Drama, Music]<br/><br/>This movie tells the intersecting stories of various people connected to the music business in Nashville. Barbara Jean is the reigning queen of Nashville but is near collapse. Linnea and Delbert Reese have a shaky marriage and 2 deaf children. Opal is a British journalist touring the area. These and other stories come together in a dramatic climax.|Ned Beatty as Delbert Reese, Shelley Duvall as Marthe aka "L. A. Joan", Scott Glenn as Pfc. Glenn Kelly, Jeff Goldblum as Tricycle Man, Lily Tomlin as Linnea Reese, Keenan Wynn as Mr. Green, Elliott Gould as Himself
|tt0119577|**Love Is the Devil: Study for a Portrait of Francis Bacon** (1998)|[TV Movie, Drama]<br/><br/>Biography of the British painter Francis Bacon. The movie focuses on his relationship with George Dyer, his lover. Dyer was a former small time crook.|
|tt0090830|**Children of a Lesser God** (1986)|**Love has a language all of its own.**<br/>[Drama, Romance]<br/><br/>James is a new speech teacher at a school for the deaf. He falls for Sarah, a pupil who decided to stay on at the school rather than venture into the big bad world. She shuns him at first, refusing to read his lips and only using signs. Will her feelings change over time?|William Hurt as James Leeds, Philip Bosco as Dr. Curtis Franklin
|tt0124819|**Orgazmo** (1997)|**Fighting crime never felt so good!**<br/>[Comedy]<br/><br/>Joe Young is a devout Mormon living in L.A. trying to raise enough money to go back to Utah and marry his girlfriend, Lisa. Joe is spreading the word about the church of Latter Day Saints one day when he's confronted by two burly bodyguards. A scuffle breaks out, and Joe's martial arts skills impress Maxxx Orbison, who directs pornographic movies.|
|tt0100740|**Tales from the Darkside: The Movie** (1990)|**From the depths of four twisted minds.**<br/>[Thriller, Comedy, Fantasy, Horror]<br/><br/>The first segment features an animated mummy stalking selected student victims; the second tale tells the story of a "cat from hell" who cannot be killed and leaves a trail of victims behind it; the third story is about a man who witnesses a bizarre killing and promises never to tell what he saw and the "in-between" bit is the story of a woman preparing to cook her newspaper boy for supper.|Christian Slater as Andy (segment "Lot 249"), Steve Buscemi as Bellingham (segment "Lot 249"), Julianne Moore as Susan (segment "Lot 249"), William Hickey as Drogan (segment "Cat From Hell"), Alice Drummond as Carolyn (segment "Cat From Hell"), James Remar as Preston (segment "Lover's Vow")
|tt0073747|**The Stepford Wives** (1975)|**Something strange is happening in the town of Stepford.**<br/>[Horror, Mystery, Science Fiction]<br/><br/>The urban aspirant photographer Joanna Eberhart moves from Manhattan to Stepford, Connecticut with her family. Her husband Walter Eberhart decided to live in a calm suburb, but Joanna did not like the neighborhood with beautiful and perfect housewives. She becomes friend of Bobbie Markowe and Charmaine Wimperis, and when they change Joanna tries to escape.|Mary Stuart Masterson as Kim Eberhart
|tt0134067|**The Rugrats Movie** (1998)|[Animation, Family]<br/><br/>Tommy faces responsibility when Dil, his new baby brother, is born. As with all newborns, the child becomes a bane to Tommy and the rest of his gang. They decide to return Dil to where he came from, the hospital, but they get lost along the way. Can they find their way home and can Tommy and Dil learn to get along?|Whoopi Goldberg as Ranger Margaret (voice), David Spade as Ranger Frank (voice), Tim Curry as Rex Pester (voice)
|tt0087344|**Godzilla 1985** (1984)|**Your favorite fire breathing monster... like you've never seen him before!**<br/>[Action, Drama, Horror, Science Fiction, Thriller]<br/><br/>Originally released in Japan as "The Return of Godzilla"  in 1984, this is the heavily re-edited, re-titled "Godzilla 1985".  Adding in new footage of Raymond Burr, this 16th Godzilla film ignores all previous sequels and serves as a direct follow-up to the 1956 "Godzilla King of the Monsters", which also featured scenes with Burr edited into 1954's "Godzilla". This film restores the darker tone of the original, as we witness the nuclear destruction of giant lizard terrorizing Japan.|
|tt0091541|**The Money Pit** (1986)|**For everyone who's ever been deeply in Love or deeply in debt.**<br/>[Comedy, Romance]<br/><br/>After being evicted from their Manhattan apartment, a couple buys what looks like the home of their dreams – only to find themselves saddled with a bank-account-draining nightmare. Struggling to keep their relationship together as their rambling mansion falls to pieces around them, the two watch in hilarious horror as everything – including the kitchen sink – disappears into the 'Money Pit'.|Tom Hanks as Walter Fielding, Jr., Joe Mantegna as Art Shirk, Philip Bosco as Curly, Josh Mostel as Jack Schnittman, Frankie Faison as , Michael Jeter as 
|tt0124198|**Very Bad Things** (1998)|**They've been bad. Very bad.**<br/>[Comedy, Crime, Thriller]<br/><br/>Kyle Fisher has one last night to celebrate life as a single man before marrying Laura, so he sets out to Vegas with four of his best buddies. But a drug and alcohol filled night on the town with a stripper who goes all the way, turns into a cold night in the desert with shovels when the stripper goes all the way into a body bag after dying in their bathroom. And that's just the first of the bodies to pile up before Kyle can walk down the aisle...|Cameron Diaz as Laura Garrety, Christian Slater as Robert Boyd, Jeremy Piven as Michael Berkow
|tt0120324|**A Simple Plan** (1998)|**Sometimes good people do evil things.**<br/>[Drama, Crime, Thriller]<br/><br/>Captivated by the lure of sudden wealth, the quiet rural lives of two brothers erupt into conflicts of greed, paranoia and distrust when over $4 million in cash is discovered at the remote site of a downed small airplane. Their simple plan to retain the money while avoiding detection opens a Pandora's box when the fear of getting caught triggers panicked behavior and leads to virulent consequences|Bill Paxton as Hank Mitchell, Bridget Fonda as Sarah Mitchell, Billy Bob Thornton as Jacob Mitchell
|tt0089961|**Santa Claus: The Movie** (1985)|**Guess Who's Coming To Town!**<br/>[Family, Fantasy, Adventure, Science Fiction, Comedy]<br/><br/>The first half of this film, set hundreds of years ago, shows how the old man who eventually became Santa Claus was given immortality and chosen to deliver toys to all the children of the world. The second half moves into the modern era, in which Patch, the head elf, strikes out on his own and falls in with an evil toy manufacturer who wants to corner the market and eliminate Santa Claus.|John Lithgow as B.Z.
|tt0089370|**The Jewel of the Nile** (1985)|**When the going gets tough, the tough get going.**<br/>[Action, Adventure, Comedy, Romance]<br/><br/>Joan Wilder is thrust back into a world of murder, chases, foreign intrigue... and love. This time out she's duped by a duplicitous Arab dignitary who brings her to the Middle East, ostensibly to write a book about his life. Of course he's up to no good, and Joan is just another pawn in his wicked game. But Jack Colton and his sidekick Ralph show up to help our intrepid heroine save the day.|Michael Douglas as Jack Colton, Kathleen Turner as Joan Wilder, Danny DeVito as Ralph, Holland Taylor as Gloria
|tt0091653|**Nothing in Common** (1986)|**It's a comedy. And a drama. Just like life.**<br/>[Drama, Comedy, Romance]<br/><br/>On his way up the corporate ladder, David Basner confronts his greatest challenge: his father.|Tom Hanks as David Basner, Héctor Elizondo as Charlie Gargas
|tt0097958|**National Lampoon's Christmas Vacation** (1989)|**Yule crack up.**<br/>[Comedy]<br/><br/>It's Christmas time and the Griswolds are preparing for a family seasonal celebration, but things never run smoothly for Clark, his wife Ellen and their two kids. Clark's continual bad luck is worsened by his obnoxious family guests, but he manages to keep going knowing that his Christmas bonus is due soon.|Chevy Chase as Clark Griswold, Beverly D'Angelo as Ellen Smith Griswold, Randy Quaid as Cousin Eddie, Juliette Lewis as Audry Griswold, William Hickey as Uncle Lewis, Brian Doyle-Murray as Mr. Frank Shirley
|tt0129290|**Patch Adams** (1998)|**Laughter is contagious**<br/>[Comedy, Drama]<br/><br/>Meet Patch Adams, a doctor who doesn't look, act or think like any doctor you've met before. For Patch, humor is the best medicine, and he's willing to do just anything to make his patients laugh - even if it means risking his own career.|Robin Williams as Hunter 'Patch' Adams, Philip Seymour Hoffman as Mitch Roman, Bob Gunton as Dean Walcott, Peter Coyote as Bill Davis, Michael Jeter as Rudy
|tt0120699|**The Hi-Lo Country** (1998)|**A woman like Mona can drive men to extremes.**<br/>[Drama, Action, Western, Romance]<br/><br/>An intimate story of the enduring bond of friendship between two hard-living men, set against a sweeping backdrop: the American West, post-World War II, in its twilight. Pete and Big Boy are masters of the prairie, but ultimately face trickier terrain: the human heart.|Woody Harrelson as Big Boy Matson, Patricia Arquette as Mona, John Diehl as Les Birk
|tt0093072|**The Garbage Pail Kids Movie** (1987)|**It doesn't get any grosser than this!**<br/>[Adventure, Science Fiction, Comedy]<br/><br/>Seven disgusting kids but nevertheless of interesting personality are being made of the green mud coming out of garbage can. Once alive their master gives them rules to obey although they think that life is funnier without following stupid regulations like no television or no candy. Naturally this will cause some conflicts.|
|tt0093075|**The Gate** (1987)|**Pray it's not too late.**<br/>[Horror, Fantasy]<br/><br/>Three young children accidentally release a horde of nasty, pint-sized demons from a hole in a suburban backyard. What follows is a classic battle between good and evil as the three kids struggle to overcome a nightmarish hell that is literally taking over the Earth.|
|tt0097368|**The Fly II** (1989)|**Like Father Like Son**<br/>[Horror, Science Fiction, Thriller]<br/><br/>Martin Brundle, born of the human/fly, is adopted by his father's place of employment (Bartok Inc.) while the employees simply wait for his mutant chromosomes to come out of their dormant state.|Eric Stoltz as Martin Brundle
|tt0090917|**Deadly Friend** (1986)|**There's no one alive who'll play with the girl next door.**<br/>[Horror, Science Fiction]<br/><br/>Paul is a new kid in town with a robot named "BB". He befriends Samantha and the three of them have a lot of good times together. That is, until Samantha's abusive father throws her down some stairs and kills her. In an effort to save her life, Paul implants BB's computer brain into Samantha's human brain.|Charles Fleischer as BB
|tt0091187|**Heartbreak Ridge** (1986)|**...the scars run deep.**<br/>[Action, Comedy, Drama, War]<br/><br/>A hard-nosed, hard-living Marine gunnery sergeant clashes with his superiors and his ex-wife as he takes command of a spoiled recon platoon with a bad attitude.|Clint Eastwood as Gunnery Sgt. Tom 'Gunny' Highway
|tt0054167|**Peeping Tom** (1960)|**What made this the most diabolical murder weapon ever used?**<br/>[Horror, Thriller]<br/><br/>Loner Mark Lewis works at a film studio during the day and, at night, takes racy photographs of women. Also he's making a documentary on fear, which involves recording the reactions of victims as he murders them. He befriends Helen, the daughter of the family living in the apartment below his, and he tells her vaguely about the movie he is making. She sneaks into Mark's apartment to watch it and is horrified by what she sees -- especially when Mark catches her.|
|tt0120784|**Payback** (1999)|**No More Mr. Nice Guy.**<br/>[Drama, Action, Thriller, Crime]<br/><br/>With friends like these, who needs enemies? That's the question bad guy Porter is left asking after his wife and partner steal his heist money and leave him for dead -- or so they think. Five months and an endless reservoir of bitterness later, Porter's partners and the crooked cops on his tail learn how bad payback can be.|Mel Gibson as Porter, Kris Kristofferson as Bronson, David Paymer as Arthur Stegman, John Glover as Phil, James Coburn as Justin Fairfax
|tt0138987|**20 Dates** (1998)|**A comedy so real you'll think it's fiction.**<br/>[Romance, Comedy]<br/><br/>Myles is divorced in L.A. He wants a love life and a film career. So he decides to go on 20 dates and find true love in front of a camera, making his first feature. His patient agent, Richard, finds a $60,000 investor, the shadowy Elie. Myles starts his search, sometimes telling his date she's being filmed, sometimes not. Elie wants sex and titillation, Myles wants it "real." Myles regularly talks with his old film teacher, Robert McKee, who wonders if love is possible in modern life. Half-way through the 20 dates, Myles meets Elisabeth; she's everything he desires and she likes him. Can he finish the 20 dates, satisfy Elie, and complete his film without losing Elisabeth?|
|tt0124298|**Blast from the Past** (1999)|**She'd never met anyone like him. He's never met anyone... Period.**<br/>[Comedy, Romance]<br/><br/>Following a bomb scare in the 1960s that locked the Webers into their bomb shelter for 35 years, Adam now ventures forth into Los Angeles to obtain food and supplies for his family, and a non-mutant wife for himself.|Brendan Fraser as Adam Webber, Christopher Walken as Calvin Webber, Sissy Spacek as Helen Thomas Webber
|tt0070334|**The Long Goodbye** (1973)|**Nothing says goodbye like a bullet...**<br/>[Thriller, Crime, Drama, Mystery]<br/><br/>Detective Philip Marlowe tries to help a friend who is accused of murdering his wife.|Elliott Gould as Philip Marlowe, Arnold Schwarzenegger as Hood in Augustine's office
|tt0085333|**Christine** (1983)|**How do you kill something that can't possibly be alive?**<br/>[Horror]<br/><br/>Geeky student Arnie Cunningham falls for Christine, a rusty 1958 Plymouth Fury, and becomes obsessed with restoring the classic automobile to her former glory. As the car changes, so does Arnie, whose newfound confidence turns to arrogance behind the wheel of his exotic beauty. Arnie's girlfriend Leigh and best friend Dennis reach out to him, only to be met by a Fury like no other.|Robert Prosky as Will Darnell, Harry Dean Stanton as Rudolph Junkins, Kelly Preston as Roseanne
|tt0071110|**Airport 1975** (1974)|**Something hit us... The crew is dead... Help us, please, please help us!**<br/>[Action, Adventure, Drama, Thriller]<br/><br/>"Something hit us...the crew is dead...help us, please, please help us!" With these terrifying words, 22 of Hollywood's greatest stars find themselves aboard a pilotless jumbo jet headed on a collision course with destruction in the nerve chilling sequel to the greatest disaster movie ever made.|Charlton Heston as Alan Murdock, George Kennedy as Joe Patroni
|tt0065462|**Beneath the Planet of the Apes** (1970)|**An army of civilized apes...A fortress of radiation-crazed super humans...Earth's final battle is about to begin - Beneath the atomic rubble of what was once the city of New York!**<br/>[Adventure, Science Fiction, Mystery]<br/><br/>Astronaut Brent is sent to rescue Taylor but crash lands on the Planet of the Apes, just like Taylor did in the original film. Taylor has disappeared into the Forbidden Zone so Brent and Nova try to follow and find him. He discovers a cult of humans that fear the Apes' latest military movements and finds himself in the middle. Tension mounts to a climactic battle between ape and man deep in the bowels of the planet.|Charlton Heston as Taylor
|tt0157208|**The School of Flesh** (1998)|**Lessons in love, sex and betrayal.**<br/>[Romance, Drama, Mystery]<br/><br/>In Paris, Dominique, a middle-aged fashion professional, solicits the services of the handsome Quentin , a bisexual bartender and prostitute who is 15 years her junior. After they first sleep together, their business transaction becomes transformed into a passionate love affair. However, the couple's romance becomes an ugly power struggle when social class and age distinctions begin to bubble toward the surface.|
|tt0120646|**The Deep End of the Ocean** (1999)|**The search for her son was over. The search for her family was just beginning.**<br/>[Drama, Mystery]<br/><br/>Michelle Pfeiffer is ferocious in the role of a desperate mother whose 3-year-old son disappears during her high school reunion. Nine years later, by chance, he turns up in the town in which the family has just relocated. Based on Jacquelyn Mitchard's best-selling novel (an Oprah book club selection), the movie effectively presents the troubling dynamics that exist between family members who've suffered such an unsettling loss.|Michelle Pfeiffer as Beth Cappadora, Whoopi Goldberg as Candy Bliss, Lucinda Jenney as Laurie
|tt0144814|**The Rage: Carrie 2** (1999)|**Looks Can Kill**<br/>[Horror, Thriller, Science Fiction]<br/><br/>After the suicide of her only friend, Rachel has never felt more on the outside. The one person who reached out to her, Jessie, also happens to be part of the popular crowd that lives to torment outsiders like her. But Rachel has something else that separates her from the rest, a secret amazing ability to move things with her mind. Sue Snell, the only survivor of Carrie White's rampage twenty-two years ago, may hold the key to helping Rachel come to terms with her awesome, but unwanted power. But as Rachel slowly learns to trust, a terrible trap is being laid for her. And making her angry could prove to be fatal.|
|tt0187819|**Doug's 1st Movie** (1999)|**You'll Dig Doug!**<br/>[Animation, Family, Comedy]<br/><br/>Doug and his pal Skeeter set's out to find the monster of Lucky Duck Lake. Though things get really out of hand when some one blurts out that the monster is real.|
|tt0116932|**Love, etc.** (1996)|[]<br/><br/>A triangle: love, obsession, and choice. Pierre, a ladies' man who has little cash and no fixed residence, describes his best friend Benoît as the world's oldest 32-year-old. The shy, well-employed Benoît's life changes when he answers the personal ad of Marie, a 25-year-old who restores paintings. He's attracted to her and she likes his steady calm and his honest attention. They're soon a couple, and they include Pierre in their dinners, outings, and trips. What will happen when Pierre realizes that he too is in love with Marie?|
|tt0119219|**Goodbye Lover** (1998)|**No one's ever been so good at being so bad.**<br/>[Thriller, Comedy, Crime, Mystery]<br/><br/>Police investigate when a man having an affair with his brother's wife disappears suddenly.|Patricia Arquette as Sandra Dunmore, Dermot Mulroney as Jake Dunmore, Mary-Louise Parker as Peggy Blane
|tt0188996|**Photographer** (1998)|[TV Movie, History, Documentary, War]<br/><br/>In 1987, colour slides were found in a second hand book store in Vienna which turned out to be a collections of photographs taken in the Lodz ghetto by the Nazis' chief accountant. Walter Genewein boosted productivity in the ghetto while keeping costs down, a policy which led to the Lodz ghetto surviving much longer than any other in Poland. He recorded what he considered to be the subhuman aspect of the Jewish workers and he was concerned only with the technical quality of his photos.  Director Dariusz Jabłoński's prize-winning film uses the photographs in a different way. He recreates for us the suffering of inmates, giving a compassionate picture of that it was like to be trapped in the ghetto. (Storyville)|
|tt0037098|**The Mummy's Curse** (1944)|**HANDS CREEPING LIKE COBRAS! EYES CRAWLING WITH MADNESS! THE MUMMY RUNS AMOK!**<br/>[Horror]<br/><br/>After being buried in quicksand for the past 25 years, Kharis is set free to roam the rural bayous of Louisiana, as is the soul of his beloved Princess Ananka, still housed in the body of Amina Mansouri, who seeks help and protection at a swamp draining project.|Lon Chaney Jr. as Kharis
|tt0035899|**Frankenstein Meets the Wolf Man** (1943)|[Fantasy, Horror, Science Fiction]<br/><br/>Graverobbers open the grave of the Wolfman and awake him. He doesn't like the idea of being immortal and killing people when the moon is full, so he tries to find Dr. Frankenstein, in the hopes that the Dr. can cure him, but Frankenstein is dead and only his Monster is alive and this one wants to live, not to die like the Wolfman.|Lon Chaney Jr. as Lawrence Stewart Talbot, the wolfman, Bela Lugosi as The Frankenstein Monster
|tt0084156|**It Came from Hollywood** (1982)|**We're going to make you LAUGH.**<br/>[Comedy, Documentary]<br/><br/>Dan Ackroyd, John Candy, Gilda Radner and Cheech and Chong present this compilation of classic bad films from the 50's, 60's and 70's. Special features on gorilla pictures, anti-marijuana films and a special tribute to the worst film maker of all-time, Ed Wood.|Dan Aykroyd as Himself, John Candy as Himself, Cheech Marin as Himself
|tt0052151|**Run Silent, Run Deep** (1958)|**Gable and Lancaster make the seas boil in the battle adventure that hits like a torpedo!**<br/>[Drama, War]<br/><br/>The captain of a submarine sunk by the Japanese during WWII is finally given a chance to skipper another sub after a year of working a desk job. His singleminded determination for revenge against the destroyer that sunk his previous vessel puts his new crew in unneccessary danger.|Jack Warden as Yeoman 1st Class Mueller
|tt0139809|**The Thirteenth Floor** (1999)|**Question reality. You can go there even though it doesn't exist.**<br/>[Thriller, Science Fiction, Mystery]<br/><br/>Computer scientist Hannon Fuller has discovered something extremely important. He's about to tell the discovery to his colleague, Douglas Hall, but knowing someone is after him, the old man leaves a letter in his computer generated parallel world that's just like the 30's with seemingly real people with real emotions.|Armin Mueller-Stahl as Hannon Fuller/Grierson, Vincent D'Onofrio as Jason Whitney/Jerry Ashton
|tt0128278|**Instinct** (1999)|[Drama, Mystery, Thriller]<br/><br/>Dr. Ethan Powell, an anthropologist, is in Africa studying apes when he is lost for two years. When he is found, he kills 3 men and puts 2 in the hospital. Cuba Gooding's character is a psychiatrist who wants to take up the task of trying to get Dr. Powell to speak again and maybe even stand judgment at a trial for his release from prison of mental cases. Along the way, Cuba has to deal with also helping the mental patients that are being abused and neglected. In this process Cuba learns a few things about himself and life, and so does Anthony Hopkins character, Dr. Powell.|Anthony Hopkins as Dr. Ethan Powell, Cuba Gooding Jr. as Dr. Theo Caulder, Donald Sutherland as Ben Hillard, George Dzundza as Dr. John Murray, Rex Linn as Gurad Alan
|tt0141105|**Free Enterprise** (1998)|**Love long and party**<br/>[Comedy, Romance]<br/><br/>Young filmmakers trying to hawk a movie titled "Bradykillers" about a serial killer who goes after victims Marcia, Jan, and Cindy meet their screen idol, William Shatner. The two young men, who idolize him and in their fantasies have seen him as a shadowy fairy godfather figure, are alarmed at the reality of the middle-aged non-Captain Kirk man that they meet.|William Shatner as Bill
|tt0130827|**Run Lola Run** (1998)|**Every second of every day you're faced with a decision that can change your life.**<br/>[Action, Drama, Thriller]<br/><br/>Lola receives a phone call from her boyfriend Manni. He lost 100,000 DM in a subway train that belongs to a very bad guy. She has 20 minutes to raise this amount and meet Manni. Otherwise, he will rob a store to get the money. Three different alternatives may happen depending on some minor event along Lola's run.|
|tt0158983|**South Park: Bigger, Longer & Uncut** (1999)|**UH-OH.**<br/>[Animation, Comedy, Music]<br/><br/>When the four boys see an R-rated movie featuring Canadians Terrance and Philip, they are pronounced "corrupted", and their parents pressure the United States to wage war against Canada.|George Clooney as Dr. Gouache, Minnie Driver as Brooke Shields, Brent Spiner as Conan O'Brien, Eric Idle as Dr. Vosknocker
|tt0163651|**American Pie** (1999)|**There's nothing like your first piece.**<br/>[Comedy, Romance]<br/><br/>At a high-school party, four friends find that losing their collective virginity isn't as easy as they had thought. But they still believe that they need to do so before college. To motivate themselves, they enter a pact to all "score." by their senior prom.|Eugene Levy as Jim's Father
|tt0185937|**The Blair Witch Project** (1999)|**The scariest movie of all time is a true story.**<br/>[Horror, Mystery]<br/><br/>In October of 1994 three student filmmakers disappeared in the woods near Burkittsville, Maryland, while shooting a documentary. A year later their footage was found.|
|tt0157503|**Drop Dead Gorgeous** (1999)|**The battle between the good and the bad is bound to get ugly.**<br/>[Comedy]<br/><br/>In a small Minnesota town, the annual beauty pageant is being covered by a TV crew. Former winner Gladys Leeman wants to make sure her daughter follows in her footsteps; explosions, falling lights, and trailer fires prove that. As the Leemans are the richest family in town, the police are pretty relaxed about it all. Despite everything, main rival (but sweet) Amber Atkins won't give up without a fight.|Kirsten Dunst as Amber Atkins, Ellen Barkin as Annette Atkins, Allison Janney as Loretta, Matt Malloy as John Dough, Judge #1
|tt0171363|**The Haunting** (1999)|**Some houses are born bad.**<br/>[Horror, Thriller, Fantasy, Mystery]<br/><br/>Dr. David Marrow invites Nell Vance, and Theo and Luke Sanderson to the eerie and isolated Hill House to be subjects for a sleep disorder study. The unfortunate guests discover that Marrow is far more interested in the sinister mansion itself – and they soon see the true nature of its horror.|Liam Neeson as Dr. David Marrow, Lili Taylor as Eleanor 'Nell' Vance, Bruce Dern as Mr. Dudley
|tt0049406|**The Killing** (1956)|**...Like No Other Picture Since "Scarface" and "Little Caesar"!**<br/>[Drama, Action, Thriller, Crime]<br/><br/>The Killing was Stanley Kubrick’s first film with a professional cast and the first time he achieved public recognition as the unconventional director he’s now known for. The story is of ex-prisoners who plan to set up a racetrack so they can live a life without monetary worries. One of the more exceptional films of the 1950’s.|Arthur Tovey as Racetrack Cashier (uncredited)
|tt0056193|**Lolita** (1962)|**How did they ever make a movie of ...**<br/>[Drama, Romance]<br/><br/>Humbert Humbert is a middle-aged British novelist who is both appalled by and attracted to the vulgarity of American culture. When he comes to stay at the boarding house run by Charlotte Haze, he soon becomes obsessed with Lolita, the woman's teenaged daughter.|
|tt0088939|**The Color Purple** (1985)|**It's about life. It's about love. It's about us.**<br/>[Drama]<br/><br/>An epic tale spanning forty years in the life of Celie (Whoopi Goldberg), an African-American woman living in the South who survives incredible abuse and bigotry.  After Celie's abusive father marries her off to the equally debasing "Mister" Albert Johnson (Danny Glover), things go from bad to worse, leaving Celie to find companionship anywhere she can.  She perseveres, holding on to her dream of one day being reunited with her sister in Africa.  Based on the novel by Alice Walker.|Whoopi Goldberg as Celie, Danny Glover as Albert, Dana Ivey as Miss Millie, Laurence Fishburne as Swain
|tt0093051|**From the Hip** (1987)|**Getting To The Top Means Working Like A Dog!**<br/>[Comedy, Drama]<br/><br/>Apprentice lawyer Robin Weathers turns a civil suit into a headline grabbing charade. He must reexamine his scruples after his shenanigans win him a promotion in his firm, and he must now defend a college professor who is appearantly guilty of murder.|John Hurt as Douglas Benoit
|tt0094293|**Wanted: Dead or Alive** (1987)|**Nick Randall is a loner. A legend. A bounty hunter. He's the best there is at the job he hates.**<br/>[Action, Thriller, Crime, Drama]<br/><br/>This movie features a character who is supposed to be the descendant of the character played Steve McQueen in the television series of the same name. And like McQueen's Josh Randall, Hauer's Nick Randall is also a bounty. But also an ex-CIA operative, who is asked by his former employer to help them track down a terrorrist, Malak Al Rahim, who is in the country, and has already made a move. But he is also looking for Randall, and the people, whom Randall is working for, is telling Malak, where he can find Randall.|
|tt0131325|**Bowfinger** (1999)|**The con is on.**<br/>[Comedy]<br/><br/>On the verge of bankruptcy and desperate for his big break, aspiring filmmaker Bobby Bowfinger concocts a crazy plan to make his ultimate dream movie. Rallying a ragtag team that includes a starry-eyed ingenue, a has-been diva and a film studio gofer, he sets out to shoot a blockbuster featuring the biggest star in Hollywood, Kit Ramsey -- only without letting Ramsey know he's in the picture.|Steve Martin as Bobby Bowfinger, Eddie Murphy as Kit Ramsey | Jeffernson 'Jiff' Ramsey, Heather Graham as Daisy, Christine Baranski as Carol, Terence Stamp as Terry Stricter, Robert Downey Jr. as Jerry Renfro
|tt0168987|**Better Than Chocolate** (1999)|**A delicious romantic comedy.**<br/>[Drama, Romance]<br/><br/>Not long after moving into her own place, Maggie finds herself with two unsolicited roommates: her recently divorced mother, Lila, and her young brother. The timing is especially bad, considering Maggie has fallen hard for an attractive woman, Kim, only hours before they move in. What could be a nonissue becomes increasingly complicated -- since Maggie's family is unaware of her sexual orientation, and Maggie is not open to sharing that information.|
|tt0119614|**Marcello Mastroianni: I Remember** (1997)|[Documentary]<br/><br/>In 1996, Marcello Mastroianni talks about life as an actor. It's an anecdotal and philosophical memoir, moving from topic to topic, fully conscious of a man "of a certain age" looking back. He tells stories about Fellini and De Sica's direction, of using irony in performances, of constantly working (an actor tries to find himself in characters). He's diffident about prizes, celebrates Rome and Paris, salutes Naples and its people. He answers the question, why make bad films; recalls his father and grandfather, carpenters, his mother, deaf in her old age, and his brother, a film editor; he's modest about his looks. In repose, time's swift passage holds Mastroianni inward gaze.|
|tt0089670|**National Lampoon’s European Vacation** (1985)|**For over two thousand years, Europe has survived many great disasters. Now for the real test. Chevy Chase & his family are coming from America!**<br/>[Adventure, Comedy, Romance]<br/><br/>The Griswalds win a vacation to Europe on a game show, and thus pack their bags for the continent. They do their best to catch the flavor of Europe, but they just don't know how to be be good tourists. Besides, they have trouble taking holidays in countries where they CAN speak the language.|Chevy Chase as Clark Griswold, Beverly D'Angelo as Ellen Griswold, Eric Idle as The Bike Rider
|tt0176269|**Universal Soldier: The Return** (1999)|**Prepare to become obsolete**<br/>[Action, Science Fiction]<br/><br/>Luc Deveraux, the heroic former Universal Soldier, is about to be thrown into action once again. When Seth (Michael Jai White), the supercomputer controlled ultra-warrior, decides to take revenge and destroy its creators, only Luc can stop it. All hell breaks loose as Luc battles Seth and a deadly team of perfect soldiers in a struggle that pits man against machine and good against evil.|Jean-Claude Van Damme as Luc Devereux, Xander Berkeley as Dr. Dylan Cotner, Daniel von Bargen as General Radford
|tt0105698|**Universal Soldier** (1992)|**The future has a bad attitude.**<br/>[Thriller, Action, Science Fiction, Crime]<br/><br/>An American soldier who had been killed during the Vietnam War is revived 25 years later by the military as a semi-android, UniSols, a high-tech soldier of the future. After the failure of the initiative to erase all the soldier's memories, he begins to experience flashbacks that are forcing him to recall his past.|Jean-Claude Van Damme as Luc Devreux/GR44, Rance Howard as John Devreux
|tt0188863|**Love Stinks** (1999)|[Comedy, Romance]<br/><br/>A movie about a relationship...that's worse than yours. Seth (Stewart), a sitcom writer-producer, meets Chelsea (Wilson), an interior decorator, at his best friend's (Bellamy) wedding. He's immediately sexually attracted to her while she's instantly attracted to his single-ness. They both ditch their wedding dates and start their own date that same night. The two become a couple, appearing very happy until after a couple of years of postponing a marriage proposal. When Chelsea realizes that Seth wants to remain single and together, she becomes quite bitter. In the next hour of the movie, the two engage in behavior that makes the War of the Roses look like child's play.|Bridgette Wilson as Chelsea Turner, Colleen Camp as Monica Harris
|tt0119692|**With Friends Like These...** (1998)|[Horror, Science Fiction]<br/><br/>Four small-time two-bit character actors, all close friends, are competing for same important part in the next Martin Scorsese mob film.|Bill Murray as Maurice Melnick, David Strathairn as , Beverly D'Angelo as 
|tt0163185|**Rosie** (1998)|[]<br/><br/>In a juvenile lockup, a new ward answers questions: she's Rosie, 13, no parents, a sister Irene, a brother Michel. In flashbacks we find out what happened. She lives with Irene, who's 27, whom Rosie knows is in fact her mother, but that's their secret. Irene's brother Michel, unemployed, a compulsive gambler, comes to stay with them. Around then, Irene meets Bernard; they come to care for each other. This leaves Rosie without attention, so she puts all her adolescent hopes and romantic fantasies into a relationship with Jimi, a good looking kid she sees on a bus. Is it adventures with Jimi that land her in juvie? Once she's there, why doesn't he answer her letters?|
|tt0160216|**A Dog Of Flanders** (1999)|[Drama, Family]<br/><br/>Poor but happy, young Nello and his grandfather live alone, delivering milk as a livelihood, in the outskirts of Antwerp, a city in Flanders (the Flemish or Dutch-speaking part of modern-day Belgium). They discover a beaten dog (a Bouvier, a large sturdy dog native to Flanders) and adopt it and nurse it back to health, naming it Patrasche, the middle name of Nello's mother Mary, who died when Nello was very young. Nello's mother was a talented artist, and like his mother, he delights in drawing, and his friend Aloise is his model and greatest fan and supporter.|Jack Warden as Jehan, Jon Voight as Michel, Bruce McGill as William
|tt0119586|**Lucie Aubrac** (1997)|[Drama]<br/><br/>A love story or a tale of the resistance, this poignant movie tells both the haunting story of a French resistance cell in Lyon but also the love of Lucie Aubrac for her husband...|Daniel Auteuil as Raymond Aubrac
|tt0163579|**Chill Factor** (1999)|**This Fall, Action is served on the rocks, with a twist.**<br/>[Action, Comedy, Thriller]<br/><br/>A store clerk and an ice cream truck driver are thrown together when a dying scientist entrusts them with a deadly chemical kept in ice. This chemical will kill every living thing once it melts. They have to take the chemical codenamed 'Elvis' to the next nearest military base while being chased by terrorists who want it to hold the country for ransom.|Cuba Gooding Jr. as Arlo, Skeet Ulrich as Tim Mason, David Paymer as Dr. Richard Long, Kevin J. O'Connor as Telstar
|tt0125971|**Outside Providence** (1999)|**Tim's getting a future... whether he likes it or not!**<br/>[Romance, Comedy, Drama]<br/><br/>In this coming-of-age comedy, Tim Dunphy is leading a go-nowhere existence, spending his days smoking pot and hanging out with his best friend, Drugs Delaney. But Tim's lazy days of getting high are jettisoned after a brush with the law convinces his blue-collar dad to send him to a Connecticut prep school. The one saving grace of the new school is Jane, a fellow student Tim falls for immediately.|Alec Baldwin as Old Man Dunphy, Richard Jenkins as Barney, George Wendt as Joey, George Martin as Dean Mort
|tt0157183|**West Beyrouth** (1998)|[Romance, Drama, Comedy]<br/><br/>In April, 1975, civil war breaks out; Beirut is partitioned along a Moslem-Christian line. Tarek is in high school, making Super 8 movies with his friend, Omar. At first the war is a lark: school has closed, the violence is fascinating, getting from West to East is a game. His mother wants to leave; his father refuses. Tarek spends time with May, a Christian, orphaned and living in his building. By accident, Tarek goes to an infamous brothel in the war-torn Olive Quarter, meeting its legendary madam, Oum Walid. He then takes Omar and May there using her underwear as a white flag for safe passage. Family tensions rise. As he comes of age, the war moves inexorably from adventure to tragedy.|
|tt0045251|**Othello** (1951)|**Orson Welles' magnificent screening of Shakespeare's immortal tragedy**<br/>[Drama, Romance]<br/><br/>Desdemona, daughter of a Venetian aristocrat, elopes with Moorish military hero Othello, to the great resentment of Othello's envious underling Iago. Alas, Iago knows Othello's weakness, and with chilling malice works on him with but too good effect...|Joseph Cotten as Senator (uncredited)
|tt0088178|**Stop Making Sense** (1984)|**Why stop making sense? Why a movie? Why a big suit? Where do the odd movements come from? What will the band do next?**<br/>[Documentary, Music]<br/><br/>David Byrne walks onto the stage and does a solo "Psycho Killer". Jerry Harrison, Tina Weymouth and Chris Frantz join him for two more songs. The crew is busy, still setting up. Then, three more musicians and two back-up singers join the band. Everybody sings, plays, harmonizes, dances, and runs. In this concert film, the Talking Heads hardly talk, don't stop, and always make sense.|
|tt0127296|**Splendor** (1999)|**Explore the possibilities...**<br/>[Comedy]<br/><br/>Veronica is a white-bread beauty searching for a good man in Los Angeles. While slam dancing at a Halloween rave, she meets Abel, a sensitive poet. Then she meets Zed, a supersexy tattooed drummer with incredible biceps. Who will she choose? Does she go for true love or cheap sex? She can't decide so she chooses both. But after managing to nurture a picture-perfect threesome, along comes Ernest, a rich movie director with deep baby blues that sweep Veronica off her feet. What's a girl to do now?|
|tt0173390|**Sugar Town** (1999)|[Comedy]<br/><br/>Look at the lives of struggling L.A. scene rock stars follows main character, Gwen, on her quest for the top. Working as an assistant to a film production designer, she tries to steal her boy friend who is a music producer by offering sexual favors. The producer meanwhile is trying to orchestrate a comeback for a former glam band comprised of Michael Des Barres, John Taylor and Martin Kemp. Rosanna Arquette plays the former movie star wife of the lead singer, who is fretting because she has just been offered the role as the mother of one of the new ingenious. Beverly D'Angelo also shows up as a millionairess who agrees to bankroll the group, but only if she gets a roll in the hay with the lead singer. All of the career problems, including drug proclivity, are represented in this film.|Lucinda Jenney as Kate
|tt0089175|**Fright Night** (1985)|**There are some very good reasons to be afraid of the dark.**<br/>[Horror]<br/><br/>Nobody believes teenager Charley Brewster (William Ragsdale) when he discovers that his suave new neighbor, Jerry Dandrige (Chris Sarandon), is a vampire. So when the bloodsucker starts stalking Charley, he turns to has-been actor Peter Vincent (Roddy McDowall), famed for portraying a ghoul hunter. Unfortunately for the would-be vampire slayers, Dandrige has set his sights on Charley's girlfriend (Amanda Bearse) in this clever spoof of the horror genre.|Roddy McDowall as Peter Vincent, Art Evans as Detective Lennox
|tt0159421|**The Adventures of Elmo in Grouchland** (1999)|**The Good, The Bad and The Stinky.**<br/>[Family]<br/><br/>Elmo loves his fuzzy, blue blanket, and would never let anything happen to it. However, a tug-of-war with his friend Zoe sends his blanket to a faraway land, and Elmo in hot pursuit. Facing life without his cherished blanket, Elmo musters all of his determination and courage and heads off on an action-packed rescue mission that plunges him into Grouchland-a place full of grouchy creatures, stinky garbage and the villainous Huxley. Along the way, Elmo learns an important lesson about sharing, realizing that he was selfish with his friend and responsible for what happened.|Frank Oz as Bert / Grover / Cookie Monster (voice)
|tt0120188|**Three Kings** (1999)|**It's good to be King.**<br/>[Action, Adventure, Comedy]<br/><br/>A group of American soldiers stationed in Iraq at the end of the Gulf War find a map they believe will take them to a huge cache of stolen Kuwaiti gold hidden near their base, and they embark on a secret mission that's destined to change everything.|George Clooney as Archie Gates, Mark Wahlberg as Troy Barlow, Mykelti Williamson as Colonel Horn
|tt0162360|**Happy, Texas** (1999)|**Escaped convicts disguised as beauty pageant experts? This could get ugly**<br/>[Comedy, Crime]<br/><br/>Two escaped convicts roll into the village of Happy, Texas, where they're mistaken for a gay couple who work as beauty pageant consultants. They go along with it to duck the police, but the local sheriff has a secret of his own.|Steve Zahn as Wayne Wayne Wayne Jr., William H. Macy as Sheriff Chappy Dent, Jeremy Northam as Harry Sawyer, Ron Perlman as Marshal Nalhober, Illeana Douglas as Doreen Schaefer, Paul Dooley as The Judge
|tt2378428|**Romance** (2012)|[Romance, Drama, Comedy]<br/><br/>She's as hot as Britney Spears. Hotter. She parties like Hollywood. She's way out of his league. Or that's the story he's sticking to.|
|tt0106664|**The Dark Half** (1993)|**There are very good reasons to be afraid of the dark.**<br/>[Mystery, Horror]<br/><br/>Thad Beaumont is the author of a highly successful series of violent pulp thrillers written under the pseudonym of ‘George Stark’, but when he decides to ‘kill-off’ his alter-ego in a mock ceremony, it precipitates a string of sadistic murders matching those in his pulp novels, which are soon discovered to be the work of Stark himself. Looking like a maniacal version of his counterpart, Stark is not so willing to quit the writing game – even if it means coming after Thad's wife and their baby.|Michael Rooker as Sheriff Alan Pangborn, Beth Grant as Shayla Beaumont
|tt0167427|**Superstar** (1999)|**Dare to dream.**<br/>[Comedy, Family]<br/><br/>Orphan Mary Katherine Gallagher, an ugly duckling at St. Monica High School, has a dream: to be kissed soulfully. She decides she can realize this dream if she becomes a superstar, so her prayers, her fantasies, and her conversations with her only friend focus on achieving super-stardom.|
|tt0086200|**Risky Business** (1983)|**Time of your life, Huh Kid.**<br/>[Comedy, Romance]<br/><br/>Meet Joel Goodson, an industrious, college-bound 17-year-old and a responsible, trustworthy son. However, when his parents go away and leave him home alone in the wealthy Chicago suburbs with the Porsche at his disposal he quickly decides he has been good for too long and it is time to enjoy himself. After an unfortunate incident with the Porsche Joel must raise some cash, in a risky way.|Tom Cruise as Joel Goodsen, Sean Penn as Cameo (uncredited), Joe Pantoliano as Guido, Richard Masur as Rutherford
|tt0082979|**Reds** (1981)|**Not since Gone With The Wind has there been a great romantic epic like it!**<br/>[Drama, History]<br/><br/>A radical American journalist becomes involved with the Communist revolution in Russia and hopes to bring its spirit and idealism to the United States.|Warren Beatty as John Reed, Diane Keaton as Louise Bryant, Jack Nicholson as Eugene O'Neill, Paul Sorvino as Louis Fraina, M. Emmet Walsh as Speaker - Liberal Club, Gene Hackman as Pete Van Wherry, Miriam Margolyes as Woman writing in Notebook
|tt0033804|**The Lady Eve** (1941)|**Barbara Stanwyck has Henry Fonda Bewitched and Bewildered**<br/>[Comedy, Romance]<br/><br/>Returning from a year up the Amazon studying snakes, the rich but unsophisticated Charles Pike meets con-artist Jean Harrington on a ship. They fall in love, but a misunderstanding causes them to split on bad terms. To get back at him, Jean disguises herself as an English lady, and comes back to tease and torment him.|Henry Fonda as Charles Pike, Bert Moorhouse as Diner on Ship (uncredited)
|tt0038559|**Gilda** (1946)|**There NEVER was a woman like Gilda!**<br/>[Drama, Romance, Thriller]<br/><br/>Just arrived in Argentina, small-time crooked gambler Johnny Farrell is saved from a gunman by sinister Ballin Mundson, who later makes Johnny his right-hand man. But their friendship based on mutual lack of scruples is strained when Mundson returns from a trip with a wife: the supremely desirable Gilda, whom Johnny once knew and learned to hate. The relationship of Johnny and Gilda, a battlefield of warring emotions, becomes even more bizarre after Mundson disappears...|
|tt0052225|**South Pacific** (1958)|**There is nothing you can name that is anything like... [South Pacific]**<br/>[Music, Romance, War]<br/><br/>Can a girl from Little Rock find happiness with a mature French planter she got to know one enchanted evening away from the military hospital where she is a nurse? Or should she just wash that man out of her hair? Bloody Mary is the philosopher of the island and it's hard to believe she could be the mother of Liat who has captured the heart of Lt. Joseph Cable USMC. While waiting for action in the war in the South Pacific, sailors and nurses put on a musical comedy show. The war gets closer and the saga of Nellie Forbush and Emile de Becque becomes serious drama.|
|tt0085549|**Flashdance** (1983)|**When the dancer becomes the dance.**<br/>[Drama, Music, Romance]<br/><br/>The popular 1980’s dance movie that depicts the life of an exotic dancer with a side job as a welder who true desire is to get into ballet school. It’s her dream to be a professional dancer and now is her chance. The film has a great soundtrack with an Oscar winning song.|
|tt0119256|**Hard Eight** (1996)|**When good luck is a long shot, you have to hedge your bets.**<br/>[Drama, Thriller]<br/><br/>A stranger mentors a young Reno gambler who weds a hooker and befriends a vulgar casino regular.|Philip Baker Hall as Sydney Brown, Gwyneth Paltrow as Clementine, John C. Reilly as John Finnegan, Samuel L. Jackson as Jimmy, Philip Seymour Hoffman as Young craps player
|tt0017423|**Sparrows** (1926)|[Drama]<br/><br/>Evil Mr.Grimes keeps a rag-tag bunch orphans on his farm deep in a swamp in the US South. He forces them to work in his garden and treats them like slaves. They are watched over by the eldest, Molly. A gang in league with Mr. Grimes kidnaps Doris, the beautiful little daughter of a rich man, and hides her out on Grimes' farm, awaiting ransom. When the police close in, and Mr. Grimes threatens to throw Doris into the bottomless mire, Molly must lead her little flock out through the alligator-infested swamp.|
|tt0048977|**The Bad Seed** (1956)|**A WOMAN'S SHAME...Out in the Open!**<br/>[Crime, Drama, Horror, Thriller]<br/><br/>Young Rhoda Penmark is sweet, smart … and inherently evil! After a school chum dies during a picnic, no one suspects Rhoda, except the janitor of her apartment building. But when Rhoda's mother finds out that her own mother was a cold-blooded killer, she begins to suspect Rhoda might be the victim of some faulty genetics.|
|tt0081633|**Time Bandits** (1981)|**All the dreams you've ever had and not just the good ones.**<br/>[Family, Fantasy, Science Fiction, Adventure, Comedy]<br/><br/>Young history buff Kevin can scarcely believe it when six dwarfs emerge from his closet one night. Former employees of the Supreme Being, they've purloined a map charting all of the holes in the fabric of time and are using it to steal treasures from different historical eras. Taking Kevin with them, they variously drop in on Napoleon, Robin Hood and King Agamemnon before the Supreme Being catches up with them.|Sean Connery as King Agamemnon / Fireman, John Cleese as Robin Hood, Shelley Duvall as Dame Pansy / Pansy, Ian Holm as Napoleon, Peter Vaughan as Winston the Ogre, David Warner as Evil Genius, Kenny Baker as Fidgit, Jim Broadbent as Compere
|tt0142201|**Crazy in Alabama** (1999)|[Comedy, Drama]<br/><br/>An abused wife heads to California to become a movie star while her nephew back in Alabama has to deal with a racially-motivated murder involving a corrupt sheriff.|Melanie Griffith as Lucille Vinson, David Morse as Dove Bullis, Cathy Moriarty as Earlene Bullis, Rod Steiger as Judge Louis Mead, Richard Schiff as Norman the Chauffeur
|tt0172627|**Body Shots** (1999)|**Tonight it's for sex. Tomorrow it's for real.**<br/>[Drama, Romance]<br/><br/>The Los Angeles club scene is a place of booze-fueled decadence and debauchery. In a night full of possibilities, eight 20-somethings take to the clubs seeking good times, companionship and maybe a little sex. But in the harsh light of the morning after, their worlds are thrown into a spin of confusion when hungover Sara accuses hard-partying Mike of date rape. Loyalties are tested as each among them is forced to take sides.|
|tt0059319|**The Ipcress File** (1965)|**The spy story of the century.**<br/>[Thriller]<br/><br/>This espionage thriller represents a landmark in spy movies introducing the sly, dry intelligence agent Harry Palmer. The story, centers on Palmer's investigation into British Intelligence security. He's soon enmeshed in a world of double-dealing, kidnap and murder and finds a traitor is operating at the heart of the secret service. Will the mysterious 'Ipcress File' reveal who the traitor is?|
|tt0097742|**Licence to Kill** (1989)|**His bad side is a dangerous place to be.**<br/>[Adventure, Action, Thriller]<br/><br/>James Bond and his American colleague Felix Leiter arrest the drug lord Sanchez who succeeds in escaping and takes revenge on Felix and his wife. Bond knows but just one thing: revenge.|Benicio del Toro as Dario, Desmond Llewelyn as Q, Grand L. Bush as Hawkins
|tt0181288|**American Movie** (1999)|[Documentary]<br/><br/>AMERICAN MOVIE is the story of filmmaker Mark Borchardt, his mission, and his dream. Spanning over two years of intense struggle with his film, his family, financial decline, and spiritual crisis, AMERICAN MOVIE is a portrayal of ambition, obsession, excess, and one man's quest for the American Dream.|
|tt0117365|**Portraits Chinois** (1996)|[Drama, Comedy]<br/><br/>Ada and Lise are both costume designers, the first is around 20, the other around 30. Both are working hard on their break through. There are also jobs for the movies. This is where Lise meets producer Alphonse, who is nearly 20 years older than she. Because he is unhappy with his girlfriend a secret relationship evolves. Ada has problems as well, but she's not the only one. There are also the young Emma and Nina, as well as Yves and Guido - enough people to get into complicated relationship entanglements.|Helena Bonham Carter as Ada
|tt0082121|**Bustin' Loose** (1981)|**He's mad, he's bad, and he's Bustin' Loose!**<br/>[Adventure, Comedy]<br/><br/>After ex-con Joe Braxton violates his probation he is given a second chance, all he has to do is drive a group of special kids across the country. What could possibly go wrong?|
|tt0089885|**Re-Animator** (1985)|**Herbert West has a good head on his shoulders... and another on his desk.**<br/>[Science Fiction, Comedy, Horror]<br/><br/>A dedicated student at a medical college and his girlfriend become involved in bizarre experiments centering around the re-animation of dead tissue when an odd new student arrives on campus.|Jeffrey Combs as Herbert West
|tt0100475|**Repossessed** (1990)|**The Devil's back. But he's never dealt with an exorcist like this!**<br/>[Comedy, Horror]<br/><br/>It's been some time since Father Jebedaiah Mayii exorcised the Devil from little Nancy Aglet, who is now grown up with a husband and two children of her own. But the prince of darkness wants to go a second round and has returned to repossess her! With Father Mayii unwilling to help, Father Luke Brophy tries his best to help Nancy, even when TV's Ernest Weller plans to air the exorcism live on TV.|Ned Beatty as Ernest Weller, Leslie Nielsen as Father Jebedaiah Mayii, Ian Abercrombie as Ice Tea Waiter
|tt0050371|**A Face in the Crowd** (1957)|**POWER! He loved it! He took it raw in big gulpfuls...he liked the taste, the way it mixed with the bourbon and the sin in his blood!**<br/>[Drama]<br/><br/>"A Face in the Crowd" charts the rise of a raucous hayseed named Lonesome Rhodes from itinerant Ozark guitar picker to local media rabble-rouser to TV superstar and political king-maker. Marcia Jeffries is the innocent Sarah Lawrence girl who discovers the great man in a back-country jail and is the first to fall under his spell|Walter Matthau as Mel Miller
|tt0087700|**Meatballs Part II** (1984)|**The insanity continues...**<br/>[Comedy]<br/><br/>The second in-name-only sequel to the first Meatballs summer camp movie sets us at Camp Sasquash where the owner Giddy tries to keep his camp open after it's threatened with foreclosure after Hershey, the militant owner of Camp Patton located just across the lake, wants to buy the entire lake area to expand Camp Patton. Giddy suggests settling the issue with the traditional end-of-the-summer boxing match over rights to the lake. Meanwhile, a tough, inner city punk, nicknamed Flash, is at Camp Sasquash for community service as a counselor-in-training where he sets his sights on the naive and intellectual Cheryl, while Flash's young charges befriend an alien, whom they name Meathead, also staying at the camp for the summer.|Paul Reubens as Albert/Hara Krishna
|tt0149691|**Anywhere But Here** (1999)|**A story of a mother who knows best... and a daughter who knows better.**<br/>[Drama, Comedy, Romance]<br/><br/>In this adaptation of the Mona Simpson novel, single mother Adele August is bad with money, and even worse when it comes to making decisions. Her straight-laced daughter, Ann, is a successful high school student with Ivy League aspirations. When Adele decides to pack up and move the two of them from the Midwest to Beverly Hills, Calif., to pursue her dreams of Hollywood success, Ann grows frustrated with her mother's irresponsible and impulsive ways.|Susan Sarandon as Adele August, John Diehl as Jimmy, Bonnie Bedelia as Carol, John Carroll Lynch as Jack Irwin, Caroline Aaron as Gail Letterfine
|tt0120655|**Dogma** (1999)|**It can be Hell getting into Heaven**<br/>[Fantasy, Comedy, Adventure]<br/><br/>The latest battle in the eternal war between Good and Evil has come to New Jersey in the late, late 20th Century. Angels, demons, apostles and prophets (of a sort) walk among the cynics and innocents of America and duke it out for the fate of humankind.|Ben Affleck as Bartleby, Matt Damon as Loki, Salma Hayek as Serendipity, Janeane Garofalo as Liz
|tt0190641|**Pokémon: The First Movie: Mewtwo Strikes Back** (1998)|**The Pokémon Match of All Time is Here.**<br/>[Animation]<br/><br/>The adventure explodes into action with the debut of Mewtwo, a bio-engineered Pokemon created from the DNA of Mew, the rarest of all Pokemon. Determined to prove its superiority, Mewtwo lures Ash, Pikachu and others into a Pokemon match like none before. Mewtwo vs. Mew. Super-clones vs. Pokemon. It's the ultimate showdown ... with the very future of the world at stake!|
|tt0165773|**Felicia's Journey** (1999)|[Drama]<br/><br/>A solitary middle-aged bachelor and a naive Irish teenager transform one another's lives to arrive at a place of recognition, redemption and wisdom in Atom Egoyan's adaptation of William Trevor's celebrated 1994 novel. Seventeen and pregnant, Felicia travels to England in search of her lover and is found instead by Joseph Ambrose Hilditch, a helpful catering manager whose kindness masks a serial killer. Hilditch has murdered several young women, but he has no conscious awareness of the crimes; like Felicia, he doesn't see his true self. Felicia's Journey is a story of innocence lost and regained: Felicia awakens to the world's dangers and duplicities; and Hilditch, who grew up lonely and unloved, comes to realize what was taken from him, and what he himself has taken.|Bob Hoskins as Joe Hilditch
|tt0118150|**Where's Marlowe?** (1998)|**A comedy about passion, murder and things that can only happen in the movies.**<br/>[Comedy, Mystery, Thriller]<br/><br/>After making a 3-hour fiasco about New York City's water supply, a two-man film crew decides to take it up a notch by documenting life in the private investigator offices of "Boone and Murphy". Cheating husbands and missing dogs fail to bring in the big bucks however, and after sleeping with the wife of one of their clients Murphy leaves. To stop Boone from having to close down the business the two film-makers must resort to a hands-on approach in the investigations to ensure the completion of their movie.|Miguel Ferrer as Joe Boone
|tt0040522|**Bicycle Thieves** (1948)|**A man and his son search for a stolen bicycle vital for his job.**<br/>[Drama]<br/><br/>Antonio, an unemployed man in the depressed post-WWII economy of Italy, at last finds a good job hanging up posters, the only requirement for which is that he must have his own bicycle. When Antonio's bicycle is stolen, he and his son are forced to walk the streets of Rome in search of it, or else face ruin.|
|tt0050738|**My Man Godfrey** (1957)|**She wanted her breakfast in bed - but not alone.**<br/>[Comedy]<br/><br/>The eccentric Bullock household again need a new butler. Daughter Irene encounters bedraggled Godfrey Godfrey at the docks and, fancying him and noticing his obviously good manners, gets him the job. He proves a great success, but keeps his past to himself. When an old flame turns up Irene's sister Cordelia starts making waves.|Bess Flowers as Party Guest


In [27]:
prompt=f"""
You are assessing movies. Given the query below and the results, assess which movies are relevant for the query and return the output with the ID's using the line: print(collection.prompt_context([<string-list of id's>])) 

Query: {query}

Results:
{collection.prompt_context(result)}
"""

print(prompt)


You are assessing movies. Given the query below and the results, assess which movies are relevant for the query and return the output with the ID's using the line: print(collection.prompt_context([<string-list of id's>])) 

Query: Movies like 'The good the bad and the ugly'

Results:
ID: tt0114885
Title: Waiting to Exhale (1995)
Summary: Friends are the people who let you be yourself... and never let you forget it.. Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusive "good man" to break a string of less-than-stellar lovers. Friends and confidants Vannah, Bernie, Glo and Robin talk it all out, determined to find a better way to breathe.
Genres: [Comedy, Drama, Romance]
Cast: Angela Bassett as Bernadine 'Bernie' Harris, Mykelti Williamson as Troy, Wesley Snipes as James Wheeler

ID: tt0113041
Title: Father of the Bride Part II (1995)
Summary: Just When His World Is Back To Normal... He's In For The Surprise Of His Life!. Just when George Ba

In [28]:
# print(collection.prompt_context(["tt0076759", "tt0120915"]))
print(collection.prompt_context(['tt0058461','tt0064116','tt0068699','tt0061747','tt0105695','tt0065214','tt0065988','tt0067411','tt0068762','tt0108358','tt0044706','tt0064115']))


ID: tt0058461
Title: A Fistful of Dollars (1964)
Summary: In his own way he is perhaps, the most dangerous man who ever lived!. The Man With No Name enters the Mexican village of San Miguel in the midst of a power struggle among the three Rojo brothers and sheriff John Baxter. When a regiment of Mexican soldiers bearing gold intended to pay for new weapons is waylaid by the Rojo brothers, the stranger inserts himself into the middle of the long-simmering battle, selling false information to both sides for his own benefit.
Genres: [Western]
Cast: Clint Eastwood as Joe

ID: tt0064116
Title: Once Upon a Time in the West (1968)
Summary: There were three men in her life. One to take her... one to love her... and one to kill her.. This classic western masterpiece is an epic film about a widow whose land and life are in danger as the railroad is getting closer and closer to taking them over. A mysterious harmonica player joins forces with a desperado to protect the woman and her land.
Genres: